# Analysis

This notebook contains code to analyze putative splicing altering variants in four archaic hominin genomes. The initial code "picks up" after the previous notebook. If you wish to run the analyses below using the full dataframe, load the libraries, navigate to your desired directory, and load the full dataframe starting [here](#startwithdataframe).

I have organized the analyses that follow to largely reflect the order in which they appear in the manuscript. Feel free to navigate to a specific section using the Table of Contents below.

# Table of Contents
- [Load Dataframe](#loaddataframe)
- [Load sQTLs](#loadsqtls)
- [Spliceosome Variants](#spliceosomevariants)
- [Data Description](#datadescription)
    - [Multiple Annotations](#multipleannotations)
    - [Multiple Alleles](#multiplealleles)
    - [Genotypes](#genotypes)
    - [Annotations](#annotations)
    - [Delta Thresholds](#deltathresholds)
    - [Variant Distribution](#variantdistribution)
    - [Delta Correlations](#deltacorrelations)
    - [Multiple Deltas](#multipledeltas)
    - [SAV Genotypes](#SAVgenotypes)
    - [1KG vs gnomAD SAVs](#1KGgnomADSAVs)
    - [SAV Allele Origin](#SAValleleorigin)
    - [Introgression Set Overlap](#introgressionsetoverlap)
    - [SAV Distribution](#SAVdistribution)
- [SAV Genes](#SAVEgenes)
    - [Top 20 AG, AL, DG, and DL](#top20)
    - [N Genes](#ngenes)
    - [N Genes Distribution](#ngenesdistribution)
    - [N Genes by Origin](#ngenesbyorigin)
    - [N Genes by Distribution](#ngenesbydistribution)
    - [Archaic-Specific Genes](#archaicspecificgenes)
    - [Gene Overlap](#geneoverlap)
- [Comparisons](#comparions)
    - [SAV Genes and DR genes](#drgenes)
    - [SAV Genes and Circadian Genes](#circadiangenes)
- [Phenotype Enrichment](#phenotypeenrichment)
- [Novel Isoforms](#novelisoforms)
- [Gene Characteristics](#genecharacteristics)
    - [N Exons, CDS Length, and Gene Length](#physical)
    - [N Isoforms](#isoforms)
    - [Constraint and Conservation](#constraintconservation)
- [Purifying Selection](#purifyingselection)
    - [Varied Deltas](#varieddeltas)
    - [Lineage-Specific](#lineagespecific)
- [SAVs in Moderns](#SAVsinmoderns)
    - [Introgressed SAV Distribution](#introgressedSAVdistribution)
    - [Allele Frequency and Max Delta](#allelefrequencymaxdelta)
    - [Introgressed Genes](#introgressedgenes)
    - [ASE](#ASE)
- [Gene Expression](#geneexpression)
    - [sQTLs](#sqtls)
        - [Allele Origin Differences](#sqtlalleleorigindifferences)
        - [Tissue Specificity](#tissuespecificity)
        - [European sQTLs](#eursqtls)
    - [Gene-Level](#genelevel)
        - [Allele Origin Differences](#genealleleorigindifferences)
- [Genes of Evolutionary Significance](#evolutionarysignificantgenes)
- [Average SAVs in 1KG](#avg1KGSAVs)
- [Introgressed Reference Alleles](#introgressedrefs)
- [Introgressed Fixed Alleles](#introgressedfixed)

Load libraries and change directories.

In [1]:
from pingouin import partial_corr
from math import sqrt
import numpy as np
import pandas as pd
import re
import sys
from scipy.stats import chi2
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 110)

In [2]:
cd ../../data/dataframes

/wynton/group/capra/projects/archaic_splicing/data/dataframes


In [3]:
sys.float_info.min

/wynton/home/capra/cbrand/miniconda3/envs/jupyter/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/wynton/home/capra/cbrand/miniconda3/envs/jupyter/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


2.2250738585072014e-308

# Load Dataframe <a class = 'anchor' id = 'loaddataframe'></a>

In [4]:
#data = pd.read_csv('archaic_data_with_constraint_moderns_introgression.txt', sep='\t', header=0)
#data.head(10)

In [5]:
#len(data)

# Load sQTLs <a class = 'anchor' id = 'loadsqtls'></a>

Let's immediately add our sQTLs and make sure we're only adding sQTLs that match the ref and alt allele. Load the sQTL dataframe.

In [6]:
#sQTLs_header = ['chrom','start','pos','sQTL_ref_allele','sQTL_alt_allele','Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']
#sQTLs = pd.read_csv('../GTEx_sQTLs/sQTLs_hg19.txt', sep = '\t', names = sQTLs_header)
#sQTLs.head(10)

Add a column with the GTEx tissue N per variant.

In [7]:
#sQTLs['N_GTEx_tissues'] = sQTLs[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']].sum(axis=1)
#sQTLs.head(10)

Merge the data using an indicator.

In [8]:
#data = pd.merge(data, sQTLs, on = ['chrom','pos'], how = 'left', indicator = True)
#data = data.drop(columns = ['start'])
#data.head(10)

Classify variants as an sQTL or not depending on whether the position and ref/alt alleles match.

In [9]:
#def sQTL(data):
#    if (data['_merge'] == 'left_only'):
#        return 'no'
#    elif (data['_merge'] == 'both') & (data['ref_allele'] != data['sQTL_ref_allele']) & (data['alt_allele'] != data['sQTL_alt_allele']):
#        return 'no'
#    elif (data['_merge'] == 'both') & (data['ref_allele'] != data['sQTL_ref_allele']) & (data['alt_allele'] == data['sQTL_alt_allele']):
#        return 'no'
#    elif (data['_merge'] == 'both') & (data['ref_allele'] == data['sQTL_ref_allele']) & (data['alt_allele'] != data['sQTL_alt_allele']):
#        return 'no'
#    elif (data['_merge'] == 'both') & (data['ref_allele'] == data['sQTL_ref_allele']) & (data['alt_allele'] == data['sQTL_alt_allele']):
#        return 'yes'

#data['sQTL'] = data.apply(sQTL, axis = 1)

In [10]:
#data.head(10)

Let's also set the GTEx columns to NaN for variants that are not sQTLs.

In [11]:
#data.loc[data['sQTL'] == 'no',['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood','N_GTEx_tissues']] = np.nan

Check that that worked.

In [12]:
#data.groupby(['sQTL','N_GTEx_tissues']).size().to_frame('count')

Drop columns we no longer need.

In [13]:
#data = data.drop(columns = ['sQTL_ref_allele','sQTL_alt_allele','_merge'])
#data.head(10)

Pandas panicked and created lots of duplicates. Let's get our N back to where it should be: 1,607,350.

In [14]:
#data = data.drop_duplicates(['chrom','pos','ref_allele','alt_allele','annotation'])
#len(data)

Finally, while many sQTLs appear to also be variant in 1KG and gnomAD, there appear to be some novel variants. Let's reclassify these from 'archaic-specific' to 'low-confidence ancient' for both Vernot and Browning.

In [15]:
#def new_Vernot_allele_origin(data):
#    if (data['Vernot_allele_origin'] == 'ancient'):
#        return 'ancient'
#    elif (data['Vernot_allele_origin'] == 'introgressed'):
#        return 'introgressed'
#    elif (data['Vernot_allele_origin'] == 'low-confidence ancient'):
#        return 'low-confidence ancient'
#    elif (data['Vernot_allele_origin'] == 'archaic-specific') & (data['sQTL'] == 'yes'):
#        return 'low-confidence ancient'
#    elif (data['Vernot_allele_origin'] == 'archaic-specific') & (data['sQTL'] == 'no'):
#        return 'archaic-specific'

#data['new_Vernot_allele_origin'] = data.apply(new_Vernot_allele_origin, axis = 1)

In [16]:
#def new_Browning_allele_origin(data):
#    if (data['Browning_allele_origin'] == 'ancient'):
#        return 'ancient'
#    elif (data['Browning_allele_origin'] == 'introgressed'):
#        return 'introgressed'
#    elif (data['Browning_allele_origin'] == 'low-confidence ancient'):
#        return 'low-confidence ancient'
#    elif (data['Browning_allele_origin'] == 'archaic-specific') & (data['sQTL'] == 'yes'):
#        return 'low-confidence ancient'
#    elif (data['Browning_allele_origin'] == 'archaic-specific') & (data['sQTL'] == 'no'):
#        return 'archaic-specific'

#data['new_Browning_allele_origin'] = data.apply(new_Browning_allele_origin, axis = 1)

Drop the old columns.

In [17]:
#data = data.drop(columns = ['Vernot_allele_origin'])
#data.rename(columns={'new_Vernot_allele_origin': 'Vernot_allele_origin'}, inplace=True)

In [18]:
#data = data.drop(columns = ['Browning_allele_origin'])
#data.rename(columns={'new_Browning_allele_origin': 'Browning_allele_origin'}, inplace=True)

In [19]:
#data.head(5)

Save this dataframe.

In [20]:
#data.to_csv('archaic_data_with_constraint_moderns_introgression_sQTLs.txt', sep='\t', header = True, index = False)

Add 'n/a' to blank cells for the published dataset.

In [21]:
#data_with_NAs = data.fillna('n/a')

Save this dataframe.

In [22]:
#data_with_NAs.to_csv('archaic_data_with_constraint_moderns_introgression_sQTLs_with_NAs.txt', sep = '\t', header = True, index = False)

**Start here with the full dataframe.** <a class = 'anchor' id = 'startwithdataframe'></a>

In [23]:
data = pd.read_csv('archaic_data_with_constraint_moderns_introgression_sQTLs.txt', sep='\t', header=0)
data.head(10)

/scratch/cbrand/ipykernel_47833/1210275765.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('archaic_data_with_constraint_moderns_introgression_sQTLs.txt', sep='\t', header=0)


,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
0,chr1,861808,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,3479.0,5096.0,0.68,0.55,0.97,0.34,0.83,0.88,0.313840,yes,112289.0,143198.0,0.784152,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.5082,-3.43610,0.89656,0.47484,-0.683,0.0,0.0,0.01,0.0,0.01,-29,-27,24,-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,yes,ancient,ancient
1,chr1,861808,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,3479.0,5096.0,0.68,0.55,0.97,0.34,0.83,0.88,0.313840,yes,112289.0,143198.0,0.784152,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AL645608.1,1.2217,-0.64548,0.73515,0.49579,-0.683,0.0,0.0,0.00,0.0,0.00,48,-14,-46,-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,yes,ancient,ancient
2,chr1,862072,C,T,C,derived,snv,1/1,1/1,0/0,0/1,True,True,False,True,Neanderthal,yes,9.0,5096.0,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,yes,8.0,143246.0,0.000056,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.5082,-3.43610,0.89656,0.47484,0.197,0.0,0.0,0.00,0.0,0.00,3,28,-38,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
3,chr1,862072,C,T,C,derived,snv,1/1,1/1,0/0,0/1,True,True,False,True,Neanderthal,yes,9.0,5096.0,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,yes,8.0,143246.0,0.000056,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AL645608.1,1.2217,-0.64548,0.73515,0.49579,0.197,0.0,0.0,0.00,0.0,0.00,-6,-50,24,-48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [24]:
len(data)

1607350

# Spliceosome Variants <a class = 'anchor' id = 'spliceosomevariants'></a>

Let's take a quick detour to examine variants in components of the major spliceosome. We'll start with genes identified in the HGNC: https://www.genenames.org/data/genegroup/#!/group/1518.

In [25]:
spliceosome = pd.read_csv('../annotations/major_spliceosome_genes.txt', sep='\t', header=0)
spliceosome.head(10)

,HGNC ID,Approved symbol,Approved name,Status,Locus type,Previous symbols,Alias symbols,Chromosome,NCBI Gene ID,Ensembl gene ID,Vega gene ID,Group ID,Group name
0,HGNC:17040,CASC3,CASC3 exon junction complex subunit,Approved,gene with protein product,NaN,"MLN51, BTZ",17q21.1,22794,ENSG00000108349,OTTHUMG00000133323,1238,Exon junction complex
1,HGNC:18683,EIF4A3,eukaryotic translation initiation factor 4A3,Approved,gene with protein product,DDX48,"KIAA0111, EIF4AIII, Fal1",17q25.3,9775,ENSG00000141543,OTTHUMG00000177538,1238,Exon junction complex
2,HGNC:6815,MAGOH,"mago homolog, exon junction complex subunit",Approved,gene with protein product,NaN,"MAGOHA, MAGOH1",1p32.3,4116,ENSG00000162385,OTTHUMG00000008932,1238,Exon junction complex
3,HGNC:25504,MAGOHB,"mago homolog B, exon junction complex subunit",Approved,gene with protein product,NaN,"FLJ10292, MGN2",12p13.2,55110,ENSG00000111196,OTTHUMG00000168407,1238,Exon junction complex
4,HGNC:9905,RBM8A,RNA binding motif protein 8A,Approved,gene with protein product,RBM8,"ZNRP, BOV-1A, BOV-1B, BOV-1C, RBM8B, Y14",1q21.1,9939,ENSG00000265241,OTTHUMG00000013736,1238,Exon junction complex
5,HGNC:20472,LSM1,"LSM1 homolog, mRNA degradation associated",Approved,gene with protein product,NaN,"CASM, YJL124C",8p11.23,27257,ENSG00000175324,OTTHUMG00000164051,1505,LSm proteins
6,HGNC:17562,LSM10,"LSM10, U7 small nuclear RNA associated",Approved,gene with protein product,NaN,MGC15749,1p34.3,84967,ENSG00000181817,OTTHUMG00000008140,1505,LSm proteins
7,HGNC:30860,LSM11,"LSM11, U7 small nuclear RNA associated",Approved,gene with protein product,NaN,FLJ38273,5q33.3,134353,ENSG00000155858,OTTHUMG00000130255,1505,LSm proteins
8,HGNC:26407,LSM12,LSM12 homolog,Approved,gene with protein product,NaN,FLJ30656,17q21.31,124801,ENSG00000161654,OTTHUMG00000181804,1505,LSm proteins
9,HGNC:24489,LSM14A,LSM14A mRNA processing body assembly factor,Approved,gene with protein product,"C19orf13, FAM61A","DKFZP434D1335, RAP55A, RAP55",19q13.11,26065,ENSG00000257103,OTTHUMG00000180491,1505,LSm proteins


Let's get just the gene names.

In [26]:
spliceosome_genes = spliceosome['Approved symbol'].unique()
spliceosome_genes

array(['CASC3', 'EIF4A3', 'MAGOH', 'MAGOHB', 'RBM8A', 'LSM1', 'LSM10',
       'LSM11', 'LSM12', 'LSM14A', 'LSM14B', 'LSM2', 'LSM3', 'LSM4',
       'LSM5', 'LSM6', 'LSM7', 'LSM8', 'NAA38', 'BCAS2', 'CDC5L',
       'CTNNBL1', 'CWC15', 'PLRG1', 'PRPF19', 'SF3A1', 'SF3A2', 'SF3A3',
       'PHF5A', 'SF3B1', 'SF3B2', 'SF3B3', 'SF3B4', 'SF3B5', 'SF3B6',
       'SNRPB', 'SNRPD1', 'SNRPD2', 'SNRPD3', 'SNRPE', 'SNRPF', 'SNRPG',
       'SNRPN', 'CCAR1', 'CHERP', 'DDX46', 'DHX15', 'HNRNPA1', 'HNRNPAB',
       'PRPF40A', 'PUF60', 'RBM10', 'RBM17', 'RBM25', 'RBM5', 'SF1',
       'SMNDC1', 'SUGP1', 'THRAP3', 'U2AF1', 'U2AF2', 'U2SURP', 'AQR',
       'BUD31', 'CRNKL1', 'IK', 'ISY1', 'MFAP1', 'PPIE', 'PPIL1', 'PQBP1',
       'PRPF38A', 'RBM22', 'SMU1', 'SNW1', 'TFIP11', 'WBP11', 'WBP4',
       'XAB2', 'ZMAT2', 'CCDC12', 'CDC40', 'CWC22', 'CWC25', 'CWC27',
       'DHX16', 'EFTUD2', 'GPATCH1', 'GPKOW', 'PPIL2', 'PRCC', 'PRPF8',
       'RNF113A', 'RNU5A-1', 'RNU6-1', 'SAP18', 'SNRNP200', 'SNRNP40',
      

In [27]:
len(spliceosome_genes)

147

Now to subset all our variants for just those in spliceosome genes.

In [28]:
spliceosome_variants = data[data['annotation'].isin(spliceosome_genes)]
spliceosome_variants.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
21285,chr1,25549345,G,C,G,derived,snv,1/1,1/1,0/1,1/1,True,True,True,True,Shared,yes,37.0,5096.0,0.01,0.0,0.00,0.03,0.00,0.0,0.029240,yes,1116.0,143230.0,0.007792,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SYF2,0.93809,0.25699,0.42189,2.0202,0.650,0.00,0.0,0.0,0.00,0.00,33,-6,48,-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
21286,chr1,25549970,G,C,g,derived,snv,0/0,0/0,0/1,0/0,False,False,True,False,Denisovan,yes,242.0,5096.0,0.05,0.0,0.00,0.17,0.01,0.0,0.173489,yes,6387.0,143170.0,0.044611,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SYF2,0.93809,0.25699,0.42189,2.0202,-0.187,0.00,0.0,0.0,0.00,0.00,-48,-30,38,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
21287,chr1,25550063,G,A,G,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,23.0,5096.0,0.00,0.0,0.00,0.02,0.00,0.0,0.018519,yes,752.0,143264.0,0.005249,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SYF2,0.93809,0.25699,0.42189,2.0202,-0.797,0.00,0.0,0.0,0.00,0.00,28,-30,28,-41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
21288,chr1,25551415,G,A,G,derived,snv,0/0,0/0,0/1,0/0,False,False,True,False,Denisovan,yes,12.0,5096.0,0.00,0.0,0.00,0.01,0.00,0.0,0.010721,yes,317.0,143254.0,0.002213,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SYF2,0.93809,0.25699,0.42189,2.0202,-0.467,0.00,0.0,0.0,0.00,0.00,-9,-23,0,-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

How many variants did we get?

In [29]:
len(spliceosome_variants.groupby(['chrom','pos']))

4571

How many variants are uniquely archaic (i.e., absent from 1KG and gnomAD)?

In [30]:
archaic_specific_spliceosome_variants = spliceosome_variants[spliceosome_variants['Vernot_allele_origin'] == 'archaic-specific']

In [31]:
len(archaic_specific_spliceosome_variants.groupby(['chrom','pos']))

1746

Now let's format the data to run through Ensembl's Variant Effect Predictor. We will do this will all archaic-specific variants as well as all variants (archaic-specific or not) per archaic individual.

In [32]:
spliceosome_variants_for_VEP = spliceosome_variants[['chrom','pos','ref_allele','alt_allele','altai_gt_boolean','chagyrskaya_gt_boolean','denisovan_gt_boolean','vindija_gt_boolean']].copy()
spliceosome_variants_for_VEP['null'] = '.'
spliceosome_variants_for_VEP['chrom'] = spliceosome_variants_for_VEP['chrom'].str.replace('chr', '')

In [33]:
altai_spliceosome_variants_for_VEP = spliceosome_variants_for_VEP[spliceosome_variants_for_VEP['altai_gt_boolean'] == True]
altai_spliceosome_variants_for_VEP = altai_spliceosome_variants_for_VEP[['chrom','pos','null','ref_allele','alt_allele','null','null','null']]
len(altai_spliceosome_variants_for_VEP)

2907

In [34]:
chagyrskaya_spliceosome_variants_for_VEP = spliceosome_variants_for_VEP[spliceosome_variants_for_VEP['chagyrskaya_gt_boolean'] == True]
chagyrskaya_spliceosome_variants_for_VEP = chagyrskaya_spliceosome_variants_for_VEP[['chrom','pos','null','ref_allele','alt_allele','null','null','null']]
len(chagyrskaya_spliceosome_variants_for_VEP)

2831

In [35]:
denisovan_spliceosome_variants_for_VEP = spliceosome_variants_for_VEP[spliceosome_variants_for_VEP['denisovan_gt_boolean'] == True]
denisovan_spliceosome_variants_for_VEP = denisovan_spliceosome_variants_for_VEP[['chrom','pos','null','ref_allele','alt_allele','null','null','null']]
len(denisovan_spliceosome_variants_for_VEP)

2973

In [36]:
vindija_spliceosome_variants_for_VEP = spliceosome_variants_for_VEP[spliceosome_variants_for_VEP['vindija_gt_boolean'] == True]
vindija_spliceosome_variants_for_VEP = vindija_spliceosome_variants_for_VEP[['chrom','pos','null','ref_allele','alt_allele','null','null','null']]
len(vindija_spliceosome_variants_for_VEP)

2846

In [37]:
archaic_specific_spliceosome_variants_for_VEP = archaic_specific_spliceosome_variants[['chrom','pos','ref_allele','alt_allele']].copy()
archaic_specific_spliceosome_variants_for_VEP['null'] = '.'
archaic_specific_spliceosome_variants_for_VEP['chrom'] = archaic_specific_spliceosome_variants_for_VEP['chrom'].str.replace('chr', '')
archaic_specific_spliceosome_variants_for_VEP = archaic_specific_spliceosome_variants_for_VEP[['chrom','pos','null','ref_allele','alt_allele','null','null','null']]
archaic_specific_spliceosome_variants_for_VEP.head(10)

,chrom,pos,null,ref_allele,alt_allele,null,null,null
21290,1,25554269,.,T,A,.,.,.
24432,1,31734327,.,T,C,.,.,.
24433,1,31735461,.,G,A,.,.,.
24440,1,31741832,.,C,T,.,.,.
24442,1,31743354,.,C,T,.,.,.
24443,1,31743490,.,T,C,.,.,.
24446,1,31745979,.,T,G,.,.,.
24451,1,31751687,.,G,A,.,.,.
24454,1,31754447,.,C,T,.,.,.
24458,1,31759381,.,A,G,.,.,.


Save the files.

In [38]:
altai_spliceosome_variants_for_VEP.to_csv('../spliceosome/altai_spliceosome_variants_for_VEP.txt', sep="\t", header = False, index = False)
chagyrskaya_spliceosome_variants_for_VEP.to_csv('../spliceosome/chagyrskaya_spliceosome_variants_for_VEP.txt', sep="\t", header = False, index = False)
denisovan_spliceosome_variants_for_VEP.to_csv('../spliceosome/denisovan_spliceosome_variants_for_VEP.txt', sep="\t", header = False, index = False)
vindija_spliceosome_variants_for_VEP.to_csv('../spliceosome/vindija_spliceosome_variants_for_VEP.txt', sep="\t", header = False, index = False)
archaic_specific_spliceosome_variants_for_VEP.to_csv('../spliceosome/archaic_specific_spliceosome_variants_for_VEP.txt', sep="\t", header = False, index = False)

Run this file through the Ensembl Variant Effect Predictor and upload the results to the directory. 

Let's examine the archaic-specific results here.

In [39]:
archaic_specific_spliceosome_variants = pd.read_csv("../spliceosome/archaic_specific_spliceosome_variants.vep", sep='\t', header=0)
archaic_specific_spliceosome_variants.head(10)

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,SIFT,PolyPhen,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
0,.,1:25554269-25554269,A,intron_variant,MODIFIER,SYF2,ENSG00000117614,Transcript,ENST00000236273.4,protein_coding,-,4/6,-,-,-,-,-,-,-,-,-,-1,-,HGNC,19824,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,.,1:25554269-25554269,A,intron_variant,MODIFIER,SYF2,ENSG00000117614,Transcript,ENST00000354361.3,protein_coding,-,3/5,-,-,-,-,-,-,-,-,-,-1,-,HGNC,19824,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,.,1:25554269-25554269,A,downstream_gene_variant,MODIFIER,SYF2,ENSG00000117614,Transcript,ENST00000474160.1,processed_transcript,-,-,-,-,-,-,-,-,-,-,1237,-1,-,HGNC,19824,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,.,1:25554269-25554269,A,downstream_gene_variant,MODIFIER,SYF2,ENSG00000117614,Transcript,ENST00000476231.1,processed_transcript,-,-,-,-,-,-,-,-,-,-,2531,-1,-,HGNC,19824,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,.,1:31734327-31734327,C,intron_variant,MODIFIER,SNRNP40,ENSG00000060688,Transcript,ENST00000263694.4,protein_coding,-,9/9,-,-,-,-,-,-,-,-,-,-1,-,HGNC,30857,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
5,.,1:31734327-31734327,C,intron_variant,MODIFIER,SNRNP40,ENSG00000060688,Transcript,ENST00000373720.3,protein_coding,-,3/3,-,-,-,-,-,-,-,-,-,-1,-,HGNC,30857,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
6,.,1:31734327-31734327,C,intron_variant,MODIFIER,SNRNP40,ENSG00000060688,Transcript,ENST00000446633.2,protein_coding,-,9/10,-,-,-,-,-,-,-,-,-,-1,-,HGNC,30857,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
7,.,1:31734327-31734327,C,downstream_gene_variant,MODIFIER,RP11-490K7.4,ENSG00000229447,Transcript,ENST00000452686.2,processed_pseudogene,-,-,-,-,-,-,-,-,-,-,1765,-1,-,Clone_based_vega_gene,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
8,.,1:31734327-31734327,C,"intron_variant,non_coding_transcript_variant",MODIFIER,SNRNP40,ENSG00000060688,Transcript,ENST00000486941.1,processed_transcript,-,1/1,-,-,-,-,-,-,-,-,-,-1,-,HGNC,30857,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
9,.,1:31734327-31734327,C,"intron_variant,non_coding_transcript_variant",MODIFIER,SNRNP40,ENSG00000060688,Transcript,ENST00000489853.1,processed_transcript,-,5/5,-,-,-,-,-,-,-,-,-,-1,-,HGNC,30857,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


Subset to variants with non-synonymous effects.

In [40]:
nonsynonymous_archaic_specific_spliceosome_variants = archaic_specific_spliceosome_variants[archaic_specific_spliceosome_variants['Consequence'] == 'missense_variant']
nonsynonymous_archaic_specific_spliceosome_variants.head(10)

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,SIFT,PolyPhen,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
559,.,1:150297443-150297443,G,missense_variant,MODERATE,PRPF3,ENSG00000117360,Transcript,ENST00000324862.6,protein_coding,2/16,-,-,-,208,43,15,I/V,Ata/Gta,rs1395340408,-,1,-,HGNC,17348,-,-,-,-,tolerated(0.67),benign(0.001),-,-,-,-,-,-,-,-,-,-
560,.,1:150297443-150297443,G,missense_variant,MODERATE,PRPF3,ENSG00000117360,Transcript,ENST00000414970.2,protein_coding,2/15,-,-,-,131,43,15,I/V,Ata/Gta,rs1395340408,-,1,-,HGNC,17348,-,-,-,-,tolerated(0.75),benign(0),-,-,-,-,-,-,-,-,-,-
1560,.,11:116633557-116633557,G,missense_variant,MODERATE,BUD13,ENSG00000137656,Transcript,ENST00000260210.4,protein_coding,4/10,-,-,-,772,748,250,D/H,Gac/Cac,rs1425200283,-,-1,-,HGNC,28199,-,-,-,-,deleterious(0),benign(0.341),-,-,-,-,-,-,-,-,-,-
1891,.,13:21714974-21714974,G,missense_variant,MODERATE,SAP18,ENSG00000150459,Transcript,ENST00000450573.1,protein_coding,1/3,-,-,-,62,13,5,F/V,Ttc/Gtc,-,-,1,cds_end_NF,HGNC,10530,-,-,-,-,tolerated_low_confidence(1),unknown(0),-,-,-,-,-,-,-,-,-,-
3692,.,15:35219290-35219290,C,missense_variant,MODERATE,AQR,ENSG00000021776,Transcript,ENST00000156471.5,protein_coding,13/35,-,-,-,1290,1064,355,N/S,aAt/aGt,rs767183682,-,-1,-,HGNC,29513,-,-,-,-,tolerated(0.09),benign(0.062),-,-,-,-,-,-,-,-,-,-
3963,.,16:2810331-2810331,T,missense_variant,MODERATE,SRRM2,ENSG00000167978,Transcript,ENST00000301740.8,protein_coding,10/15,-,-,-,1412,863,288,T/I,aCa/aTa,rs747144756,-,1,-,HGNC,16639,-,-,-,-,deleterious_low_confidence(0.04),benign(0.006),-,-,-,-,-,-,-,-,-,-
3967,.,16:2810331-2810331,T,missense_variant,MODERATE,SRRM2,ENSG00000167978,Transcript,ENST00000571378.1,protein_coding,9/10,-,-,-,760,575,192,T/I,aCa/aTa,rs747144756,-,1,cds_end_NF,HGNC,16639,-,-,-,-,tolerated_low_confidence(0.18),benign(0.006),-,-,-,-,-,-,-,-,-,-
3973,.,16:2810331-2810331,T,missense_variant,MODERATE,SRRM2,ENSG00000167978,Transcript,ENST00000575009.1,protein_coding,9/10,-,-,-,830,575,192,T/I,aCa/aTa,rs747144756,-,1,cds_end_NF,HGNC,16639,-,-,-,-,tolerated_low_confidence(0.22),benign(0.006),-,-,-,-,-,-,-,-,-,-
3980,.,16:2810331-2810331,T,missense_variant,MODERATE,SRRM2,ENSG00000167978,Transcript,ENST00000576924.1,protein_coding,10/11,-,-,-,1123,863,288,T/I,aCa/aTa,rs747144756,-,1,cds_end_NF,HGNC,16639,-,-,-,-,tolerated_low_confidence(0.13),benign(0.006),-,-,-,-,-,-,-,-,-,-
4017,.,16:2813610-2813610,C,missense_variant,MODERATE,SRRM2,ENSG00000167978,Transcript,ENST00000301740.8,protein_coding,11/15,-,-,-,3630,3081,1027,Q/H,caA/caC,rs530922815,-,1,-,HGNC,16639,-,-,-,-,deleterious_low_confidence(0.04),benign(0.365),-,-,-,-,-,-,-,-,-,-


How many transcripts are impacted?

In [41]:
len(nonsynonymous_archaic_specific_spliceosome_variants)

36

How many variants does this represent?

In [42]:
nonsynonymous_archaic_specific_spliceosome_variants.groupby(by=['Location', 'Allele', 'Gene']).size().reset_index(name='transcript_count')

,Location,Allele,Gene,transcript_count
0,11:116633557-116633557,G,ENSG00000137656,1
1,13:21714974-21714974,G,ENSG00000150459,1
2,15:35219290-35219290,C,ENSG00000021776,1
3,16:2810331-2810331,T,ENSG00000167978,4
4,16:2813610-2813610,C,ENSG00000167978,1
5,16:2817499-2817499,G,ENSG00000167978,1
6,16:2819197-2819197,C,ENSG00000167978,1
7,17:1554120-1554120,T,ENSG00000174231,1
8,17:1557188-1557188,A,ENSG00000174231,2
9,17:36963046-36963046,C,ENSG00000108296,2


Now let's look at the potential effects starting with SIFT designations.

In [43]:
nonsynonymous_archaic_specific_spliceosome_variants_SIFT = nonsynonymous_archaic_specific_spliceosome_variants.groupby(by=['Location','SIFT']).size().reset_index(name='SIFT_count')
nonsynonymous_archaic_specific_spliceosome_variants_SIFT

,Location,SIFT,SIFT_count
0,11:116633557-116633557,deleterious(0),1
1,13:21714974-21714974,tolerated_low_confidence(1),1
2,15:35219290-35219290,tolerated(0.09),1
3,16:2810331-2810331,deleterious_low_confidence(0.04),1
4,16:2810331-2810331,tolerated_low_confidence(0.13),1
5,16:2810331-2810331,tolerated_low_confidence(0.18),1
6,16:2810331-2810331,tolerated_low_confidence(0.22),1
7,16:2813610-2813610,deleterious_low_confidence(0.04),1
8,16:2817499-2817499,deleterious_low_confidence(0),1
9,16:2819197-2819197,deleterious_low_confidence(0),1


In [44]:
SIFT_deleterious = nonsynonymous_archaic_specific_spliceosome_variants_SIFT[nonsynonymous_archaic_specific_spliceosome_variants_SIFT['SIFT'].str.contains('deleterious')]
SIFT_deleterious

,Location,SIFT,SIFT_count
0,11:116633557-116633557,deleterious(0),1
3,16:2810331-2810331,deleterious_low_confidence(0.04),1
7,16:2813610-2813610,deleterious_low_confidence(0.04),1
8,16:2817499-2817499,deleterious_low_confidence(0),1
9,16:2819197-2819197,deleterious_low_confidence(0),1
10,17:1554120-1554120,deleterious_low_confidence(0),1
14,19:36124120-36124120,deleterious_low_confidence(0),5
16,19:54663336-54663336,deleterious(0.02),1


And PolyPhen.

In [45]:
nonsynonymous_archaic_specific_spliceosome_variants_PolyPhen = nonsynonymous_archaic_specific_spliceosome_variants.groupby(by=['Location','PolyPhen']).size().reset_index(name='PolyPhen_count')
nonsynonymous_archaic_specific_spliceosome_variants_PolyPhen

,Location,PolyPhen,PolyPhen_count
0,11:116633557-116633557,benign(0.341),1
1,13:21714974-21714974,unknown(0),1
2,15:35219290-35219290,benign(0.062),1
3,16:2810331-2810331,benign(0.006),4
4,16:2813610-2813610,benign(0.365),1
5,16:2817499-2817499,benign(0.007),1
6,16:2819197-2819197,benign(0.106),1
7,17:1554120-1554120,unknown(0),1
8,17:1557188-1557188,benign(0.063),2
9,17:36963046-36963046,benign(0.034),1


In [46]:
PolyPhen_damaging = nonsynonymous_archaic_specific_spliceosome_variants_PolyPhen[nonsynonymous_archaic_specific_spliceosome_variants_PolyPhen['PolyPhen'].str.contains('damaging')]
PolyPhen_damaging

,Location,PolyPhen,PolyPhen_count
12,19:36124120-36124120,probably_damaging(0.993),1
13,19:36124120-36124120,probably_damaging(0.997),5
14,19:54663336-54663336,possibly_damaging(0.592),1
19,20:37632435-37632435,possibly_damaging(0.454),1
20,20:37632435-37632435,possibly_damaging(0.506),1
21,5:150080190-150080190,possibly_damaging(0.711),1
23,5:176940736-176940736,possibly_damaging(0.451),1


Do SIFT and PolyPhen agree on any positions?

In [47]:
pd.merge(SIFT_deleterious, PolyPhen_damaging, on = ['Location'])

,Location,SIFT,SIFT_count,PolyPhen,PolyPhen_count
0,19:36124120-36124120,deleterious_low_confidence(0),5,probably_damaging(0.993),1
1,19:36124120-36124120,deleterious_low_confidence(0),5,probably_damaging(0.997),5
2,19:54663336-54663336,deleterious(0.02),1,possibly_damaging(0.592),1


Now let's save the dataframe of non-synonymous variants for any downstream analyses.

In [48]:
nonsynonymous_archaic_specific_spliceosome_variants.to_csv('../spliceosome/nonsynonymous_archaic_specific_spliceosome_variants.txt', sep="\t", header = True, index = False)

# Data Description <a class = 'anchor' id = 'datadescription'></a>

Let's get a handle on the dataset as a whole. Then we will dive into the SAVs.

Count the number of positions.

In [49]:
len(data.groupby(['chrom','pos']))

1567894

Count the number of variants.

In [50]:
len(data)

1607350

## Multiple Annotations <a class = 'anchor' id = 'multipleannotations'></a>

Let's assess the degree of multiple annotation in the data.

In [51]:
multi_annotation = data.groupby(by=['chrom','pos','ref_allele','altai_gt','chagyrskaya_gt','denisovan_gt','vindija_gt','alt_allele']).size().reset_index(name='annotation_count')
multi_annotation.groupby('annotation_count').size().to_frame('count')

,count
annotation_count,
1,1537451
2,30448
3,1154
4,75
5,36
6,15
7,25
8,22
9,36


## Multiple Alleles <a class = 'anchor' id = 'multiplealleles'></a>

Let's assess the degree of multi-allelism in the data (excluding positions with multiple annotations). 

In [52]:
single_annotation_data = data.drop_duplicates(subset=['chrom','pos','ref_allele','altai_gt','chagyrskaya_gt','denisovan_gt','vindija_gt','alt_allele','variant_type'], keep='first').reset_index()
len(single_annotation_data)

1569556

In [53]:
multi_allelic = single_annotation_data.groupby(['chrom','pos',]).size().reset_index(name='allele_count')
multi_allelic.head(10)

,chrom,pos,allele_count
0,chr1,861808,1
1,chr1,862072,1
2,chr1,862093,1
3,chr1,862124,1
4,chr1,862383,1
5,chr1,862389,1
6,chr1,863124,1
7,chr1,863843,1
8,chr1,863863,1
9,chr1,863978,1


In [54]:
multi_allelic.groupby('allele_count').size().to_frame('allele count')

,allele count
allele_count,
1,1566233
2,1660
3,1


## Genotypes <a class = 'anchor' id = 'genotypes'></a>

Now let's take a look at the distribution of genotypes across individuals. 

In [55]:
data.groupby(['altai_gt']).size().to_frame('size')

,size
altai_gt,
./.,6226
0/0,591371
0/1,126120
1/0,320
1/1,883313


In [56]:
data.groupby(['chagyrskaya_gt']).size().to_frame('size')

,size
chagyrskaya_gt,
./.,37156
0/0,583787
0/1,97932
1/0,144
1/1,888331


In [57]:
data.groupby(['denisovan_gt']).size().to_frame('size')

,size
denisovan_gt,
./.,21574
0/0,546792
0/1,139522
1/0,252
1/1,899210


In [58]:
data.groupby(['vindija_gt']).size().to_frame('size')

,size
vindija_gt,
./.,29672
0/0,570175
0/1,119718
1/0,200
1/1,887585


## Annotations <a class = 'anchor' id = 'annotations'></a>

How many annotations are represented in the total data?

In [59]:
len(data.groupby(['annotation']))

17631

What is the distribution of these annotations?

In [60]:
n_annotations = data.groupby(['annotation']).size().to_frame('count')
n_annotations.groupby('count').size().to_frame('n')

,n
count,
1,577
2,546
3,473
4,475
5,427
...,...
3868,1
3956,1
4747,1


## Delta Thresholds <a class = 'anchor' id = 'deltathresholds'></a>

Let's do a quick subset of the data for variants that include one delta >= 0.2 and another for deltas >= 0.5.

In [61]:
data_2 = data[data['delta_max']>=0.2]
data_2.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
24,chr1,864726,T,A,T,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,4859.0,5096.0,0.95,1.00,1.00,0.83,0.99,1.00,0.812865,yes,137320.0,143300.0,0.958269,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.50820,-3.43610,0.89656,0.47484,0.398,0.00,0.00,0.02,0.22,0.22,17,-2,43,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
65,chr1,875708,C,G,T,derived,snv,1/1,./.,./.,./.,True,False,False,False,Altai,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,139672.0,0.000007,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.50820,-3.43610,0.89656,0.47484,-0.847,0.00,0.00,0.29,0.01,0.29,-43,33,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
130,chr1,909768,A,G,-,.,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,5094.0,5096.0,1.00,1.00,1.00,1.00,1.00,1.00,0.998051,yes,143298.0,143312.0,0.999902,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PLEKHN1,1.25930,-1.79880,0.80467,1.00890,-1.259,0.00,0.29,0.13,0.03,0.29,7,26,-24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
155,chr1,956622,C,T,C,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,4.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,0.002924,yes,62.0,143332.0,0.000433,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AGRN,0.98257,0.22552,0.31914,6.01430,-0.542,0.00,0.00,0.24,0.02,0.24,-2,50,-2,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [62]:
len(data_2)

5950

In [63]:
len(data_2.groupby(['chrom','pos']))

5811

Now let's examine multiple annotations in these data.

In [64]:
SAV_2_multiple_annotation = data_2.groupby(['chrom','pos',]).size().reset_index(name='SAV_multiple_annotation')
SAV_2_multiple_annotation.groupby('SAV_multiple_annotation').size().to_frame('count')

,count
SAV_multiple_annotation,
1,5679
2,129
3,2
7,1


Now for delta >= 0.5.

In [65]:
data_5 = data[data['delta_max']>=0.5]
data_5.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
409,chr1,1166887,T,C,T,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,898.0,5096.0,0.18,0.17,0.13,0.26,0.11,0.17,0.269981,yes,23482.0,143206.0,0.163974,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SDF4,0.90199,0.55251,0.387200,2.23540,0.260,0.00,0.00,0.52,0.00,0.52,-9,-2,0,-4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,49.0,yes,ancient,ancient
1881,chr1,2524237,A,C,C,ancestral,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,1.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,yes,2.0,142446.0,0.000014,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MMEL1,0.98394,0.12602,0.768240,1.56890,-0.920,0.01,0.00,0.00,0.56,0.56,-42,10,35,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
3147,chr1,3445088,C,A,C,derived,snv,1/1,0/0,0/0,0/0,True,False,False,False,Altai,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MEGF6,0.95847,0.45803,0.738560,2.27280,0.205,0.00,0.00,0.00,0.73,0.73,-4,44,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
4234,chr1,5935162,A,T,T,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NPHP4,1.02290,-0.24249,0.781550,1.66690,-0.557,1.00,0.91,0.00,0.00,1.00,-2,-8,-43,-44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [66]:
len(data_5)

1049

In [67]:
len(data_5.groupby(['chrom','pos']))

1017

In [68]:
SAV_5_multiple_annotation = data_5.groupby(['chrom','pos',]).size().reset_index(name='SAV_multiple_annotation')
SAV_5_multiple_annotation.groupby('SAV_multiple_annotation').size().to_frame('count')

,count
SAV_multiple_annotation,
1,986
2,30
3,1


Now let's count the number of each class.

In [69]:
len(data_2[(data_2['ag_delta'] >= 0.2)])

1511

In [70]:
len(data_2[(data_2['al_delta'] >= 0.2)])

1377

In [71]:
len(data_2[(data_2['dg_delta'] >= 0.2)])

1894

In [72]:
len(data_2[(data_2['dl_delta'] >= 0.2)])

1647

In [73]:
len(data_5[(data_5['ag_delta'] >= 0.5)])

180

In [74]:
len(data_5[(data_5['al_delta'] >= 0.5)])

191

In [75]:
len(data_5[(data_5['dg_delta'] >= 0.5)])

390

In [76]:
len(data_5[(data_5['dl_delta'] >= 0.5)])

349

Let's get a count of each class where the effect is unique to that class per threshold.

In [77]:
len(data_2[(data_2['ag_delta'] >= 0.2) & (data_2['al_delta'] == 0) & (data_2['dg_delta'] == 0) & (data_2['dl_delta'] == 0)])

616

In [78]:
len(data_2[(data_2['al_delta'] >= 0.2) & (data_2['ag_delta'] == 0) & (data_2['dg_delta'] == 0) & (data_2['dl_delta'] == 0)])

569

In [79]:
len(data_2[(data_2['dg_delta'] >= 0.2) & (data_2['ag_delta'] == 0) & (data_2['al_delta'] == 0) & (data_2['dl_delta'] == 0) ])

899

In [80]:
len(data_2[(data_2['dl_delta'] >= 0.2) & (data_2['ag_delta'] == 0) & (data_2['al_delta'] == 0) & (data_2['dg_delta'] == 0)])

761

In [81]:
len(data_5[(data_5['ag_delta'] >= 0.5) & (data_5['al_delta'] == 0) & (data_5['dg_delta'] == 0) & (data_5['dl_delta'] == 0)])

39

In [82]:
len(data_5[(data_5['al_delta'] >= 0.5) & (data_5['ag_delta'] == 0) & (data_5['dg_delta'] == 0) & (data_5['dl_delta'] == 0)])

37

In [83]:
len(data_5[(data_5['dg_delta'] >= 0.5) & (data_5['ag_delta'] == 0) & (data_5['al_delta'] == 0) & (data_5['dl_delta'] == 0)])

146

In [84]:
len(data_5[(data_5['dl_delta'] >= 0.5) & (data_5['ag_delta'] == 0) & (data_5['al_delta'] == 0) & (data_5['dg_delta'] == 0)])

133

## Variant Distribution <a class = 'anchor' id = 'variantdistribution'></a>

In [85]:
data.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,81916
Chagyrskaya,53675
Denisovan,411492
Late Neanderthal,44651
Neanderthal,266965
Other,104455
Shared,573197
Vindija,70999


In [86]:
data_2.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,399
Chagyrskaya,218
Denisovan,1492
Late Neanderthal,172
Neanderthal,973
Other,456
Shared,1933
Vindija,307


In [87]:
data_5.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,75
Chagyrskaya,35
Denisovan,254
Late Neanderthal,22
Neanderthal,178
Other,84
Shared,328
Vindija,73


Let's get the total count per individual for delta >= 0.2.

In [88]:
len(data_2[data_2['altai_gt_boolean'] == True])

3705

In [89]:
len(data_2[data_2['chagyrskaya_gt_boolean'] == True])

3482

In [90]:
len(data_2[data_2['denisovan_gt_boolean'] == True])

3671

In [91]:
len(data_2[data_2['vindija_gt_boolean'] == True])

3627

## Delta Correlations <a class = 'anchor' id = 'deltacorrelations'></a>

Let's examine the correlations between deltas. There are a lot of zeroes so we'll first exclude rows with all zeroes as this will bias our analysis.

In [92]:
data_nonzero = data[(data['ag_delta']>0) | (data['al_delta']>0) | (data['dg_delta']>0) | (data['dl_delta']>0)]
data_nonzero.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
0,chr1,861808,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,3479.0,5096.0,0.68,0.55,0.97,0.34,0.83,0.88,0.313840,yes,112289.0,143198.0,0.784152,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.5082,-3.43610,0.89656,0.47484,-0.683,0.00,0.00,0.01,0.00,0.01,-29,-27,24,-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,yes,ancient,ancient
19,chr1,863978,G,A,g,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,497.0,5096.0,0.10,0.15,0.01,0.18,0.11,0.03,0.180312,yes,9475.0,143334.0,0.066104,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AL645608.1,1.2217,-0.64548,0.73515,0.49579,0.197,0.00,0.00,0.00,0.01,0.01,-15,4,2,-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
20,chr1,864678,C,T,C,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.5082,-3.43610,0.89656,0.47484,0.486,0.00,0.00,0.03,0.00,0.03,-48,26,46,-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
21,chr1,864678,C,T,C,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AL645608.1,1.2217,-0.64548,0.73515,0.49579,0.486,0.01,0.02,0.00,0.00,0.02,-2,44,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [93]:
len(data_nonzero)

188302

Run correlations.

In [94]:
rho, p = spearmanr(data_nonzero['ag_delta'], data_nonzero['al_delta'])
print(rho,p)

-0.14267746909602813 0.0


In [95]:
rho, p = spearmanr(data_nonzero['ag_delta'], data_nonzero['dg_delta'])
print(rho,p)

-0.2239766968064798 0.0


In [96]:
rho, p = spearmanr(data_nonzero['ag_delta'], data_nonzero['dl_delta'])
print(rho,p)

-0.34023522469107176 0.0


In [97]:
rho, p = spearmanr(data_nonzero['al_delta'], data_nonzero['dg_delta'])
print(rho,p)

-0.33985645661346087 0.0


In [98]:
rho, p = spearmanr(data_nonzero['al_delta'], data_nonzero['dl_delta'])
print(rho,p)

-0.2146624536999448 0.0


In [99]:
rho, p = spearmanr(data_nonzero['dg_delta'], data_nonzero['dl_delta'])
print(rho,p)

-0.2036868319219267 0.0


Repeat for splice altering variants.

In [100]:
rho, p = spearmanr(data_2['ag_delta'], data_2['al_delta'])
print(rho,p)

0.1500250528757491 2.7088750503541857e-31


In [101]:
rho, p = spearmanr(data_2['ag_delta'], data_2['dg_delta'])
print(rho,p)

-0.32319408269234423 1.0314796151065124e-144


In [102]:
rho, p = spearmanr(data_2['ag_delta'], data_2['dl_delta'])
print(rho,p)

-0.46608147540520983 0.0


In [103]:
rho, p = spearmanr(data_2['al_delta'], data_2['dg_delta'])
print(rho,p)

-0.48795009291846625 0.0


In [104]:
rho, p = spearmanr(data_2['al_delta'], data_2['dl_delta'])
print(rho,p)

-0.27635622346475713 9.11316178310797e-105


In [105]:
rho, p = spearmanr(data_2['dg_delta'], data_2['dl_delta'])
print(rho,p)

-0.02349927032949021 0.06990668106164923


The p-values above are listed as 0, which means those values are less than the floating point minimum. Let's get that value for the manuscript.

In [106]:
sys.float_info.min

2.2250738585072014e-308

## Multiple Deltas <a class = 'anchor' id = 'multipledeltas'></a>

How many variants have a single variant effect? This result should match the sum of the four individual counts above.

In [107]:
delta_counts = (data_2[['ag_delta','al_delta','dg_delta','dl_delta']] > 0).sum(1).to_frame('count')
len(delta_counts[delta_counts['count'] == 1])

2845

And it does. Bravo.

Let's assess the number of variants that result in both a gain and a loss for the donor and acceptor. Presumably, these are strong candidates for alternative splicing.

In [108]:
len(data_nonzero[(data_nonzero['ag_delta'] >= 0.2) & (data_nonzero['al_delta'] >= 0.2)])

119

In [109]:
len(data_nonzero[(data_nonzero['dg_delta'] >= 0.2) & (data_nonzero['dl_delta'] >= 0.2)])

108

Now let's examine when both the donor and acceptor are affected. These events likely represent exon skipping (Jaganathan et al. 2019).

In [110]:
len(data_nonzero[(data_nonzero['ag_delta'] >= 0.2) & (data_nonzero['dg_delta'] >= 0.2)])

119

In [111]:
len(data_nonzero[(data_nonzero['ag_delta'] >= 0.2) & (data_nonzero['dl_delta'] >= 0.2)])

6

In [112]:
len(data_nonzero[(data_nonzero['al_delta'] >= 0.2) & (data_nonzero['dg_delta'] >= 0.2)])

6

In [113]:
len(data_nonzero[(data_nonzero['al_delta'] >= 0.2) & (data_nonzero['dl_delta'] >= 0.2)])

126

Let's save these cases of exon skipping to a dataframe.

In [114]:
exon_skipping_2 = data_nonzero[((data_nonzero['ag_delta'] >= 0.2) & (data_nonzero['dg_delta'] >= 0.2)) | ((data_nonzero['ag_delta'] >= 0.2) & (data_nonzero['dl_delta'] >= 0.2)) | ((data_nonzero['al_delta'] >= 0.2) & (data_nonzero['dg_delta'] >= 0.2)) | ((data_nonzero['al_delta'] >= 0.2) & (data_nonzero['dl_delta'] >= 0.2))] 

In [115]:
len(exon_skipping_2)

252

In [116]:
exon_skipping_5 = data_nonzero[((data_nonzero['ag_delta'] >= 0.5) & (data_nonzero['dg_delta'] >= 0.5)) | ((data_nonzero['ag_delta'] >= 0.5) & (data_nonzero['dl_delta'] >= 0.5)) | ((data_nonzero['al_delta'] >= 0.5) & (data_nonzero['dg_delta'] >= 0.5)) | ((data_nonzero['al_delta'] >= 0.5) & (data_nonzero['dl_delta'] >= 0.5))] 

In [117]:
len(exon_skipping_5)

24

In [118]:
exon_skipping_2.to_csv('../dataframes/exon_skipping_2.txt', sep="\t", header = True, index = False)
exon_skipping_5.to_csv('../dataframes/exon_skipping_5.txt', sep="\t", header = True, index = False)

Let's check the distribution and allele origin of these exon skipping variants out of curiosity.

In [119]:
exon_skipping_2.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,11
Chagyrskaya,8
Denisovan,63
Late Neanderthal,9
Neanderthal,49
Other,15
Shared,88
Vindija,9


In [120]:
exon_skipping_2.groupby(['Vernot_allele_origin']).size().to_frame('origin')

,origin
Vernot_allele_origin,
ancient,92
archaic-specific,88
introgressed,15
low-confidence ancient,57


In [121]:
exon_skipping_2.groupby(['Browning_allele_origin']).size().to_frame('origin')

,origin
Browning_allele_origin,
ancient,88
archaic-specific,88
introgressed,24
low-confidence ancient,52


## SAV Genotypes <a class = 'anchor' id = 'SAVgenotypes'></a>

We just examined the distribution of genotypes but let's do this again for SAVs.

In [122]:
data_2.groupby(['altai_gt']).size().to_frame('size')

,size
altai_gt,
./.,18
0/0,2227
0/1,537
1/0,1
1/1,3167


In [123]:
data_5.groupby(['altai_gt']).size().to_frame('size')

,size
altai_gt,
./.,4
0/0,386
0/1,101
1/1,558


In [124]:
data_2.groupby(['chagyrskaya_gt']).size().to_frame('size')

,size
chagyrskaya_gt,
./.,266
0/0,2202
0/1,395
1/0,2
1/1,3085


In [125]:
data_5.groupby(['chagyrskaya_gt']).size().to_frame('size')

,size
chagyrskaya_gt,
./.,69
0/0,394
0/1,60
1/0,1
1/1,525


In [126]:
data_2.groupby(['denisovan_gt']).size().to_frame('size')

,size
denisovan_gt,
./.,157
0/0,2122
0/1,584
1/0,3
1/1,3084


In [127]:
data_5.groupby(['denisovan_gt']).size().to_frame('size')

,size
denisovan_gt,
./.,45
0/0,376
0/1,106
1/0,1
1/1,521


In [128]:
data_2.groupby(['vindija_gt']).size().to_frame('size')

,size
vindija_gt,
./.,163
0/0,2160
0/1,496
1/1,3131


In [129]:
data_5.groupby(['vindija_gt']).size().to_frame('size')

,size
vindija_gt,
./.,35
0/0,361
0/1,108
1/1,545


## 1KG vs gnomAD SAVs <a class = 'anchor' id = '1KGgnomADSAVs'></a>

Let's briefly compare 1KG and gnomAD SAVs. 

In [130]:
data_2.groupby(['present_in_gnomAD']).size().to_frame('N')

,N
present_in_gnomAD,
no,1705
yes,4245


In [131]:
data_2.groupby(['present_in_gnomAD','present_in_1KG']).size().to_frame('origin')

origin
present_in_gnomAD present_in_1KG        
no                no                1674
                  yes                 31
yes               no                 751
                  yes               3494

How many rare variants (allele frequency < 0.0001) occur only in gnomAD? These are likely recurrent mutations.

In [132]:
len(data_2[(data_2['present_in_gnomAD'] == 'yes') & (data_2['present_in_1KG'] == 'no') & (data_2['gnomAD_allele_frequency'] < 0.0001)])

512

Let's test for CpG enrichment among those sites. Enrichment for CpGs supports the hypothesis that they are recurrent mutations.

In [133]:
gnomAD_CpGs_header = ['chrom','start','ref_allele','alt_allele','allele_count','allele_number','allele_frequency','C_dinucleotide']
gnomAD_CpGs = pd.read_csv('../archaic_variants_in_humans/archaic_SNVs_in_gnomAD_allele_frequencies_hg19_with_CpG_annotation.txt', sep = '\t', names = gnomAD_CpGs_header)
gnomAD_CpGs.head(10)

,chrom,start,ref_allele,alt_allele,allele_count,allele_number,allele_frequency,C_dinucleotide
0,chr10,183370,C,G,1,143172,0.000007,CT
1,chr10,184616,T,G,437,143284,0.003050,n
2,chr10,185171,G,T,132752,143248,0.926728,n
3,chr10,185358,C,T,132720,143208,0.926764,CA
4,chr10,185611,C,G,1,141008,0.000007,CG
5,chr10,185611,C,T,1,141008,0.000007,CG
6,chr10,185612,G,A,432,138944,0.003109,n
7,chr10,188101,C,T,7,143212,0.000049,CG
8,chr10,188859,A,G,437,143332,0.003049,n
9,chr10,190851,T,C,129638,140998,0.919431,n


In [134]:
def CpG_enrichment_rare_common(rare_allele_frequency, common_allele_frequency):
    rare_CpGs = len(gnomAD_CpGs[(gnomAD_CpGs['allele_frequency'] < rare_allele_frequency) & (gnomAD_CpGs['C_dinucleotide'] == 'CG')])
    rare_non_CpGs = len(gnomAD_CpGs[(gnomAD_CpGs['allele_frequency'] < rare_allele_frequency) & (gnomAD_CpGs['C_dinucleotide'] != 'CG')])
    common_CpGs = len(gnomAD_CpGs[(gnomAD_CpGs['allele_frequency'] >= common_allele_frequency) & (gnomAD_CpGs['C_dinucleotide'] == 'CG')])
    common_non_CpGs = len(gnomAD_CpGs[(gnomAD_CpGs['allele_frequency'] >= common_allele_frequency) & (gnomAD_CpGs['C_dinucleotide'] != 'CG')])
    
    oddsratio, pvalue = fisher_exact([[rare_CpGs, rare_non_CpGs], [common_CpGs, common_non_CpGs]])
    return rare_CpGs, rare_non_CpGs, common_CpGs, common_non_CpGs, oddsratio, pvalue

In [135]:
CpG_enrichment_rare_common(0.0001, 0.01)

(62328, 402159, 65449, 794925, 1.8823853718697245, 0.0)

## SAV Allele Origin <a class = 'anchor' id = 'SAValleleorigin'></a>

Let's consider the allele origin for our data. Let's start with Vernot.

In [136]:
data.groupby(['Vernot_allele_origin']).size().to_frame('origin')

,origin
Vernot_allele_origin,
ancient,681416
archaic-specific,523407
introgressed,68046
low-confidence ancient,334481


In [137]:
data_2.groupby(['Vernot_allele_origin']).size().to_frame('origin')

,origin
Vernot_allele_origin,
ancient,2252
archaic-specific,2186
introgressed,244
low-confidence ancient,1268


In [138]:
data_5.groupby(['Vernot_allele_origin']).size().to_frame('origin')

,origin
Vernot_allele_origin,
ancient,383
archaic-specific,404
introgressed,59
low-confidence ancient,203


Now for Browning.

In [139]:
data.groupby(['Browning_allele_origin']).size().to_frame('origin')

,origin
Browning_allele_origin,
ancient,670611
archaic-specific,523407
introgressed,98189
low-confidence ancient,315143


In [140]:
data_2.groupby(['Browning_allele_origin']).size().to_frame('origin')

,origin
Browning_allele_origin,
ancient,2195
archaic-specific,2186
introgressed,386
low-confidence ancient,1183


In [141]:
data_5.groupby(['Browning_allele_origin']).size().to_frame('origin')

,origin
Browning_allele_origin,
ancient,376
archaic-specific,404
introgressed,71
low-confidence ancient,198


Let's take a quick peak at the ancient category and add some additional parameters to test the robustness of that category. Presumably, these variants should be shared in the archaics and occur at high frequency in Africans.

In [142]:
data_2.head(5)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
24,chr1,864726,T,A,T,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,4859.0,5096.0,0.95,1.0,1.00,0.83,0.99,1.00,0.812865,yes,137320.0,143300.0,0.958269,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.50820,-3.43610,0.89656,0.47484,0.398,0.0,0.00,0.02,0.22,0.22,17,-2,43,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
65,chr1,875708,C,G,T,derived,snv,1/1,./.,./.,./.,True,False,False,False,Altai,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,139672.0,0.000007,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SAMD11,1.50820,-3.43610,0.89656,0.47484,-0.847,0.0,0.00,0.29,0.01,0.29,-43,33,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
130,chr1,909768,A,G,-,.,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,5094.0,5096.0,1.00,1.0,1.00,1.00,1.00,1.00,0.998051,yes,143298.0,143312.0,0.999902,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PLEKHN1,1.25930,-1.79880,0.80467,1.00890,-1.259,0.0,0.29,0.13,0.03,0.29,7,26,-24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
155,chr1,956622,C,T,C,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,4.0,5096.0,0.00,0.0,0.00,0.00,0.00,0.00,0.002924,yes,62.0,143332.0,0.000433,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AGRN,0.98257,0.22552,0.31914,6.01430,-0.542,0.0,0.00,0.24,0.02,0.24,-2,50,-2,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [143]:
len(data_2[(data_2['Vernot_allele_origin'] == 'ancient') & (data_2['distribution'] == 'Shared') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

1380

In [144]:
1380/2252

0.6127886323268206

In [145]:
len(data_2[(data_2['Browning_allele_origin'] == 'ancient') & (data_2['distribution'] == 'Shared') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

1380

In [146]:
1380/2195

0.6287015945330297

What about dropping the shared archaic part?

In [147]:
len(data_2[(data_2['Vernot_allele_origin'] == 'ancient') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

2098

In [148]:
2098/2252

0.9316163410301954

In [149]:
len(data_2[(data_2['Browning_allele_origin'] == 'ancient') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

2097

In [150]:
2097/2195

0.9553530751708428

How many introgressed variants may have evolved prior to the archaic-modern split?

In [151]:
len(data_2[(data_2['Vernot_allele_origin'] == 'introgressed') & (data_2['distribution'] == 'Shared')])

68

In [152]:
68/244

0.2786885245901639

In [153]:
len(data_2[(data_2['Browning_allele_origin'] == 'introgressed') & (data_2['distribution'] == 'Shared')])

122

In [154]:
122/386

0.3160621761658031

In [155]:
len(data_2[(data_2['Vernot_allele_origin'] == 'introgressed') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

1

In [156]:
len(data_2[(data_2['Browning_allele_origin'] == 'introgressed') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.05)])

2

In [157]:
len(data_2[(data_2['Vernot_allele_origin'] == 'introgressed') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.01)])

2

In [158]:
len(data_2[(data_2['Browning_allele_origin'] == 'introgressed') & (data_2['1KG_non_ASW_AFR_AF'] >= 0.01)])

16

## Introgression Set Overlap <a class = 'anchor' id = 'introgressionsetoverlap'></a>

How well do Vernot et al. 2016 and Browning et al. 2018 overlap for this set of variants? Let's look at all variants and then SAVs at delta >= 0.2.

In [159]:
introgressed_vars = data[(data['Vernot_allele_origin'] == 'introgressed') | (data['Browning_allele_origin'] == 'introgressed')]
len(introgressed_vars)

116144

In [160]:
len(introgressed_vars[introgressed_vars['Vernot_allele_origin'] == 'introgressed'])

68046

In [161]:
len(introgressed_vars[introgressed_vars['Browning_allele_origin'] == 'introgressed'])

98189

In [162]:
len(introgressed_vars[(introgressed_vars['Vernot_allele_origin'] == 'introgressed') & (introgressed_vars['Browning_allele_origin'] == 'introgressed')])

50091

In [163]:
introgressed_SAVs = data_2[(data_2['Vernot_allele_origin'] == 'introgressed') | (data_2['Browning_allele_origin'] == 'introgressed')]
len(introgressed_SAVs)

460

In [164]:
len(introgressed_SAVs[introgressed_SAVs['Vernot_allele_origin'] == 'introgressed'])

244

In [165]:
len(introgressed_SAVs[introgressed_SAVs['Browning_allele_origin'] == 'introgressed'])

386

In [166]:
len(introgressed_SAVs[(introgressed_SAVs['Vernot_allele_origin'] == 'introgressed') & (introgressed_SAVs['Browning_allele_origin'] == 'introgressed')])

170

## SAV Distribution <a class = 'anchor' id = 'SAVdistribution'></a>

Get distribution of archaic-specific variants for Figure 1. 

In [167]:
archaic_specific = data_2[(data_2['Vernot_allele_origin'] == 'archaic-specific')]
archaic_specific.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,300
Chagyrskaya,169
Denisovan,916
Late Neanderthal,94
Neanderthal,236
Other,128
Shared,112
Vindija,231


In [168]:
archaic_specific = data_2[(data_2['Browning_allele_origin'] == 'archaic-specific')]
archaic_specific.groupby(['distribution']).size().to_frame('distribution')

,distribution
distribution,
Altai,300
Chagyrskaya,169
Denisovan,916
Late Neanderthal,94
Neanderthal,236
Other,128
Shared,112
Vindija,231


Let's use parsimony to categorize the 'Other' distribution category. 

In [169]:
archaic_specific_other = archaic_specific[(archaic_specific['distribution'] == 'Other')]

DAC

In [170]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == True) & (archaic_specific_other['chagyrskaya_gt_boolean'] == True) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == False)])

5

DAV

In [171]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == True) & (archaic_specific_other['chagyrskaya_gt_boolean'] == False) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == True)])

7

DCV

In [172]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == False) & (archaic_specific_other['chagyrskaya_gt_boolean'] == True) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == True)])

2

DA

In [173]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == True) & (archaic_specific_other['chagyrskaya_gt_boolean'] == False) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == False)])

20

DC

In [174]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == False) & (archaic_specific_other['chagyrskaya_gt_boolean'] == True) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == False)])

2

DV

In [175]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == False) & (archaic_specific_other['chagyrskaya_gt_boolean'] == False) & (archaic_specific_other['denisovan_gt_boolean'] == True) & (archaic_specific_other['vindija_gt_boolean'] == True)])

2

AC

In [176]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == True) & (archaic_specific_other['chagyrskaya_gt_boolean'] == True) & (archaic_specific_other['denisovan_gt_boolean'] == False) & (archaic_specific_other['vindija_gt_boolean'] == False)])

38

AV

In [177]:
len(archaic_specific_other[(archaic_specific_other['altai_gt_boolean'] == True) & (archaic_specific_other['chagyrskaya_gt_boolean'] == False) & (archaic_specific_other['denisovan_gt_boolean'] == False) & (archaic_specific_other['vindija_gt_boolean'] == True)])

52

Let's add 38 (5+7+2+20+2+2) to the archaic common ancestor and 90 (38+52) to the Neanderthal common ancestor in Figure 1.

# SAV Genes <a class = 'anchor' id = 'SAVgenes'></a>

## Top 20 AG, AL, DG, and DL <a class = 'anchor' id = 'top20'></a>

Now let's take a look at the top 20 variants with the highest deltas for each of the four classes.

In [178]:
data_2_ag = data_2.sort_values(['ag_delta'], ascending = [False])
data_2_ag.head(20)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
78185,chr1,144923837,A,T,NaN,derived,snv,0/1,0/1,./.,0/1,True,True,False,True,Neanderthal,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PDE4DIP,1.12420,-1.40300,0.68045,3.12940,-0.264,1.00,0.97,0.0,0.0,1.00,-2,-16,-16,-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
302444,chr11,108810969,C,G,C,derived,snv,0/1,0/0,0/0,1/1,True,False,False,True,Other,yes,2.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,yes,8.0,143086.0,0.000056,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,DDX10,0.96014,0.30359,0.70688,1.82750,-0.175,1.00,0.01,0.0,0.0,1.00,1,4,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
4234,chr1,5935162,A,T,T,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NPHP4,1.02290,-0.24249,0.78155,1.66690,-0.557,1.00,0.91,0.0,0.0,1.00,-2,-8,-43,-44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
121279,chr1,218475625,T,A,A,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,436.0,5096.0,0.09,0.06,0.04,0.23,0.04,0.01,0.230019,yes,11299.0,143136.0,0.078939,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,RRP15,1.10360,-0.45285,0.62321,1.17020,-0.372,1.00,0.70,0.0,0.0,1.00,2,11,11,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0

In [179]:
data_2_al = data_2.sort_values(['al_delta'], ascending = [False])
data_2_al.head(20)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
607092,chr16,85105387,A,T,A,derived,snv,0/0,./.,./.,0/1,False,False,False,True,Vindija,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,KIAA0513,1.05040,-0.28542,0.241480,3.503700,0.533,0.86,1.00,0.0,0.0,1.00,6,2,-40,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
1326583,chr6,160482533,A,C,A,derived,snv,0/1,0/0,0/1,0/0,True,False,True,False,Other,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,IGF2R,0.84588,2.06550,0.141620,8.714900,0.533,0.02,1.00,0.0,0.0,1.00,19,2,44,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
1073592,chr4,71339723,G,A,G,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,1060.0,5096.0,0.21,0.20,0.13,0.28,0.14,0.25,0.290448,yes,26632.0,142744.0,0.186572,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MUC7,1.06550,-0.32621,1.789600,-0.773480,0.563,0.86,1.00,0.0,0.0,1.00,2,1,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,yes,ancient,ancient
1002905,chr3,14965507,A,G,G,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,4927.0,5096.0,0.97,1.00,0.92,1.00,0.96,0.94,1.000000,yes,134716.0,143286.0,0.940190,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,FGD5,0.98381,0.16707,0.070985,6.462300,-0.684,0.49,0.99,0.0,0.0,0.99,8,2,0,15,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1

In [180]:
data_2_dg = data_2.sort_values(['dg_delta'], ascending = [False])
data_2_dg.head(20)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
299707,chr11,104761100,T,C,C,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,CASP12,0.72169,1.259300,0.603180,1.33920,-0.282,0.0,0.00,1.00,0.00,1.00,46,-42,1,-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
642626,chr17,45485046,A,G,-,.,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,760.0,5096.0,0.15,0.33,0.08,0.07,0.11,0.17,0.058480,yes,11933.0,143208.0,0.083326,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,EFCAB13,0.87046,1.001500,1.068500,-0.41674,-0.364,0.0,0.00,0.99,0.16,0.99,-27,28,-1,30,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,yes,ancient,ancient
939053,chr20,61467445,C,T,C,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,541.0,5096.0,0.11,0.01,0.05,0.32,0.03,0.03,0.328460,yes,18606.0,143286.0,0.129852,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,COL9A3,1.05080,-0.382240,0.541080,3.00410,0.313,0.0,0.00,0.99,0.10,0.99,-39,-6,-2,-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
1376781,chr7,56088811,T,C,T,derived,snv,0/1,0/1,0/1,0/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PSPH,0.80477,0.777190,0.602230,1.16340,-1.431,0.0,0.00,0.99,0.62,0.99,1,26,1,-45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [181]:
data_2_dl = data_2.sort_values(['dl_delta'], ascending = [False])
data_2_dl.head(20)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
549948,chr15,99514264,C,A,C,derived,snv,0/0,0/0,0/0,0/1,False,False,False,True,Vindija,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PGPEP1L,1.28090,-1.094500,1.68290,-1.54260,0.650,0.00,0.00,0.01,1.00,1.00,-3,49,-3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
613603,chr17,1386153,A,C,A,derived,snv,0/1,0/1,./.,./.,True,True,False,False,Other,yes,886.0,5096.0,0.17,0.13,0.24,0.17,0.17,0.16,0.154971,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MYO1C,1.06990,-0.636630,0.51054,3.70140,0.334,0.00,0.00,0.92,1.00,1.00,11,19,46,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
549796,chr15,99442852,T,A,T,derived,snv,./.,0/0,0/1,0/0,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,IGF1R,0.73351,2.728600,0.18874,5.99420,0.460,0.04,0.00,0.00,1.00,1.00,-20,-31,-20,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
933419,chr20,54963211,C,G,C,derived,snv,0/0,./.,0/1,./.,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AURKA,0.68585,1.595000,0.15147,3.49930,0.585,0.00,0.26,0.41,1.00,1.00,-49,47,-49,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## N Genes <a class = 'anchor' id = 'ngenes'></a>

Now let's get a list of genes for all variants with delta >= 0.2.

In [182]:
all_genes_2 = data_2['annotation'].to_frame('gene')
all_genes_2 = all_genes_2.sort_values(['gene'], ascending = [True])
all_genes_2.head(10)

,gene
755832,A1BG
11555,AADACL4
1127491,AADAT
879182,AAMP
879179,AAMP
920210,AAR2
1263240,AARS2
636461,AATF
666602,AATK
562265,ABAT


In [183]:
len(all_genes_2)

5950

How many unique genes are contained in these vectors?

In [184]:
len(all_genes_2.drop_duplicates())

4242

Save this gene list.

In [185]:
all_genes_2.to_csv('../genes/all_genes_2.txt', sep="\t", header=False, index=False)

Repeat for delta >= 0.5.

In [186]:
all_genes_5 = data_5['annotation'].to_frame('gene')
all_genes_5 = all_genes_5.sort_values(['gene'], ascending = [True])
all_genes_5.head(10)

,gene
879182,AAMP
920210,AAR2
636461,AATF
656165,ABCA5
715609,ABCA7
1433756,ABCB8
570288,ABCC1
570362,ABCC1
579405,ABCC12
570587,ABCC6


In [187]:
len(all_genes_5)

1049

In [188]:
len(all_genes_5.drop_duplicates())

962

In [189]:
all_genes_5.to_csv('../genes/all_genes_5.txt', sep="\t", header=False, index=False)

## Distribution of N Genes <a class = 'anchor' id = 'ngenedistribution'></a>

Let's get the distribution of N SAVs per gene.

In [190]:
SAVs_per_gene = data_2.groupby(by=['annotation']).size().reset_index(name='N_SAVs')
SAVs_per_gene.groupby('N_SAVs').size().to_frame('N')

,N
N_SAVs,
1,3111
2,769
3,246
4,71
5,19
6,14
7,6
8,1
9,2


In [191]:
SAVs_per_gene[SAVs_per_gene['N_SAVs']==7]

,annotation,N_SAVs
682,CDH13,7
686,CDH23,7
907,CSMD1,7
1059,DNAH17,7
1662,HLA-DPA1,7
1942,LAMA5,7


In [192]:
SAVs_per_gene[SAVs_per_gene['N_SAVs']==8]

,annotation,N_SAVs
108,ADARB2,8


In [193]:
SAVs_per_gene[SAVs_per_gene['N_SAVs']==9]

,annotation,N_SAVs
3198,SDK1,9
4093,WWOX,9


In [194]:
SAVs_per_gene[SAVs_per_gene['N_SAVs']==10]

,annotation,N_SAVs
1663,HLA-DPB1,10


In [195]:
SAVs_per_gene[SAVs_per_gene['N_SAVs']==11]

,annotation,N_SAVs
829,CNTNAP2,11
3508,SSPO,11


Let's take a closer look at the genes with multiple SAVs. 

How often do genes with >= 2 SAVs match in distribution or allele origin?

In [196]:
multiple_SAVs_per_gene = data_2[data_2.duplicated(subset=['annotation','distribution'])]
multiple_SAVs_per_gene.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
224,chr1,982099,T,C,T,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,142512.0,0.000007,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AGRN,0.98257,0.225520,0.319140,6.0143,-0.375,0.00,0.00,0.04,0.48,0.48,-48,-6,16,-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
1176,chr1,1872053,T,C,T,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,7.0,143266.0,0.000049,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.000049,CFAP74,NaN,NaN,NaN,NaN,-0.669,0.22,0.00,0.00,0.00,0.22,49,-2,49,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
1889,chr1,2526280,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,2.0,143228.0,0.000014,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MMEL1,0.98394,0.126020,0.768240,1.5689,-0.242,0.00,0.01,0.03,0.25,0.25,-37,39,-37,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
3354,chr1,3520127,T,C,C,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,4811.0,5096.0,0.94,0.98,0.97,0.86,0.99,0.95,0.855750,yes,134537.0,143196.0,0.939530,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MEGF6,0.95847,0.458030,0.738560,2.2728,-1.183,0.00,0.00,0.00,0.24,0.24,-28,47,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [197]:
len(multiple_SAVs_per_gene)

591

In [198]:
591/2839

0.20817189151109547

In [199]:
multiple_SAVs_per_gene = data_2[data_2.duplicated(subset=['annotation','Vernot_allele_origin'])]
multiple_SAVs_per_gene.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
218,chr1,980460,G,A,G,derived,snv,1/1,./.,1/1,1/1,True,False,True,True,Other,yes,4642.0,5096.0,0.91,1.00,0.93,0.77,0.93,0.97,0.781676,yes,128061.0,143108.0,0.894856,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,AGRN,0.98257,0.22552,0.319140,6.0143,0.561,0.00,0.00,0.03,0.43,0.43,30,-33,30,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14.0,yes,ancient,ancient
479,chr1,1226233,G,A,g,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,211.0,5096.0,0.04,0.00,0.00,0.15,0.01,0.00,0.160819,yes,5454.0,143298.0,0.038060,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,SCNN1D,1.10050,-0.78105,1.237300,-1.3261,0.299,0.40,0.01,0.00,0.00,0.40,-10,42,41,-44,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,yes,low-confidence ancient,low-confidence ancient
1176,chr1,1872053,T,C,T,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,7.0,143266.0,0.000049,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.000049,CFAP74,NaN,NaN,NaN,NaN,-0.669,0.22,0.00,0.00,0.00,0.22,49,-2,49,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
3281,chr1,3498077,C,T,C,derived,snv,0/0,1/1,0/0,0/0,False,True,False,False,Chagyrskaya,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,2.0,143324.0,0.000014,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,MEGF6,0.95847,0.45803,0.738560,2.2728,-1.579,0.00,0.00,0.30,0.00,0.30,-2,-44,-1,-44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [200]:
len(multiple_SAVs_per_gene)

810

In [201]:
810/2839

0.28531172948221206

In [202]:
data_2.groupby(by=['Vernot_introgressed','Vernot_allele_origin']).size().reset_index(name='count')

,Vernot_introgressed,Vernot_allele_origin,count
0,no,ancient,2252
1,no,archaic-specific,2169
2,no,low-confidence ancient,1268
3,yes,archaic-specific,17
4,yes,introgressed,244


In [203]:
data_2.groupby(by=['present_in_1KG','Vernot_allele_origin']).size().reset_index(name='count')

,present_in_1KG,Vernot_allele_origin,count
0,no,archaic-specific,2186
1,no,introgressed,7
2,no,low-confidence ancient,232
3,yes,ancient,2252
4,yes,introgressed,237
5,yes,low-confidence ancient,1036


In [204]:
data_2.groupby(by=['Browning_introgressed','Browning_allele_origin']).size().reset_index(name='count')

,Browning_introgressed,Browning_allele_origin,count
0,no,ancient,2195
1,no,archaic-specific,2180
2,no,low-confidence ancient,1183
3,yes,archaic-specific,6
4,yes,introgressed,386


In [205]:
data_2.groupby(by=['present_in_1KG','Browning_allele_origin']).size().reset_index(name='count')

,present_in_1KG,Browning_allele_origin,count
0,no,archaic-specific,2186
1,no,introgressed,9
2,no,low-confidence ancient,230
3,yes,ancient,2195
4,yes,introgressed,377
5,yes,low-confidence ancient,953


## N Genes by Origin <a class = 'anchor' id = 'ngenesbyorigin'></a>

Let's get gene lists for archaic-specific variants. Note that our current filtering has resulted in complete agreement between Vernot and Browning for archaic-specific variants so we don't need to specify the introgression callset.

In [206]:
archaic_specific_genes_2 = data_2.loc[(data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_genes_2 = archaic_specific_genes_2.sort_values(['gene'], ascending = [True])
archaic_specific_genes_2.to_csv('../genes/archaic_specific_genes_2.txt', sep="\t", header=False, index=False)

In [207]:
archaic_specific_genes_5 = data_5.loc[(data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_genes_5 = archaic_specific_genes_5.sort_values(['gene'], ascending = [True])
archaic_specific_genes_5.to_csv('../genes/archaic_specific_genes_5.txt', sep="\t", header=False, index=False)

How many unique genes are contained in archaic-specific vectors?

In [208]:
len(archaic_specific_genes_2.drop_duplicates())

1907

In [209]:
len(archaic_specific_genes_5.drop_duplicates())

392

And for ancient and introgressed alleles.

In [210]:
Vernot_ancient_genes_2 = data_2.loc[(data_2['Vernot_allele_origin']=='ancient'), 'annotation'].to_frame('gene')
Vernot_ancient_genes_2 = Vernot_ancient_genes_2.sort_values(['gene'], ascending = [True])
Vernot_ancient_genes_2.to_csv('../genes/Vernot_ancient_genes_2.txt', sep="\t", header=False, index=False)

In [211]:
Vernot_ancient_genes_5 = data_5.loc[(data_5['Vernot_allele_origin']=='ancient'), 'annotation'].to_frame('gene')
Vernot_ancient_genes_5 = Vernot_ancient_genes_5.sort_values(['gene'], ascending = [True])
Vernot_ancient_genes_5.to_csv('../genes/Vernot_ancient_genes_5.txt', sep="\t", header=False, index=False)

In [212]:
Browning_ancient_genes_2 = data_2.loc[(data_2['Browning_allele_origin']=='ancient'), 'annotation'].to_frame('gene')
Browning_ancient_genes_2 = Browning_ancient_genes_2.sort_values(['gene'], ascending = [True])
Browning_ancient_genes_2.to_csv('../genes/Browning_ancient_genes_2.txt', sep="\t", header=False, index=False)

In [213]:
Browning_ancient_genes_5 = data_5.loc[(data_5['Browning_allele_origin']=='ancient'), 'annotation'].to_frame('gene')
Browning_ancient_genes_5 = Browning_ancient_genes_5.sort_values(['gene'], ascending = [True])
Browning_ancient_genes_5.to_csv('../genes/Browning_ancient_genes_5.txt', sep="\t", header=False, index=False)

Check the numebr of ancient genes.

In [214]:
len(Vernot_ancient_genes_2.drop_duplicates())

1896

In [215]:
len(Vernot_ancient_genes_5.drop_duplicates())

370

In [216]:
len(Browning_ancient_genes_2.drop_duplicates())

1856

In [217]:
len(Browning_ancient_genes_5.drop_duplicates())

363

In [218]:
Vernot_introgressed_genes_2 = data_2.loc[(data_2['Vernot_allele_origin']=='introgressed'), 'annotation'].to_frame('gene')
Vernot_introgressed_genes_2 = Vernot_introgressed_genes_2.sort_values(['gene'], ascending = [True])
Vernot_introgressed_genes_2.to_csv('../genes/Vernot_introgressed_genes_2.txt', sep="\t", header=False, index=False)

In [219]:
Vernot_introgressed_genes_5 = data_5.loc[(data_5['Vernot_allele_origin']=='introgressed'), 'annotation'].to_frame('gene')
Vernot_introgressed_genes_5 = Vernot_introgressed_genes_5.sort_values(['gene'], ascending = [True])
Vernot_introgressed_genes_5.to_csv('../genes/Vernot_introgressed_genes_5.txt', sep="\t", header=False, index=False)

In [220]:
Browning_introgressed_genes_2 = data_2.loc[(data_2['Browning_allele_origin']=='introgressed'), 'annotation'].to_frame('gene')
Browning_introgressed_genes_2 = Browning_introgressed_genes_2.sort_values(['gene'], ascending = [True])
Browning_introgressed_genes_2.to_csv('../genes/Browning_introgressed_genes_2.txt', sep="\t", header=False, index=False)

In [221]:
Browning_introgressed_genes_5 = data_5.loc[(data_5['Browning_allele_origin']=='introgressed'), 'annotation'].to_frame('gene')
Browning_introgressed_genes_5 = Browning_introgressed_genes_5.sort_values(['gene'], ascending = [True])
Browning_introgressed_genes_5.to_csv('../genes/Browning_introgressed_genes_5.txt', sep="\t", header=False, index=False)

Check the numebr of introgressed genes.

In [222]:
len(Vernot_introgressed_genes_2.drop_duplicates())

239

In [223]:
len(Vernot_introgressed_genes_5.drop_duplicates())

59

In [224]:
len(Browning_introgressed_genes_2.drop_duplicates())

361

In [225]:
len(Browning_introgressed_genes_5.drop_duplicates())

67

## N Genes by Distribution <a class = 'anchor' id = 'ngenesbydistribution'></a>

In [226]:
altai_genes = data_2.loc[(data_2['altai_gt_boolean']==True), 'annotation'].to_frame('gene')
chagyrskaya_genes = data_2.loc[(data_2['chagyrskaya_gt_boolean']==True), 'annotation'].to_frame('gene')
denisovan_genes = data_2.loc[(data_2['denisovan_gt_boolean']==True), 'annotation'].to_frame('gene')
vindija_genes = data_2.loc[(data_2['vindija_gt_boolean']==True), 'annotation'].to_frame('gene')

In [227]:
altai_genes = altai_genes.drop_duplicates()
chagyrskaya_genes = chagyrskaya_genes.drop_duplicates()
denisovan_genes = denisovan_genes.drop_duplicates()
vindija_genes = vindija_genes.drop_duplicates()

In [228]:
len(altai_genes)

2914

In [229]:
len(chagyrskaya_genes)

2755

In [230]:
len(denisovan_genes)

2910

In [231]:
len(vindija_genes)

2889

## Archaic-Specific Genes <a class = 'anchor' id = 'archaicspecificgenes'></a>

Now let's dig into the various archaic-specific distributions.

In [232]:
archaic_specific_altai_genes_2 = data_2.loc[(data_2['distribution']=='Altai') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_altai_genes_2 = archaic_specific_altai_genes_2.sort_values(['gene'], ascending = [True])

archaic_specific_chagyrskaya_genes_2 = data_2.loc[(data_2['distribution']=='Chagyrskaya') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_chagyrskaya_genes_2 = archaic_specific_chagyrskaya_genes_2.sort_values(['gene'], ascending = [True])

archaic_specific_denisovan_genes_2 = data_2.loc[(data_2['distribution']=='Denisovan') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_denisovan_genes_2 = archaic_specific_denisovan_genes_2.sort_values(['gene'], ascending = [True])

archaic_specific_vindija_genes_2 = data_2.loc[(data_2['distribution']=='Vindija') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_vindija_genes_2 = archaic_specific_vindija_genes_2.sort_values(['gene'], ascending = [True])

In [233]:
archaic_specific_late_neanderthal_genes_2 = data_2.loc[(data_2['distribution']=='Late Neanderthal') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_late_neanderthal_genes_2 = archaic_specific_late_neanderthal_genes_2.sort_values(['gene'], ascending = [True])

archaic_specific_neanderthal_genes_2 = data_2.loc[(data_2['distribution']=='Neanderthal') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_neanderthal_genes_2 = archaic_specific_neanderthal_genes_2.sort_values(['gene'], ascending = [True])

archaic_specific_shared_genes_2 = data_2.loc[(data_2['distribution']=='Shared') & (data_2['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_shared_genes_2 = archaic_specific_shared_genes_2.sort_values(['gene'], ascending = [True])

In [234]:
archaic_specific_altai_genes_2.to_csv('../genes/archaic_specific_altai_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_chagyrskaya_genes_2.to_csv('../genes/archaic_specific_chagyrskaya_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_denisovan_genes_2.to_csv('../genes/archaic_specific_denisovan_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_vindija_genes_2.to_csv('../genes/archaic_specific_vindija_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_late_neanderthal_genes_2.to_csv('../genes/archaic_specific_late_neanderthal_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_neanderthal_genes_2.to_csv('../genes/archaic_specific_neanderthal_genes_2.txt', sep="\t", header=False, index=False)
archaic_specific_shared_genes_2.to_csv('../genes/archaic_specific_shared_genes_2.txt', sep="\t", header=False, index=False)

How many unique genes are contained in lineage-specific vectors?

In [235]:
archaic_specific_altai_genes_2_no_dups = archaic_specific_altai_genes_2.drop_duplicates()
len(archaic_specific_altai_genes_2_no_dups)

283

In [236]:
archaic_specific_chagyrskaya_genes_2_no_dups = archaic_specific_chagyrskaya_genes_2.drop_duplicates()
len(archaic_specific_chagyrskaya_genes_2_no_dups)

165

In [237]:
archaic_specific_denisovan_genes_2_no_dups = archaic_specific_denisovan_genes_2.drop_duplicates()
len(archaic_specific_denisovan_genes_2_no_dups)

859

In [238]:
archaic_specific_vindija_genes_2_no_dups = archaic_specific_vindija_genes_2.drop_duplicates()
len(archaic_specific_vindija_genes_2_no_dups)

228

In [239]:
archaic_specific_late_neanderthal_genes_2_no_dups = archaic_specific_late_neanderthal_genes_2.drop_duplicates()
len(archaic_specific_late_neanderthal_genes_2_no_dups)

93

In [240]:
archaic_specific_neanderthal_genes_2_no_dups = archaic_specific_neanderthal_genes_2.drop_duplicates()
len(archaic_specific_neanderthal_genes_2_no_dups)

227

In [241]:
archaic_specific_shared_genes_2_no_dups = archaic_specific_shared_genes_2.drop_duplicates()
len(archaic_specific_shared_genes_2_no_dups)

106

Repeat for delta >= 0.5.

In [242]:
archaic_specific_altai_genes_5 = data_5.loc[(data_5['distribution']=='Altai') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_altai_genes_5 = archaic_specific_altai_genes_5.sort_values(['gene'], ascending = [True])

archaic_specific_chagyrskaya_genes_5 = data_5.loc[(data_5['distribution']=='Chagyrskaya') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_chagyrskaya_genes_5 = archaic_specific_chagyrskaya_genes_5.sort_values(['gene'], ascending = [True])

archaic_specific_denisovan_genes_5 = data_5.loc[(data_5['distribution']=='Denisovan') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_denisovan_genes_5 = archaic_specific_denisovan_genes_5.sort_values(['gene'], ascending = [True])

archaic_specific_vindija_genes_5 = data_5.loc[(data_5['distribution']=='Vindija') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_vindija_genes_5 = archaic_specific_vindija_genes_5.sort_values(['gene'], ascending = [True])

In [243]:
archaic_specific_late_neanderthal_genes_5 = data_5.loc[(data_5['distribution']=='Late Neanderthal') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_late_neanderthal_genes_5 = archaic_specific_late_neanderthal_genes_5.sort_values(['gene'], ascending = [True])

archaic_specific_neanderthal_genes_5 = data_5.loc[(data_5['distribution']=='Neanderthal') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_neanderthal_genes_5 = archaic_specific_neanderthal_genes_5.sort_values(['gene'], ascending = [True])

archaic_specific_shared_genes_5 = data_5.loc[(data_5['distribution']=='Shared') & (data_5['Vernot_allele_origin']=='archaic-specific'), 'annotation'].to_frame('gene')
archaic_specific_shared_genes_5 = archaic_specific_shared_genes_5.sort_values(['gene'], ascending = [True])

In [244]:
archaic_specific_altai_genes_5.to_csv('../genes/archaic_specific_altai_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_chagyrskaya_genes_5.to_csv('../genes/archaic_specific_chagyrskaya_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_denisovan_genes_5.to_csv('../genes/archaic_specific_denisovan_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_vindija_genes_5.to_csv('../genes/archaic_specific_vindija_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_late_neanderthal_genes_5.to_csv('../genes/archaic_specific_late_neanderthal_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_neanderthal_genes_5.to_csv('../genes/archaic_specific_neanderthal_genes_5.txt', sep='\t', header=False, index=False)
archaic_specific_shared_genes_5.to_csv('../genes/archaic_specific_shared_genes_5.txt', sep='\t', header=False, index=False)

In [245]:
len(archaic_specific_altai_genes_5.drop_duplicates())

52

In [246]:
len(archaic_specific_chagyrskaya_genes_5.drop_duplicates())

27

In [247]:
len(archaic_specific_denisovan_genes_5.drop_duplicates())

161

In [248]:
len(archaic_specific_vindija_genes_5.drop_duplicates())

55

In [249]:
len(archaic_specific_late_neanderthal_genes_5.drop_duplicates())

10

In [250]:
len(archaic_specific_neanderthal_genes_5.drop_duplicates())

36

In [251]:
len(archaic_specific_shared_genes_5.drop_duplicates())

26

## Gene Overlap <a class = 'anchor' id = 'geneoverlap'></a>

Do genes overlap in the archaic-specific sets?

In [252]:
neanderthal_altai_genes = pd.merge(archaic_specific_neanderthal_genes_2_no_dups, archaic_specific_altai_genes_2_no_dups)
len(neanderthal_altai_genes)

5

In [253]:
neanderthal_chagyrskaya_genes = pd.merge(archaic_specific_neanderthal_genes_2_no_dups, archaic_specific_chagyrskaya_genes_2_no_dups)
len(neanderthal_chagyrskaya_genes)

1

In [254]:
neanderthal_denisovan_genes = pd.merge(archaic_specific_neanderthal_genes_2_no_dups, archaic_specific_denisovan_genes_2_no_dups)
len(neanderthal_denisovan_genes)

20

In [255]:
neanderthal_vindija_genes = pd.merge(archaic_specific_neanderthal_genes_2_no_dups, archaic_specific_vindija_genes_2_no_dups)
len(neanderthal_vindija_genes)

3

In [256]:
altai_chagyrskaya_genes = pd.merge(archaic_specific_altai_genes_2_no_dups, archaic_specific_chagyrskaya_genes_2_no_dups)
len(altai_chagyrskaya_genes)

9

In [257]:
altai_denisovan_genes = pd.merge(archaic_specific_altai_genes_2_no_dups, archaic_specific_denisovan_genes_2_no_dups)
len(altai_denisovan_genes)

34

In [258]:
altai_vindija_genes = pd.merge(archaic_specific_altai_genes_2_no_dups, archaic_specific_vindija_genes_2_no_dups)
len(altai_vindija_genes)

3

In [259]:
chagyrskaya_denisovan_genes = pd.merge(archaic_specific_chagyrskaya_genes_2_no_dups, archaic_specific_denisovan_genes_2_no_dups)
len(chagyrskaya_denisovan_genes)

20

In [260]:
chagyrskaya_vindija_genes = pd.merge(archaic_specific_chagyrskaya_genes_2_no_dups, archaic_specific_vindija_genes_2_no_dups)
len(chagyrskaya_vindija_genes)

6

In [261]:
denisovan_vindija_genes = pd.merge(archaic_specific_denisovan_genes_2_no_dups, archaic_specific_vindija_genes_2_no_dups)
len(denisovan_vindija_genes)

19

# Comparisons <a class = 'anchor' id = 'comparisons'></a>

## SAV Genes and DR Genes <a class = 'anchor' id = 'drgenes'></a>

Let's take a second to compare genes with SAVs with divergently regulated genes from Colbran et al. 2019.

In [262]:
altai_genes = data_2.loc[(data_2['altai_gt_boolean']==True), 'annotation'].to_frame('gene')
denisovan_genes = data_2.loc[(data_2['denisovan_gt_boolean']==True), 'annotation'].to_frame('gene')
vindija_genes = data_2.loc[(data_2['vindija_gt_boolean']==True), 'annotation'].to_frame('gene')

In [263]:
altai_genes = altai_genes.drop_duplicates()
denisovan_genes = denisovan_genes.drop_duplicates()
vindija_genes = vindija_genes.drop_duplicates()

In [264]:
len(altai_genes)

2914

In [265]:
len(denisovan_genes)

2910

In [266]:
len(vindija_genes)

2889

In [267]:
altai_DR_genes = pd.read_csv('../DR_genes/altai_DR_genes.txt', sep = '\t', names = ['gene'])
denisovan_DR_genes = pd.read_csv('../DR_genes/denisovan_DR_genes.txt', sep = '\t', names = ['gene'])
vindija_DR_genes = pd.read_csv('../DR_genes/vindija_DR_genes.txt', sep = '\t', names = ['gene'])

In [268]:
len(altai_DR_genes)

1419

In [269]:
len(denisovan_DR_genes)

1171

In [270]:
len(vindija_DR_genes)

1484

In [271]:
altai_DR_overlap = pd.merge(altai_genes, altai_DR_genes)
len(altai_DR_overlap)

213

In [272]:
denisovan_DR_overlap = pd.merge(denisovan_genes, denisovan_DR_genes)
len(denisovan_DR_overlap)

154

In [273]:
vindija_DR_overlap = pd.merge(vindija_genes, vindija_DR_genes)
len(vindija_DR_overlap)

222

In [274]:
altai_DR_overlap.to_csv('../DR_genes/altai_DR_overlap.txt', sep='\t', header=False, index=False)
denisovan_DR_overlap.to_csv('../DR_genes/denisovan_DR_overlap.txt', sep='\t', header=False, index=False)
vindija_DR_overlap.to_csv('../DR_genes/vindija_DR_overlap.txt', sep='\t', header=False, index=False)

## SAV Genes and Circadian Genes <a class = 'anchor' id = 'circadiangenes'></a>

Let's also compare the overlap between genes involved in circadian biology (Velázquez-Arcelay et al., in prep) and our genes with SAVs.

In [275]:
archaic_specific_genes_2_no_dups = archaic_specific_genes_2.drop_duplicates()
Vernot_introgressed_genes_2_no_dups = Vernot_introgressed_genes_2.drop_duplicates()

In [276]:
circadian_genes = pd.read_csv('../circadian_genes/circadian_genes.txt', sep = '\t', header = 0)
circadian_genes = circadian_genes[['GeneName']]
circadian_genes.rename(columns={'GeneName': 'gene'}, inplace=True)

In [277]:
circadian_archaic_specific_overlap = pd.merge(archaic_specific_genes_2_no_dups, circadian_genes)
len(circadian_archaic_specific_overlap)

27

In [278]:
circadian_introgressed_overlap = pd.merge(Vernot_introgressed_genes_2_no_dups, circadian_genes)
len(circadian_introgressed_overlap)

4

In [279]:
circadian_archaic_specific_overlap.to_csv('../circadian_genes/circadian_genes_archaic_specific_overlap.txt', sep='\t', header=False, index=False)
circadian_introgressed_overlap.to_csv('../circadian_genes/circadian_genes_introgressed_overlap.txt', sep='\t', header=False, index=False)

Let's also subset the main dataframe for any variants that occur in circadian genes.

In [280]:
circadian_genes.head(5)

,gene
0,PER3
1,RERE
2,DNAJC16
3,ECE1
4,HTR1D


In [281]:
circadian_mask = data['annotation'].isin(circadian_genes['gene'])
circadian_data = data[circadian_mask]
circadian_data.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
7274,chr1,7845695,T,C,C,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,3690.0,5096.0,0.72,0.76,0.66,0.78,0.76,0.65,0.785575,yes,98348.0,143106.0,0.687239,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PER3,1.0116,-0.10855,0.63671,2.5313,-0.215,0.00,0.0,0.0,0.02,0.02,36,19,-49,19,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,16.0,yes,ancient,ancient
7275,chr1,7846527,A,C,A,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,2249.0,5096.0,0.44,0.45,0.37,0.62,0.26,0.38,0.639376,yes,59442.0,142192.0,0.418040,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PER3,1.0116,-0.10855,0.63671,2.5313,-0.379,0.00,0.0,0.0,0.00,0.00,13,-7,-12,-7,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,20.0,yes,ancient,ancient
7276,chr1,7849200,A,T,A,derived,snv,1/1,./.,1/1,1/1,True,False,True,True,Other,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,2.0,125184.0,0.000016,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PER3,1.0116,-0.10855,0.63671,2.5313,0.172,0.00,0.0,0.0,0.00,0.00,22,-20,-2,-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
7277,chr1,7853294,A,G,A,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,6.0,143348.0,0.000042,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,PER3,1.0116,-0.10855,0.63671,2.5313,-0.371,0.00,0.0,0.0,0.00,0.00,-47,17,-47,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [282]:
len(circadian_data)

31775

In [283]:
circadian_data.to_csv('../circadian_genes/all_circadian_variants.txt', sep='\t', header=True, index=False)

Now just the SAVs.

In [284]:
circadian_SAVs = circadian_data[circadian_data['delta_max'] >= 0.2]

In [285]:
len(circadian_SAVs)

91

In [286]:
circadian_SAVs.head(5)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
18537,chr1,21588603,C,G,G,ancestral,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,3942.0,5096.0,0.77,0.89,0.61,0.91,0.69,0.69,0.932749,yes,99055.0,143024.0,0.692576,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ECE1,0.69508,2.3066,0.171130,4.9124,-0.748,0.22,0.00,0.00,0.00,0.22,47,0,-44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,ancient,ancient
25058,chr1,32797241,G,C,G,derived,snv,1/1,1/1,1/1,1/1,True,True,True,True,Shared,yes,90.0,5096.0,0.02,0.00,0.00,0.06,0.01,0.00,0.067251,yes,2708.0,143162.0,0.018916,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,HDAC1,0.50102,3.0373,0.205120,3.9837,-1.113,0.21,0.13,0.00,0.00,0.21,6,40,-11,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
79980,chr1,151796494,G,A,G,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,6.0,143212.0,0.000042,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,RORC,0.63127,2.2852,0.033914,4.8612,0.561,0.00,0.00,0.20,0.00,0.20,2,-43,2,-43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
80982,chr1,154542319,G,A,G,derived,snv,0/0,0/0,0/1,0/0,False,False,True,False,Denisovan,yes,1.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000975,yes,5.0,143174.0,0.000035,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,CHRNB2,0.67329,2.1034,0.455610,2.1139,-0.752,0.00,0.00,0.00,0.46,0.46,-36,-1,-13,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [287]:
circadian_SAVs.to_csv('../circadian_genes/all_circadian_SAVs.txt', sep='\t', header=True, index=False)

# Phenotype Enrichment <a class = 'anchor' id = 'phenotypeenrichment'></a>

Let's subset data for our phenotype enrichment analyses. 

In [288]:
enrichment_data = data[['delta_max','annotation','distribution','Vernot_allele_origin','Browning_allele_origin']]

In [289]:
enrichment_data.to_csv('archaic_data_for_enrichment.txt', sep='\t', header=True, index=False)

We have plenty of GWAS and HPO terms. Let's make life a bit easier by categorizing them into systems starting with GWAS. Write a function to generate a new enrichment summary file with an added column that lists the system per trait.

In [290]:
def map_GWAS_systems():
    GWAS_mapping = pd.read_csv(f'../phenotype_enrichment/ontologies/GWAS_terms_and_systems.txt', sep = '\t', header = 0)
    GWAS_system = pd.Series(GWAS_mapping['system'].values, index = GWAS_mapping['GWAS_term']).to_dict()
    setnames = ['altai','archaic_specific','Browning_introgressed','chagyrskaya','denisovan','neanderthal','shared','Vernot_introgressed','vindija']
    
    for set in setnames:
        dataframe = pd.read_csv(f'../phenotype_enrichment/enrichment/{set}_GWAS_enrichment.txt', sep = '\t', header = 0)
        dataframe['system'] = dataframe['label'].map(GWAS_system)
        dataframe.to_csv(f'../phenotype_enrichment/enrichment/{set}_GWAS_enrichment_with_system.txt', sep='\t', header=True, index=False)   

Run the function.

In [291]:
map_GWAS_systems()

Repeat with HPO.

In [292]:
def map_HPO_systems():
    HPO_mapping = pd.read_csv(f'../phenotype_enrichment/ontologies/HPO_terms_and_systems.txt', sep = '\t', header = 0)
    HPO_system = pd.Series(HPO_mapping['system'].values, index = HPO_mapping['HPO_term']).to_dict()
    setnames = ['altai','archaic_specific','Browning_introgressed','chagyrskaya','denisovan','neanderthal','shared','Vernot_introgressed','vindija']
    
    for set in setnames:
        dataframe = pd.read_csv(f'../phenotype_enrichment/enrichment/{set}_HPO_enrichment.txt', sep = '\t', header = 0)
        dataframe['system'] = dataframe['label'].map(HPO_system)
        dataframe.to_csv(f'../phenotype_enrichment/enrichment/{set}_HPO_enrichment_with_system.txt', sep='\t', header=True, index=False)   

In [293]:
map_HPO_systems()

Now it's time to get FDR corrected p-values for our gene enrichment analyses. Let's write a function to start.

In [294]:
fdr_table = []

In [295]:
def reportFDRcorrectedPthreshold(set_name, ontology, q_value_threshold, resolution=0.0001, minStart=0):
    fdr_empiric = pd.read_csv(f'../phenotype_enrichment/empiric_FDR/{set_name}_{ontology}_empiric_FDR.txt', sep = '\t', header = None, index_col = 0)
    obs = pd.read_csv(f'../phenotype_enrichment/enrichment/{set_name}_{ontology}_enrichment.txt', sep = '\t')

    fdr_threshold = []
    for i in np.arange(minStart,0.05,resolution):
        
        observed_positive = sum(obs['p_value'] <= i)
        average_false_positive = (fdr_empiric <= i).sum().mean()
        q = average_false_positive/observed_positive
        fdr_threshold.append([set_name, ontology, q_value_threshold, i, observed_positive, average_false_positive, q])
        
        if (q != np.inf) & (q > q_value_threshold):
            break
    
    threshold = fdr_threshold[-2]
    fdr_table.append(threshold)
    #fdr_threshold = pd.DataFrame(fdr_threshold, columns = ['pval_threshold','obsPos','avgFalsePos','q'])
    #return fdr_threshold.tail(2).head(1)

Now to generate all the combinations for which to get a corrected p-value.

In [296]:
combinations = [(set_name,ontology,q_value_threshold) for set_name in ['altai','archaic_specific','Browning_introgressed','chagyrskaya','denisovan','neanderthal','shared','Vernot_introgressed','vindija'] for ontology in ['GWAS','HPO'] for q_value_threshold in [0.05,0.1]]

And run!

In [297]:
[reportFDRcorrectedPthreshold(set_name, ontology, q_value_threshold) for set_name, ontology, q_value_threshold in combinations]

/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_47833/47096580.py:10: RuntimeWarning: inva

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [298]:
fdr_table = pd.DataFrame(fdr_table, columns = ['set', 'ontology', 'q_value_threshold', 'p_value_threshold','observed_positive','average_false_positive','q'])
fdr_table

,set,ontology,q_value_threshold,p_value_threshold,observed_positive,average_false_positive,q
0,altai,GWAS,0.05,0.0009,1,0.000,0.000000
1,altai,GWAS,0.10,0.0009,1,0.000,0.000000
2,altai,HPO,0.05,0.0010,0,0.221,inf
3,altai,HPO,0.10,0.0010,0,0.221,inf
4,archaic_specific,GWAS,0.05,0.0009,2,0.000,0.000000
5,archaic_specific,GWAS,0.10,0.0009,2,0.000,0.000000
6,archaic_specific,HPO,0.05,0.0049,48,1.958,0.040792
7,archaic_specific,HPO,0.10,0.0129,74,7.026,0.094946
8,Browning_introgressed,GWAS,0.05,0.0019,7,0.223,0.031857
9,Browning_introgressed,GWAS,0.10,0.0069,19,1.720,0.090526


# Novel Isoforms <a class = 'anchor' id = 'novelisoforms'></a>

Let's subset the main dataframe to a simpler file to estimate the effects of a given SAV on the canonical transcript/protein.

In [299]:
isoform_builder_input = data_2[['chrom','pos','ref_allele','alt_allele','annotation','ag_delta','al_delta','dg_delta','dl_delta','ag_pos','al_pos','dg_pos','dl_pos']]
isoform_builder_input.head(5)

,chrom,pos,ref_allele,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
24,chr1,864726,T,A,SAMD11,0.0,0.00,0.02,0.22,17,-2,43,-2
65,chr1,875708,C,G,SAMD11,0.0,0.00,0.29,0.01,-43,33,0,12
130,chr1,909768,A,G,PLEKHN1,0.0,0.29,0.13,0.03,7,26,-24,1
155,chr1,956622,C,T,AGRN,0.0,0.00,0.24,0.02,-2,50,-2,40
210,chr1,975133,T,A,AGRN,0.0,0.00,0.22,0.08,-3,14,-3,14


In [300]:
isoform_builder_input.to_csv('../novel_isoforms/isoform_builder_input.txt', sep = '\t', header = False, index = False)

Let's analyze the data.

In [301]:
novel_isoforms = pd.read_csv('../novel_isoforms/novel_isoforms.txt', sep = '\t', header = 0)
novel_isoforms['pos'] = novel_isoforms['pos']
novel_isoforms.head(10)

,chrom,pos,ref_allele,alt_allele,gene,strand,consequence,delta_pos,exon_number,exonic_intronic,sequence_effect,canonical_transcript_len,novel_transcript_len,canonical_len-novel_len,n_amino_acids,canonical_start_codon,novel_start_codon,n_premature_stops,result
0,chr1,864726,T,A,SAMD11,+,donor loss,-2,2,intronic,"intron not retained, variant excluded",2551,2551,0,681,ATG,ATG,.,No effect
1,chr1,875708,C,G,SAMD11,+,donor gain,0,7,intronic,"upstream effect, variant excluded, sequence ad...",2551,3419,868,681,ATG,ATG,8,Longer protein w/ PTCs
2,chr1,909768,A,G,PLEKHN1,+,acceptor loss,26,16,intronic,"exon not skipped, variant excluded",2404,2404,0,611,ATG,ATG,.,No effect
3,chr1,956622,C,T,AGRN,+,donor gain,-2,1,intronic,"upstream effect, variant excluded, sequence ad...",7323,8190,867,1907,ATG,CCG,.,Longer UTR
4,chr1,975133,T,A,AGRN,+,donor gain,-3,3,intronic,"upstream effect, variant excluded, sequence ad...",7323,11749,4426,1907,ATG,ATG,51,Longer protein w/ PTCs
5,chr1,980460,G,A,AGRN,+,donor loss,0,12,intronic,"intron not retained, variant excluded",7323,7323,0,1907,ATG,ATG,.,No effect
6,chr1,982099,T,C,AGRN,+,donor loss,-6,18,exonic,"intron not retained, variant included",7323,7323,0,1907,ATG,ATG,.,Single missense
7,chr1,1117486,A,G,TTLL10,+,donor gain,-1,9,intronic,"upstream effect, variant excluded, sequence ad...",2283,2573,290,673,ATG,ATG,3,Longer protein w/ PTCs
8,chr1,1153630,G,A,SDF4,-,donor gain,-2,6,intronic,"upstream effect, variant excluded, sequence ad...",2079,2285,206,348,ATG,ATG,2,Longer protein w/ PTCs
9,chr1,1166887,T,C,SDF4,-,donor gain,0,1,intronic,"upstream effect, variant excluded, sequence ad...",2079,2464,385,348,ATG,CCA,.,Longer UTR


In [302]:
len(novel_isoforms)

5950

In [303]:
Vernot_allele_origin_series = data_2['Vernot_allele_origin'].reset_index(drop = True)
Browning_allele_origin_series = data_2['Browning_allele_origin'].reset_index(drop = True)

In [304]:
novel_isoforms_with_allele_origin = novel_isoforms.copy()
novel_isoforms_with_allele_origin['Vernot_allele_origin'] = Vernot_allele_origin_series
novel_isoforms_with_allele_origin['Browning_allele_origin'] = Browning_allele_origin_series
novel_isoforms_with_allele_origin.head(5)

,chrom,pos,ref_allele,alt_allele,gene,strand,consequence,delta_pos,exon_number,exonic_intronic,sequence_effect,canonical_transcript_len,novel_transcript_len,canonical_len-novel_len,n_amino_acids,canonical_start_codon,novel_start_codon,n_premature_stops,result,Vernot_allele_origin,Browning_allele_origin
0,chr1,864726,T,A,SAMD11,+,donor loss,-2,2,intronic,"intron not retained, variant excluded",2551,2551,0,681,ATG,ATG,.,No effect,ancient,ancient
1,chr1,875708,C,G,SAMD11,+,donor gain,0,7,intronic,"upstream effect, variant excluded, sequence ad...",2551,3419,868,681,ATG,ATG,8,Longer protein w/ PTCs,archaic-specific,archaic-specific
2,chr1,909768,A,G,PLEKHN1,+,acceptor loss,26,16,intronic,"exon not skipped, variant excluded",2404,2404,0,611,ATG,ATG,.,No effect,ancient,ancient
3,chr1,956622,C,T,AGRN,+,donor gain,-2,1,intronic,"upstream effect, variant excluded, sequence ad...",7323,8190,867,1907,ATG,CCG,.,Longer UTR,low-confidence ancient,low-confidence ancient
4,chr1,975133,T,A,AGRN,+,donor gain,-3,3,intronic,"upstream effect, variant excluded, sequence ad...",7323,11749,4426,1907,ATG,ATG,51,Longer protein w/ PTCs,ancient,ancient


In [305]:
len(novel_isoforms_with_allele_origin)

5950

In [306]:
novel_isoforms_with_allele_origin.groupby('result').size().to_frame('N')

,N
result,
Longer UTR,479
Longer protein w/ PTCs,1897
Longer protein w/o PTCs,333
No effect,2391
Single missense,396
Translated product unknown,6
Truncated UTR,116
Truncated protein,332


In [307]:
novel_isoforms_with_allele_origin.groupby(['Vernot_allele_origin','result']).size().to_frame('N')

N
Vernot_allele_origin   result                          
ancient                Longer UTR                   151
                       Longer protein w/ PTCs       634
                       Longer protein w/o PTCs       95
                       No effect                   1092
                       Single missense              142
                       Translated product unknown     1
                       Truncated UTR                 44
                       Truncated protein             93
archaic-specific       Longer UTR                   206
                       Longer protein w/ PTCs       753
                       Longer protein w/o PTCs      153
                       No effect                    729
                       Single missense              137
                       Translated product unknown     5
                       Truncated UTR                 48
                       Truncated protein            155
introgressed           Longer UTR                    26
                       Longer protein w/ PTCs        91
                       Longer protein w/o PTCs       14
                       No effect                     69
                       Single missense               21
                       Truncated UTR                  5
                       Truncated protein             18
low-confidence ancient Longer UTR                    96
                       Longer protein w/ PTCs       419
                       Longer protein w/o PTCs       71
                       No effect                    501
                       Single missense               96
                       Truncated UTR                 19
                       Truncated protein             66

In [308]:
novel_isoforms_with_allele_origin.groupby(['Browning_allele_origin','result']).size().to_frame('N')

N
Browning_allele_origin result                          
ancient                Longer UTR                   149
                       Longer protein w/ PTCs       620
                       Longer protein w/o PTCs       90
                       No effect                   1061
                       Single missense              139
                       Translated product unknown     1
                       Truncated UTR                 42
                       Truncated protein             93
archaic-specific       Longer UTR                   206
                       Longer protein w/ PTCs       753
                       Longer protein w/o PTCs      153
                       No effect                    729
                       Single missense              137
                       Translated product unknown     5
                       Truncated UTR                 48
                       Truncated protein            155
introgressed           Longer UTR                    25
                       Longer protein w/ PTCs       133
                       Longer protein w/o PTCs       25
                       No effect                    148
                       Single missense               28
                       Truncated UTR                  9
                       Truncated protein             18
low-confidence ancient Longer UTR                    99
                       Longer protein w/ PTCs       391
                       Longer protein w/o PTCs       65
                       No effect                    453
                       Single missense               92
                       Truncated UTR                 17
                       Truncated protein             66

Let's test whether these distributions are statistically different to one another.

In [309]:
vernot_effect_counts = np.array([[151,634,95,1092,142,44,93],[206,753,153,729,137,48,155],[26,91,14,69,21,5,18]])

In [310]:
g, p, dof, expctd = chi2_contingency(vernot_effect_counts, lambda_ = 'log-likelihood')
g, p

(137.72250812383166, 1.725254484851994e-23)

In [311]:
browning_effect_counts = np.array([[149,620,90,1061,139,42,93],[206,753,153,729,137,48,155],[25,133,25,148,28,9,18]])

In [312]:
g, p, dof, expctd = chi2_contingency(browning_effect_counts, lambda_ = 'log-likelihood')
g, p

(121.2718284505004, 3.446077675710777e-20)

# Gene Characteristics <a class = 'anchor' id = 'genecharacteristics'></a>

## N Exons, CDS Length, and Gene Length <a class = 'anchor' id = 'physical'></a>

Do genes with more exons or longer genes have more SAVs? Let's load the annotation data first.

In [313]:
annotation_header = ['name','chrom','strand','tx_start','tx_end','exon_start','exon_end']
annotations = pd.read_csv('../annotations/grch37_exon_annotations.txt', sep='\t', skiprows=1, names=annotation_header)
annotations.head(10)

,name,chrom,strand,tx_start,tx_end,exon_start,exon_end
0,OR4F5,chr1,+,69090,70008,"69090,","70008,"
1,AL627309.1,chr1,-,134900,139379,"134900,137620,","135802,139379,"
2,OR4F29,chr1,+,367639,368634,"367639,","368634,"
3,OR4F16,chr1,-,621095,622034,"621095,","622034,"
4,AL669831.1,chr1,-,738531,739137,"738531,738787,739120,","738618,738812,739137,"
5,AL645608.2,chr1,+,818042,819983,"818042,819495,819960,","818058,819513,819983,"
6,SAMD11,chr1,+,861117,879955,"861117,861301,865534,866418,871151,874419,8746...","861180,861393,865716,866469,871276,874509,8748..."
7,AL645608.1,chr1,-,861263,866445,"861263,863254,865555,865665,865989,866425,","861406,863261,865660,865719,865996,866445,"
8,NOC2L,chr1,-,879583,894670,"879583,880436,880897,881552,881781,883510,8838...","880180,880526,881033,881666,881925,883612,8839..."
9,KLHL17,chr1,+,895966,901095,"895966,896672,897008,897205,897734,898083,8984...","896180,896932,897130,897427,897851,898297,8986..."


Get the length.

In [314]:
len(annotations)

20274

Now let's count the number of exons using the delimiter in either of the last columns.

In [315]:
annotations['n_exons'] = annotations['exon_start'].str.count(',')
annotations.head(10)

,name,chrom,strand,tx_start,tx_end,exon_start,exon_end,n_exons
0,OR4F5,chr1,+,69090,70008,"69090,","70008,",1
1,AL627309.1,chr1,-,134900,139379,"134900,137620,","135802,139379,",2
2,OR4F29,chr1,+,367639,368634,"367639,","368634,",1
3,OR4F16,chr1,-,621095,622034,"621095,","622034,",1
4,AL669831.1,chr1,-,738531,739137,"738531,738787,739120,","738618,738812,739137,",3
5,AL645608.2,chr1,+,818042,819983,"818042,819495,819960,","818058,819513,819983,",3
6,SAMD11,chr1,+,861117,879955,"861117,861301,865534,866418,871151,874419,8746...","861180,861393,865716,866469,871276,874509,8748...",14
7,AL645608.1,chr1,-,861263,866445,"861263,863254,865555,865665,865989,866425,","861406,863261,865660,865719,865996,866445,",6
8,NOC2L,chr1,-,879583,894670,"879583,880436,880897,881552,881781,883510,8838...","880180,880526,881033,881666,881925,883612,8839...",19
9,KLHL17,chr1,+,895966,901095,"895966,896672,897008,897205,897734,898083,8984...","896180,896932,897130,897427,897851,898297,8986...",12


Now the total length of the gene from the start of the first exon to the end of the last exon.

In [316]:
annotations['gene_length'] = annotations['tx_end'] - annotations['tx_start']

Now let's get the total exon length. Let's extract information as lists to make this easier and then bring everything back together. First, get a list of gene names copied by the number of exons present in that gene.

In [317]:
gene_names = annotations['name'].loc[annotations.index.repeat(annotations['n_exons'])].tolist()

Let's also get a list of chromosome names.

In [318]:
chroms = annotations['chrom'].loc[annotations.index.repeat(annotations['n_exons'])].tolist()

Then grab a list of exon starts and stops.

In [319]:
exon_starts = annotations['exon_start'].str.split(',').sum()
exon_stops = annotations['exon_end'].str.split(',').sum()

The trailing commas are creating blanks in the list so let's get rid of those.

In [320]:
exon_starts = list(filter(None, exon_starts))
exon_stops = list(filter(None, exon_stops))

Create a new dataframe and bring everything together.

In [321]:
exon_length = pd.DataFrame()
exon_length['name'] = gene_names
exon_length['exon_start'] = exon_starts
exon_length['exon_stop'] = exon_stops

Make sure it looks the way it should!

In [322]:
exon_length.head(10)

,name,exon_start,exon_stop
0,OR4F5,69090,70008
1,AL627309.1,134900,135802
2,AL627309.1,137620,139379
3,OR4F29,367639,368634
4,OR4F16,621095,622034
5,AL669831.1,738531,738618
6,AL669831.1,738787,738812
7,AL669831.1,739120,739137
8,AL645608.2,818042,818058
9,AL645608.2,819495,819513


Quick sidebar! This information would be very useful as annotation data. Let's also include the strand.

In [323]:
strands = annotations['strand'].loc[annotations.index.repeat(annotations['n_exons'])].tolist()

In [324]:
hg19_exons = pd.DataFrame()
hg19_exons['chrom'] = chroms
hg19_exons['exon_start'] = exon_length['exon_start'].astype(int)
hg19_exons['exon_stop'] = exon_stops
hg19_exons['strand'] = strands
hg19_exons['gene'] = gene_names

In [325]:
hg19_exons.head(20)

,chrom,exon_start,exon_stop,strand,gene
0,chr1,69090,70008,+,OR4F5
1,chr1,134900,135802,-,AL627309.1
2,chr1,137620,139379,-,AL627309.1
3,chr1,367639,368634,+,OR4F29
4,chr1,621095,622034,-,OR4F16
5,chr1,738531,738618,-,AL669831.1
6,chr1,738787,738812,-,AL669831.1
7,chr1,739120,739137,-,AL669831.1
8,chr1,818042,818058,+,AL645608.2
9,chr1,819495,819513,+,AL645608.2


In [326]:
hg19_exons.to_csv('../annotations/hg19_exons.bed', sep="\t", header = False, index = False)

Now change the data type for the starts and stops and create a length column.

In [327]:
exon_length['exon_stop'] = exon_length['exon_stop'].astype(int)
exon_length['exon_start'] = exon_length['exon_start'].astype(int)
exon_length['exon_length'] = exon_length['exon_stop'] - exon_length['exon_start']

Check to see that it worked.

In [328]:
exon_length.head(10)

,name,exon_start,exon_stop,exon_length
0,OR4F5,69090,70008,918
1,AL627309.1,134900,135802,902
2,AL627309.1,137620,139379,1759
3,OR4F29,367639,368634,995
4,OR4F16,621095,622034,939
5,AL669831.1,738531,738618,87
6,AL669831.1,738787,738812,25
7,AL669831.1,739120,739137,17
8,AL645608.2,818042,818058,16
9,AL645608.2,819495,819513,18


Now let's group by the gene to get the total exon length.

In [329]:
exon_length = exon_length.groupby('name')['exon_length'].sum().to_frame('exon_length')

Create a new dataframe.

In [330]:
n_exons_length = annotations[['name','n_exons','gene_length']]
n_exons_length.head(10)

,name,n_exons,gene_length
0,OR4F5,1,918
1,AL627309.1,2,4479
2,OR4F29,1,995
3,OR4F16,1,939
4,AL669831.1,3,606
5,AL645608.2,3,1941
6,SAMD11,14,18838
7,AL645608.1,6,5182
8,NOC2L,19,15087
9,KLHL17,12,5129


In [331]:
n_exons_length = pd.merge(n_exons_length, exon_length['exon_length'], on='name')

Let's take a peek at the distributions of these variables.

In [332]:
n_exons_hist = n_exons_length.groupby('n_exons').size().to_frame('count')
n_exons_hist

,count
n_exons,
1,1597
2,1525
3,1482
4,1604
5,1517
6,1389
7,1152
8,1088
9,1024


In [333]:
gene_length_hist = n_exons_length.groupby('gene_length').size().to_frame('count')
gene_length_hist

,count
gene_length,
9,2
11,1
13,1
16,3
17,6
...,...
1987243,1
2056874,1
2057828,1


In [334]:
exon_length_hist = n_exons_length.groupby('exon_length').size().to_frame('count')
exon_length_hist

,count
exon_length,
9,2
11,1
13,1
16,3
17,6
...,...
30355,1
33679,1
34526,1


Now to actually test their relationship to variant count at our two thresholds.

In [335]:
n_variants = data.rename(columns={'annotation':'name'})
n_variants = n_variants.groupby('name').size().to_frame('n_variants')
variant_characterstics = pd.merge(n_exons_length, n_variants, on='name', how='left')

In [336]:
n_splice_variants_2 = data_2.rename(columns={'annotation':'name'})
n_splice_variants_2 = n_splice_variants_2.groupby('name').size().to_frame('variant_count_2')
variant_characterstics = pd.merge(variant_characterstics, n_splice_variants_2, on='name', how='left')

In [337]:
n_splice_variants_5 = data_5.rename(columns={'annotation':'name'})
n_splice_variants_5 = n_splice_variants_5.groupby('name').size().to_frame('variant_count_5')
variant_characterstics = pd.merge(variant_characterstics, n_splice_variants_5, on='name', how='left')

In [338]:
variant_characterstics = variant_characterstics.fillna(0)
variant_characterstics.head(20)

,name,n_exons,gene_length,exon_length,n_variants,variant_count_2,variant_count_5
0,OR4F5,1,918,918,0.0,0.0,0.0
1,AL627309.1,2,4479,2661,0.0,0.0,0.0
2,OR4F29,1,995,995,0.0,0.0,0.0
3,OR4F16,1,939,939,0.0,0.0,0.0
4,AL669831.1,3,606,129,0.0,0.0,0.0
5,AL645608.2,3,1941,57,0.0,0.0,0.0
6,SAMD11,14,18838,2551,53.0,2.0,0.0
7,AL645608.1,6,5182,336,17.0,0.0,0.0
8,NOC2L,19,15087,2790,37.0,0.0,0.0
9,KLHL17,12,5129,2560,7.0,0.0,0.0


In [339]:
len(variant_characterstics)

20274

Save this dataframe for plotting.

In [340]:
variant_characterstics.to_csv('../dataframes/variant_characteristics.txt', sep="\t", header = True, index = False)

Subset to genes with at least one variant.

In [341]:
variant_characterstics = variant_characterstics[variant_characterstics['n_variants'] > 0]

In [342]:
len(variant_characterstics)

17631

Let's run some correlations on the characteristics.

In [343]:
rho, p = spearmanr(variant_characterstics['n_exons'], variant_characterstics['variant_count_2'])
print(rho,p)

0.3160848388955107 0.0


In [344]:
rho, p = spearmanr(variant_characterstics['exon_length'], variant_characterstics['variant_count_2'])
print(rho,p)

0.1851170800689867 1.0667367197057646e-135


In [345]:
rho, p = spearmanr(variant_characterstics['gene_length'], variant_characterstics['variant_count_2'])
print(rho,p)

0.2919566369123417 0.0


In [346]:
rho, p = spearmanr(variant_characterstics['n_exons'], variant_characterstics['variant_count_5'])
print(rho,p)

0.13299027146082146 2.2009095588908162e-70


In [347]:
rho, p = spearmanr(variant_characterstics['exon_length'], variant_characterstics['variant_count_5'])
print(rho,p)

0.0624799065089255 1.009030502488749e-16


In [348]:
rho, p = spearmanr(variant_characterstics['gene_length'], variant_characterstics['variant_count_5'])
print(rho,p)

0.1017201812950686 8.985235346295254e-42


## N Isoforms <a class = 'anchor' id = 'isoforms'></a>

Does the number of the known isoforms associate with N SAVs?

In [349]:
isoforms_header = ['name','isoform_count']
isoforms = pd.read_csv('../annotations/GENCODE_Release_40_hg38_N_isoforms.txt', sep = '\t', skiprows = 1, names = isoforms_header)
isoforms.head(10)

,name,isoform_count
0,MIR1302-2HG,2
1,FAM138A,2
2,OR4F5,1
3,ENSG00000238009,5
4,ENSG00000239945,1
5,ENSG00000239906,1
6,ENSG00000241860,6
7,ENSG00000241599,1
8,ENSG00000286448,1
9,ENSG00000236601,3


Merge with above dataframe. Let's use a new dataframe.

In [350]:
n_isoforms = pd.merge(variant_characterstics, isoforms, on = ['name'])
n_isoforms.head(10)

,name,n_exons,gene_length,exon_length,n_variants,variant_count_2,variant_count_5,isoform_count
0,SAMD11,14,18838,2551,53.0,2.0,0.0,15
1,NOC2L,19,15087,2790,37.0,0.0,0.0,6
2,KLHL17,12,5129,2560,7.0,0.0,0.0,4
3,PLEKHN1,16,8612,2404,21.0,1.0,0.0,5
4,PERM1,3,5825,3340,10.0,0.0,0.0,4
5,HES4,4,1148,899,2.0,0.0,0.0,4
6,ISG15,2,1118,711,7.0,0.0,0.0,3
7,AGRN,36,35994,7323,85.0,4.0,0.0,10
8,RNF223,2,3342,1902,3.0,0.0,0.0,1
9,C1orf159,10,34276,1841,52.0,0.0,0.0,19


In [351]:
len(n_isoforms)

16151

In [352]:
rho, p = spearmanr(n_isoforms['isoform_count'], n_isoforms['variant_count_2'])
print(rho,p)

0.1767337559003496 1.8670449850386394e-113


In [353]:
rho, p = spearmanr(n_isoforms['isoform_count'], n_isoforms['variant_count_5'])
print(rho,p)

0.07692978918607588 1.2368500958019458e-22


In [354]:
rho, p = spearmanr(n_isoforms['n_exons'], n_isoforms['isoform_count'])
print(rho,p)

0.5639685614316065 0.0


In [355]:
rho, p = spearmanr(n_isoforms['gene_length'], n_isoforms['variant_count_5'])
print(rho,p)

0.09741069465365773 2.3604050238515602e-35


Run a partial correlation to control for the number of exons, which seems to be the strongest covariate.

In [356]:
partial_corr(data = n_isoforms, x = 'isoform_count', y = 'variant_count_2', covar=['n_exons'], method = 'spearman')

,n,r,CI95%,p-val
spearman,16151,-0.003008,"[-0.02, 0.01]",0.702242


In [357]:
partial_corr(data = n_isoforms, x = 'isoform_count', y = 'variant_count_5', covar=['n_exons'], method = 'spearman')

,n,r,CI95%,p-val
spearman,16151,0.000817,"[-0.01, 0.02]",0.917351


## Constraint and Conservation <a class = 'anchor' id = 'constraintconservation'></a>

Let's check for differences in constraint and conservation starting with missense observed over expected. Start with Vernot.

In [358]:
Vernot_ancient_mis_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'ancient', ['annotation', 'mis_oe']].drop_duplicates().dropna()
Vernot_archaic_specific_mis_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'archaic-specific', ['annotation', 'mis_oe']].drop_duplicates().dropna()
Vernot_introgressed_mis_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'introgressed', ['annotation', 'mis_oe']].drop_duplicates().dropna()
non_SA_mis_oe = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'mis_oe']].drop_duplicates().dropna()

In [359]:
len(Vernot_ancient_mis_oe)

1817

In [360]:
len(Vernot_archaic_specific_mis_oe)

1832

In [361]:
len(Vernot_introgressed_mis_oe)

234

In [362]:
len(non_SA_mis_oe)

12708

In [363]:
kruskal(Vernot_ancient_mis_oe['mis_oe'], Vernot_archaic_specific_mis_oe['mis_oe'], Vernot_introgressed_mis_oe['mis_oe'], non_SA_mis_oe['mis_oe'])

KruskalResult(statistic=18.078829007426336, pvalue=0.0004236866085398531)

Now for missense z-score.

In [364]:
Vernot_ancient_mis_z = data_2.loc[data_2['Vernot_allele_origin'] == 'ancient', ['annotation', 'mis_z']].drop_duplicates().dropna()
Vernot_archaic_specific_mis_z = data_2.loc[data_2['Vernot_allele_origin'] == 'archaic-specific', ['annotation', 'mis_z']].drop_duplicates().dropna()
Vernot_introgressed_mis_z = data_2.loc[data_2['Vernot_allele_origin'] == 'introgressed', ['annotation', 'mis_z']].drop_duplicates().dropna()
non_SA_mis_z = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'mis_z']].drop_duplicates().dropna()

In [365]:
len(Vernot_ancient_mis_z)

1817

In [366]:
len(Vernot_archaic_specific_mis_z)

1832

In [367]:
len(Vernot_introgressed_mis_z)

234

In [368]:
len(non_SA_mis_z)

12708

In [369]:
kruskal(Vernot_ancient_mis_z['mis_z'], Vernot_archaic_specific_mis_z['mis_z'], Vernot_introgressed_mis_z['mis_z'], non_SA_mis_z['mis_z'])

KruskalResult(statistic=8.620709037824106, pvalue=0.0347828750312021)

Now for LoF observed/expected.

In [370]:
Vernot_ancient_lof_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'ancient', ['annotation', 'lof_oe']].drop_duplicates().dropna()
Vernot_archaic_specific_lof_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'archaic-specific', ['annotation', 'lof_oe']].drop_duplicates().dropna()
Vernot_introgressed_lof_oe = data_2.loc[data_2['Vernot_allele_origin'] == 'introgressed', ['annotation', 'lof_oe']].drop_duplicates().dropna()
non_SA_lof_oe = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'lof_oe']].drop_duplicates().dropna()

In [371]:
len(Vernot_ancient_lof_oe)

1811

In [372]:
len(Vernot_archaic_specific_lof_oe)

1824

In [373]:
len(Vernot_introgressed_lof_oe)

234

In [374]:
len(non_SA_lof_oe)

12484

In [375]:
kruskal(Vernot_ancient_lof_oe['lof_oe'], Vernot_archaic_specific_lof_oe['lof_oe'], Vernot_introgressed_lof_oe['lof_oe'], non_SA_lof_oe['lof_oe'])

KruskalResult(statistic=1.5332226862446645, pvalue=0.6746240497778113)

Now for LoF z-scores.

In [376]:
Vernot_ancient_lof_z = data_2.loc[data_2['Vernot_allele_origin'] == 'ancient', ['annotation', 'lof_z']].drop_duplicates().dropna()
Vernot_archaic_specific_lof_z = data_2.loc[data_2['Vernot_allele_origin'] == 'archaic-specific', ['annotation', 'lof_z']].drop_duplicates().dropna()
Vernot_introgressed_lof_z = data_2.loc[data_2['Vernot_allele_origin'] == 'introgressed', ['annotation', 'lof_z']].drop_duplicates().dropna()
non_SA_lof_z = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'lof_z']].drop_duplicates().dropna()

In [377]:
len(Vernot_ancient_lof_z)

1811

In [378]:
len(Vernot_archaic_specific_lof_z)

1824

In [379]:
len(Vernot_introgressed_lof_z)

234

In [380]:
len(non_SA_lof_z)

12484

In [381]:
kruskal(Vernot_ancient_lof_z['lof_z'], Vernot_archaic_specific_lof_z['lof_z'], Vernot_introgressed_lof_z['lof_z'], non_SA_lof_z['lof_z'])

KruskalResult(statistic=323.3105002953148, pvalue=8.955791061419506e-70)

And phyloP.

In [382]:
Vernot_ancient_phyloP = data_2.loc[data_2['Vernot_allele_origin'] == 'ancient', 'phyloP'].dropna()
Vernot_archaic_specific_phyloP = data_2.loc[data_2['Vernot_allele_origin'] == 'archaic-specific', 'phyloP'].dropna()
Vernot_introgressed_phyloP = data_2.loc[data_2['Vernot_allele_origin'] == 'introgressed', 'phyloP'].dropna()
non_SA_phyloP = data.loc[data['delta_max'] < 0.2, 'phyloP'].dropna()

In [383]:
len(Vernot_ancient_phyloP)

2251

In [384]:
len(Vernot_archaic_specific_phyloP)

2186

In [385]:
len(Vernot_introgressed_phyloP)

244

In [386]:
len(non_SA_phyloP)

1600192

In [387]:
kruskal(Vernot_ancient_phyloP, Vernot_archaic_specific_phyloP, Vernot_introgressed_phyloP, non_SA_phyloP)

KruskalResult(statistic=877.4289463274906, pvalue=6.962502007197678e-190)

How many of these are not "neutrally evolving"?

In [388]:
((Vernot_ancient_phyloP < -1.3).sum() + (Vernot_ancient_phyloP > 1.3).sum() + (Vernot_archaic_specific_phyloP < -1.3).sum() + (Vernot_archaic_specific_phyloP > 1.3).sum() + (Vernot_introgressed_phyloP < -1.3).sum() + (Vernot_introgressed_phyloP > 1.3).sum() + (non_SA_phyloP < -1.3).sum() + (non_SA_phyloP > 1.3).sum()) / (len(Vernot_ancient_phyloP) + len(Vernot_archaic_specific_phyloP) + len(Vernot_introgressed_phyloP) + len(non_SA_phyloP))

0.15650646499754187

And "neutrally evolving"?

In [389]:
1-0.15650646499754187

0.8434935350024582

Repeat for Browning.

In [390]:
Browning_ancient_mis_oe = data_2.loc[data_2['Browning_allele_origin'] == 'ancient', ['annotation', 'mis_oe']].drop_duplicates().dropna()
Browning_archaic_specific_mis_oe = data_2.loc[data_2['Browning_allele_origin'] == 'archaic-specific', ['annotation', 'mis_oe']].drop_duplicates().dropna()
Browning_introgressed_mis_oe = data_2.loc[data_2['Browning_allele_origin'] == 'introgressed', ['annotation', 'mis_oe']].drop_duplicates().dropna()
non_SA_mis_oe = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'mis_oe']].drop_duplicates().dropna()

In [391]:
len(Browning_ancient_mis_oe)

1780

In [392]:
len(Browning_archaic_specific_mis_oe)

1832

In [393]:
len(Browning_introgressed_mis_oe)

349

In [394]:
len(non_SA_mis_oe)

12708

In [395]:
kruskal(Browning_ancient_mis_oe['mis_oe'], Browning_archaic_specific_mis_oe['mis_oe'], Browning_introgressed_mis_oe['mis_oe'], non_SA_mis_oe['mis_oe'])

KruskalResult(statistic=20.889083756698465, pvalue=0.00011100996646209538)

Now for missense z-score.

In [396]:
Browning_ancient_mis_z = data_2.loc[data_2['Browning_allele_origin'] == 'ancient', ['annotation', 'mis_z']].drop_duplicates().dropna()
Browning_archaic_specific_mis_z = data_2.loc[data_2['Browning_allele_origin'] == 'archaic-specific', ['annotation', 'mis_z']].drop_duplicates().dropna()
Browning_introgressed_mis_z = data_2.loc[data_2['Browning_allele_origin'] == 'introgressed', ['annotation', 'mis_z']].drop_duplicates().dropna()
non_SA_mis_z = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'mis_z']].drop_duplicates().dropna()

In [397]:
len(Browning_ancient_mis_z)

1780

In [398]:
len(Browning_archaic_specific_mis_z)

1832

In [399]:
len(Browning_introgressed_mis_z)

349

In [400]:
len(non_SA_mis_z)

12708

In [401]:
kruskal(Browning_ancient_mis_z['mis_z'], Browning_archaic_specific_mis_z['mis_z'], Browning_introgressed_mis_z['mis_z'], non_SA_mis_z['mis_z'])

KruskalResult(statistic=7.474011387744941, pvalue=0.05822998409610527)

Now for LoF observed/expected.

In [402]:
Browning_ancient_lof_oe = data_2.loc[data_2['Browning_allele_origin'] == 'ancient', ['annotation', 'lof_oe']].drop_duplicates().dropna()
Browning_archaic_specific_lof_oe = data_2.loc[data_2['Browning_allele_origin'] == 'archaic-specific', ['annotation', 'lof_oe']].drop_duplicates().dropna()
Browning_introgressed_lof_oe = data_2.loc[data_2['Browning_allele_origin'] == 'introgressed', ['annotation', 'lof_oe']].drop_duplicates().dropna()
non_SA_lof_oe = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'lof_oe']].drop_duplicates().dropna()

In [403]:
len(Browning_ancient_lof_oe)

1774

In [404]:
len(Browning_archaic_specific_lof_oe)

1824

In [405]:
len(Browning_introgressed_lof_oe)

349

In [406]:
len(non_SA_lof_oe)

12484

In [407]:
kruskal(Browning_ancient_lof_oe['lof_oe'], Browning_archaic_specific_lof_oe['lof_oe'], Browning_introgressed_lof_oe['lof_oe'], non_SA_lof_oe['lof_oe'])

KruskalResult(statistic=1.4415470445962708, pvalue=0.6958254176524867)

Now for LoF z-scores.

In [408]:
Browning_ancient_lof_z = data_2.loc[data_2['Browning_allele_origin'] == 'ancient', ['annotation', 'lof_z']].drop_duplicates().dropna()
Browning_archaic_specific_lof_z = data_2.loc[data_2['Browning_allele_origin'] == 'archaic-specific', ['annotation', 'lof_z']].drop_duplicates().dropna()
Browning_introgressed_lof_z = data_2.loc[data_2['Browning_allele_origin'] == 'introgressed', ['annotation', 'lof_z']].drop_duplicates().dropna()
non_SA_lof_z = data.loc[~data['annotation'].isin(data_2['annotation']), ['annotation', 'lof_z']].drop_duplicates().dropna()

In [409]:
len(Browning_ancient_lof_z)

1774

In [410]:
len(Browning_archaic_specific_lof_z)

1824

In [411]:
len(Browning_introgressed_lof_z)

349

In [412]:
len(non_SA_lof_z)

12484

In [413]:
kruskal(Browning_ancient_lof_z['lof_z'], Browning_archaic_specific_lof_z['lof_z'], Browning_introgressed_lof_z['lof_z'], non_SA_lof_z['lof_z'])

KruskalResult(statistic=315.77589291885266, pvalue=3.8294331448242675e-68)

And phyloP.

In [414]:
Browning_ancient_phyloP = data_2.loc[data_2['Browning_allele_origin'] == 'ancient', 'phyloP'].dropna()
Browning_archaic_specific_phyloP = data_2.loc[data_2['Browning_allele_origin'] == 'archaic-specific', 'phyloP'].dropna()
Browning_introgressed_phyloP = data_2.loc[data_2['Browning_allele_origin'] == 'introgressed', 'phyloP'].dropna()
non_SA_phyloP = data.loc[data['delta_max'] < 0.2, 'phyloP'].dropna()

In [415]:
len(Browning_ancient_phyloP)

2194

In [416]:
len(Browning_archaic_specific_phyloP)

2186

In [417]:
len(Browning_introgressed_phyloP)

386

In [418]:
len(non_SA_phyloP)

1600192

In [419]:
kruskal(Browning_ancient_phyloP, Browning_archaic_specific_phyloP, Browning_introgressed_phyloP, non_SA_phyloP)

KruskalResult(statistic=808.2443759754741, pvalue=7.050363136869668e-175)

How many of these are not "neutrally evolving"?

In [420]:
((Browning_ancient_phyloP < -1.3).sum() + (Browning_ancient_phyloP > 1.3).sum() + (Browning_archaic_specific_phyloP < -1.3).sum() + (Browning_archaic_specific_phyloP > 1.3).sum() + (Browning_introgressed_phyloP < -1.3).sum() + (Browning_introgressed_phyloP > 1.3).sum() + (non_SA_phyloP < -1.3).sum() + (non_SA_phyloP > 1.3).sum()) / (len(Browning_ancient_phyloP) + len(Browning_archaic_specific_phyloP) + len(Browning_introgressed_phyloP) + len(non_SA_phyloP))

0.15650876845375394

And "neutrally evolving"?

In [421]:
1-0.15650876845375394

0.8434912315462461

# Purifying Selection <a class = 'anchor' id = 'purifyingselection'></a>

Now let's consider the role of purifying selection on SAVs. Start by writing a function that will use data from a dataframe to add columns with enrichment test information: the OR, p-value, and 95% CI.

In [422]:
def purifying_enrichment(df,A,B,C,D):
    odds_ratio = []
    p_value = []
    lower_CI = []
    upper_CI = []

    for i in range(len(df)):
        table = np.array([[df.loc[i, A],
                           df.loc[i, B]],
                          [df.loc[i, C],
                           df.loc[i, D]]])
        OR, p = fisher_exact(table)
    
        lCI = np.exp((np.log(OR)) - (1.96 * (sqrt((1/table[0,0]) + (1/table[0,1]) + (1/table[1,0]) + (1/table[1,1])))))
        uCI = np.exp((np.log(OR)) + (1.96 * (sqrt((1/table[0,0]) + (1/table[0,1]) + (1/table[1,0]) + (1/table[1,1])))))
    
        odds_ratio.append(OR)
        p_value.append(p)
        lower_CI.append(lCI)
        upper_CI.append(uCI)
    
    df['odds_ratio'] = odds_ratio
    df['p_value'] = p_value
    df['lower_CI'] = lower_CI
    df['upper_CI'] = upper_CI

## Varied Deltas <a class = 'anchor' id = 'varieddeltas'></a>

Now assemble the data starting with the sum of all unique counts.

In [423]:
all_unique_total = data[(data['distribution']=='Altai') | (data['distribution']=='Chagyrskaya') | (data['distribution']=='Denisovan') | (data['distribution']=='Vindija')]
all_unique_total_SA_2 = all_unique_total[(all_unique_total['delta_max']>=0.2)]
all_unique_total_SA_2_index = all_unique_total_SA_2.index
all_unique_total_non_SA_2 = all_unique_total.drop(all_unique_total_SA_2_index)

In [424]:
all_unique_total = data[(data['distribution']=='Altai') | (data['distribution']=='Chagyrskaya') | (data['distribution']=='Denisovan') | (data['distribution']=='Vindija')]
all_unique_total_SA_3 = all_unique_total[(all_unique_total['delta_max']>=0.3)]
all_unique_total_SA_3_index = all_unique_total_SA_3.index
all_unique_total_non_SA_3 = all_unique_total.drop(all_unique_total_SA_3_index)

In [425]:
all_unique_total = data[(data['distribution']=='Altai') | (data['distribution']=='Chagyrskaya') | (data['distribution']=='Denisovan') | (data['distribution']=='Vindija')]
all_unique_total_SA_4 = all_unique_total[(all_unique_total['delta_max']>=0.4)]
all_unique_total_SA_4_index = all_unique_total_SA_4.index
all_unique_total_non_SA_4 = all_unique_total.drop(all_unique_total_SA_4_index)

In [426]:
all_unique_total = data[(data['distribution']=='Altai') | (data['distribution']=='Chagyrskaya') | (data['distribution']=='Denisovan') | (data['distribution']=='Vindija')]
all_unique_total_SA_5 = all_unique_total[(all_unique_total['delta_max']>=0.5)]
all_unique_total_SA_5_index = all_unique_total_SA_5.index
all_unique_total_non_SA_5 = all_unique_total.drop(all_unique_total_SA_5_index)

Now for the shared SAVs. We'll use some of these again below.

In [427]:
all_shared = data[(data['distribution']=='Shared')]
all_shared_SAVs_2 = all_shared[(all_shared['delta_max']>=0.2)]
all_shared_SAVs_2_index = all_shared_SAVs_2.index
all_shared_non_SAVs_2 = all_shared.drop(all_shared_SAVs_2_index)

In [428]:
all_shared = data[(data['distribution']=='Shared')]
all_shared_SAVs_3 = all_shared[(all_shared['delta_max']>=0.3)]
all_shared_SAVs_3_index = all_shared_SAVs_3.index
all_shared_non_SAVs_3 = all_shared.drop(all_shared_SAVs_3_index)

In [429]:
all_shared = data[(data['distribution']=='Shared')]
all_shared_SAVs_4 = all_shared[(all_shared['delta_max']>=0.4)]
all_shared_SAVs_4_index = all_shared_SAVs_4.index
all_shared_non_SAVs_4 = all_shared.drop(all_shared_SAVs_4_index)

In [430]:
all_shared = data[(data['distribution']=='Shared')]
all_shared_SAVs_5 = all_shared[(all_shared['delta_max']>=0.5)]
all_shared_SAVs_5_index = all_shared_SAVs_5.index
all_shared_non_SAVs_5 = all_shared.drop(all_shared_SAVs_5_index)

Build a dataframe.

In [431]:
delta_threshold = ['0.2','0.3','0.4','0.5']
total_unique_SAVs = [len(all_unique_total_SA_2), len(all_unique_total_SA_3), len(all_unique_total_SA_4), len(all_unique_total_SA_5)]
total_unique_non_SAVs = [len(all_unique_total_non_SA_2), len(all_unique_total_non_SA_3), len(all_unique_total_non_SA_4), len(all_unique_total_non_SA_5)]
shared_SAVs = [len(all_shared_SAVs_2), len(all_shared_SAVs_3), len(all_shared_SAVs_4), len(all_shared_SAVs_5)]
shared_non_SAVs = [len(all_shared_non_SAVs_2), len(all_shared_non_SAVs_3), len(all_shared_non_SAVs_4), len(all_shared_non_SAVs_5)]

total_unique_enrichment = pd.DataFrame(list(zip(delta_threshold, total_unique_SAVs, total_unique_non_SAVs, shared_SAVs, shared_non_SAVs)),
                                          columns =['delta_threshold', 'N_total_unique_SAVs', 'N_total_unique_non_SAVs', 'N_shared_SAVs', 'N_shared_non_SAVs'])
total_unique_enrichment

,delta_threshold,N_total_unique_SAVs,N_total_unique_non_SAVs,N_shared_SAVs,N_shared_non_SAVs
0,0.2,2416,615666,1933,571264
1,0.3,1175,616907,945,572252
2,0.4,685,617397,533,572664
3,0.5,437,617645,328,572869


Run the enrichment tests.

In [432]:
purifying_enrichment(total_unique_enrichment,'N_total_unique_SAVs','N_shared_SAVs','N_total_unique_non_SAVs','N_shared_non_SAVs')
total_unique_enrichment

,delta_threshold,N_total_unique_SAVs,N_total_unique_non_SAVs,N_shared_SAVs,N_shared_non_SAVs,odds_ratio,p_value,lower_CI,upper_CI
0,0.2,2416,615666,1933,571264,1.159730,0.000001,1.092280,1.231345
1,0.3,1175,616907,945,572252,1.153383,0.001099,1.058636,1.256610
2,0.4,685,617397,533,572664,1.192062,0.002351,1.064410,1.335022
3,0.5,437,617645,328,572869,1.235731,0.003772,1.070824,1.426033


Save this dataframe to make plotting easier.

In [433]:
total_unique_enrichment.to_csv('total_unique_enrichment_tests.txt', sep = '\t', header = True, index = False)

## Lineage-Specific <a class = 'anchor' id = 'lineagespecific'></a>

What about lineage-specific effects? 

In [434]:
altai = data[(data['distribution'] == 'Altai')]
altai_SAVs_2 = altai[(altai['delta_max'] >= 0.2)]
altai_SAVs_2_index = altai_SAVs_2.index
altai_non_SAVs_2 = altai.drop(altai_SAVs_2_index)

In [435]:
altai = data[(data['distribution'] == 'Altai')]
altai_SAVs_5 = altai[(altai['delta_max'] >= 0.5)]
altai_SAVs_5_index = altai_SAVs_5.index
altai_non_SAVs_5 = altai.drop(altai_SAVs_5_index)

In [436]:
chagyrskaya = data[(data['distribution'] == 'Chagyrskaya')]
chagyrskaya_SAVs_2 = chagyrskaya[(chagyrskaya['delta_max'] >= 0.2)]
chagyrskaya_SAVs_2_index = chagyrskaya_SAVs_2.index
chagyrskaya_non_SAVs_2 = chagyrskaya.drop(chagyrskaya_SAVs_2_index)

In [437]:
chagyrskaya = data[(data['distribution'] == 'Chagyrskaya')]
chagyrskaya_SAVs_5 = chagyrskaya[(chagyrskaya['delta_max'] >= 0.5)]
chagyrskaya_SAVs_5_index = chagyrskaya_SAVs_5.index
chagyrskaya_non_SAVs_5 = chagyrskaya.drop(chagyrskaya_SAVs_5_index)

In [438]:
denisovan = data[(data['distribution'] == 'Denisovan')]
denisovan_SAVs_2 = denisovan[(denisovan['delta_max'] >= 0.2)]
denisovan_SAVs_2_index = denisovan_SAVs_2.index
denisovan_non_SAVs_2 = denisovan.drop(denisovan_SAVs_2_index)

In [439]:
denisovan = data[(data['distribution'] == 'Denisovan')]
denisovan_SAVs_5 = denisovan[(denisovan['delta_max'] >= 0.5)]
denisovan_SAVs_5_index = denisovan_SAVs_5.index
denisovan_non_SAVs_5 = denisovan.drop(denisovan_SAVs_5_index)

In [440]:
vindija = data[(data['distribution'] == 'Vindija')]
vindija_SAVs_2 = vindija[(vindija['delta_max'] >= 0.2)]
vindija_SAVs_2_index = vindija_SAVs_2.index
vindija_non_SAVs_2 = vindija.drop(vindija_SAVs_2_index)

In [441]:
vindija = data[(data['distribution'] == 'Vindija')]
vindija_SAVs_5 = vindija[(vindija['delta_max'] >= 0.5)]
vindija_SAVs_5_index = vindija_SAVs_5.index
vindija_non_SAVs_5 = vindija.drop(vindija_SAVs_5_index)

Build the dataframe.

In [442]:
delta_threshold = ['0.2','0.2','0.2','0.2','0.5','0.5','0.5','0.5']
lineage = ['altai','chagyrskaya','denisovan','vindija','altai','chagyrskaya','denisovan','vindija']
unique_SAVs = [len(altai_SAVs_2), len(chagyrskaya_SAVs_2), len(denisovan_SAVs_2), len(vindija_SAVs_2), len(altai_SAVs_5), len(chagyrskaya_SAVs_5), len(denisovan_SAVs_5), len(vindija_SAVs_5)]
unique_non_SAVs = [len(altai_non_SAVs_2), len(chagyrskaya_non_SAVs_2), len(denisovan_non_SAVs_2), len(vindija_non_SAVs_2), len(altai_non_SAVs_5), len(chagyrskaya_non_SAVs_5), len(denisovan_non_SAVs_5), len(vindija_non_SAVs_5)]
shared_SAVs = [len(all_shared_SAVs_2), len(all_shared_SAVs_2), len(all_shared_SAVs_2), len(all_shared_SAVs_2), len(all_shared_SAVs_5), len(all_shared_SAVs_5), len(all_shared_SAVs_5), len(all_shared_SAVs_5)]
shared_non_SAVs = [len(all_shared_non_SAVs_2), len(all_shared_non_SAVs_2), len(all_shared_non_SAVs_2), len(all_shared_non_SAVs_2), len(all_shared_non_SAVs_5), len(all_shared_non_SAVs_5), len(all_shared_non_SAVs_5), len(all_shared_non_SAVs_5)]

lineage_specific_enrichment = pd.DataFrame(list(zip(delta_threshold, lineage, unique_SAVs, unique_non_SAVs, shared_SAVs, shared_non_SAVs)),
                                          columns =['delta_threshold', 'lineage', 'N_unique_SAVs', 'N_unique_non_SAVs', 'N_shared_SAVs', 'N_shared_non_SAVs'])
lineage_specific_enrichment

,delta_threshold,lineage,N_unique_SAVs,N_unique_non_SAVs,N_shared_SAVs,N_shared_non_SAVs
0,0.2,altai,399,81517,1933,571264
1,0.2,chagyrskaya,218,53457,1933,571264
2,0.2,denisovan,1492,410000,1933,571264
3,0.2,vindija,307,70692,1933,571264
4,0.5,altai,75,81841,328,572869
5,0.5,chagyrskaya,35,53640,328,572869
6,0.5,denisovan,254,411238,328,572869
7,0.5,vindija,73,70926,328,572869


In [443]:
purifying_enrichment(lineage_specific_enrichment,'N_unique_SAVs','N_shared_SAVs','N_unique_non_SAVs','N_shared_non_SAVs')
lineage_specific_enrichment

,delta_threshold,lineage,N_unique_SAVs,N_unique_non_SAVs,N_shared_SAVs,N_shared_non_SAVs,odds_ratio,p_value,lower_CI,upper_CI
0,0.2,altai,399,81517,1933,571264,1.446538,1.152673e-10,1.298420,1.611551
1,0.2,chagyrskaya,218,53457,1933,571264,1.205194,1.078870e-02,1.047416,1.386738
2,0.2,denisovan,1492,410000,1933,571264,1.075449,3.575391e-02,1.005089,1.150736
3,0.2,vindija,307,70692,1933,571264,1.283433,8.655485e-05,1.137538,1.448040
4,0.5,altai,75,81841,328,572869,1.600559,4.931121e-04,1.245305,2.057159
5,0.5,chagyrskaya,35,53640,328,572869,1.139622,4.524517e-01,0.804170,1.615004
6,0.5,denisovan,254,411238,328,572869,1.078753,3.774332e-01,0.915704,1.270834
7,0.5,vindija,73,70926,328,572869,1.797624,2.009186e-05,1.394727,2.316906


In [444]:
lineage_specific_enrichment.to_csv('lineage_specific_enrichment_tests.txt', sep = '\t', header = True, index = False)

# SAVs in Moderns <a class = 'anchor' id = 'SAVsinmoderns'></a>

## Introgressed SAV Distribution <a class = 'anchor' id = 'introgressedSAVdistribution'></a>

We know how splice altering variants appear to be distributed among archaics. But how are introgressed variants distributed among the archaic variants? Let's use the archaic-specific variants as a background so we will assess those first.

In [445]:
archaic_specific_2 = data_2[data_2['Vernot_allele_origin']=='archaic-specific']
archaic_dist_2 = archaic_specific_2.groupby(['distribution']).size().to_frame('archaic_distribution').reset_index()
archaic_dist_2

,distribution,archaic_distribution
0,Altai,300
1,Chagyrskaya,169
2,Denisovan,916
3,Late Neanderthal,94
4,Neanderthal,236
5,Other,128
6,Shared,112
7,Vindija,231


In [446]:
archaic_dist_2 = archaic_dist_2.drop([2, 5, 6])
archaic_dist_2['prop'] = archaic_dist_2['archaic_distribution']/archaic_dist_2['archaic_distribution'].sum()
archaic_dist_2

,distribution,archaic_distribution,prop
0,Altai,300,0.291262
1,Chagyrskaya,169,0.164078
3,Late Neanderthal,94,0.091262
4,Neanderthal,236,0.229126
7,Vindija,231,0.224272


In [447]:
Vernot_introgressed_dist_2 = data_2[data_2['Vernot_allele_origin']=='introgressed'].groupby(['distribution']).size().to_frame('count').reset_index()
Vernot_introgressed_dist_2

,distribution,count
0,Altai,7
1,Denisovan,5
2,Neanderthal,143
3,Other,21
4,Shared,68


In [448]:
Vernot_introgressed_dist_2 = Vernot_introgressed_dist_2.drop([1, 3, 4])
Vernot_introgressed_dist_2['prop'] = Vernot_introgressed_dist_2['count'] / Vernot_introgressed_dist_2['count'].sum()
Vernot_introgressed_dist_2

,distribution,count,prop
0,Altai,7,0.046667
2,Neanderthal,143,0.953333


Repeat for delta >= 0.5.

In [449]:
archaic_specific_5 = data_5[data_5['Vernot_allele_origin']=='archaic-specific']
archaic_dist_5 = archaic_specific_5.groupby(['distribution']).size().to_frame('archaic_distribution').reset_index()
archaic_dist_5

,distribution,archaic_distribution
0,Altai,57
1,Chagyrskaya,27
2,Denisovan,165
3,Late Neanderthal,10
4,Neanderthal,36
5,Other,27
6,Shared,27
7,Vindija,55


In [450]:
archaic_dist_5 = archaic_dist_5.drop([2, 5, 6])
archaic_dist_5['prop'] = archaic_dist_5['archaic_distribution']/archaic_dist_5['archaic_distribution'].sum()
archaic_dist_5

,distribution,archaic_distribution,prop
0,Altai,57,0.308108
1,Chagyrskaya,27,0.145946
3,Late Neanderthal,10,0.054054
4,Neanderthal,36,0.194595
7,Vindija,55,0.297297


In [451]:
Vernot_introgressed_dist_5 = data_5[data_5['Vernot_allele_origin']=='introgressed'].groupby(['distribution']).size().to_frame('count').reset_index()
Vernot_introgressed_dist_5

,distribution,count
0,Altai,1
1,Denisovan,1
2,Neanderthal,38
3,Other,5
4,Shared,14


In [452]:
Vernot_introgressed_dist_5 = Vernot_introgressed_dist_5.drop([1, 3, 4])
Vernot_introgressed_dist_5['prop'] = Vernot_introgressed_dist_5['count'] / Vernot_introgressed_dist_5['count'].sum()
Vernot_introgressed_dist_5

,distribution,count,prop
0,Altai,1,0.025641
2,Neanderthal,38,0.974359


Now for Browning.

In [453]:
archaic_specific_2 = data_2[data_2['Vernot_allele_origin']=='archaic-specific']
archaic_dist_2 = archaic_specific_2.groupby(['distribution']).size().to_frame('archaic_distribution').reset_index()
archaic_dist_2

,distribution,archaic_distribution
0,Altai,300
1,Chagyrskaya,169
2,Denisovan,916
3,Late Neanderthal,94
4,Neanderthal,236
5,Other,128
6,Shared,112
7,Vindija,231


In [454]:
archaic_dist_2 = archaic_dist_2.drop([2, 5, 6])
archaic_dist_2['prop'] = archaic_dist_2['archaic_distribution']/archaic_dist_2['archaic_distribution'].sum()
archaic_dist_2

,distribution,archaic_distribution,prop
0,Altai,300,0.291262
1,Chagyrskaya,169,0.164078
3,Late Neanderthal,94,0.091262
4,Neanderthal,236,0.229126
7,Vindija,231,0.224272


In [455]:
Browning_introgressed_dist_2 = data_2[data_2['Browning_allele_origin']=='introgressed'].groupby(['distribution']).size().to_frame('count').reset_index()
Browning_introgressed_dist_2

,distribution,count
0,Altai,17
1,Denisovan,2
2,Neanderthal,207
3,Other,38
4,Shared,122


In [456]:
Browning_introgressed_dist_2 = Browning_introgressed_dist_2.drop([1, 3, 4])
Browning_introgressed_dist_2['prop'] = Browning_introgressed_dist_2['count'] / Browning_introgressed_dist_2['count'].sum()
Browning_introgressed_dist_2

,distribution,count,prop
0,Altai,17,0.075893
2,Neanderthal,207,0.924107


In [457]:
archaic_specific_5 = data_5[data_5['Vernot_allele_origin']=='archaic-specific']
archaic_dist_5 = archaic_specific_5.groupby(['distribution']).size().to_frame('archaic_distribution').reset_index()
archaic_dist_5

,distribution,archaic_distribution
0,Altai,57
1,Chagyrskaya,27
2,Denisovan,165
3,Late Neanderthal,10
4,Neanderthal,36
5,Other,27
6,Shared,27
7,Vindija,55


In [458]:
archaic_dist_5 = archaic_dist_5.drop([2, 5, 6])
archaic_dist_5['prop'] = archaic_dist_5['archaic_distribution']/archaic_dist_5['archaic_distribution'].sum()
archaic_dist_5

,distribution,archaic_distribution,prop
0,Altai,57,0.308108
1,Chagyrskaya,27,0.145946
3,Late Neanderthal,10,0.054054
4,Neanderthal,36,0.194595
7,Vindija,55,0.297297


In [459]:
Browning_introgressed_dist_5 = data_5[data_5['Browning_allele_origin']=='introgressed'].groupby(['distribution']).size().to_frame('count').reset_index()
Browning_introgressed_dist_5

,distribution,count
0,Altai,4
1,Denisovan,2
2,Neanderthal,38
3,Other,4
4,Shared,23


In [460]:
Browning_introgressed_dist_5 = Browning_introgressed_dist_5.drop([1, 3, 4])
Browning_introgressed_dist_5['prop'] = Browning_introgressed_dist_5['count'] / Browning_introgressed_dist_5['count'].sum()
Browning_introgressed_dist_5

,distribution,count,prop
0,Altai,4,0.095238
2,Neanderthal,38,0.904762


## Allele Frequency and Max Delta <a class = 'anchor' id = 'allelefrequencymaxdelta'></a>

If deltas represent the probability of a variant being splice altering, this metric may be related to allele frequency in modern human populations such that variants with small deltas may be high frequency because they are not actually splice altering.

In [461]:
present_in_1KG = data[data['present_in_1KG'] == 'yes']
present_in_1KG_SA_2 = data_2[data_2['present_in_1KG'] == 'yes']
present_in_1KG_SA_5 = data_5[data_5['present_in_1KG'] == 'yes']

In [462]:
rho, p = spearmanr(present_in_1KG['1KG_allele_frequency'], present_in_1KG['delta_max'])
print(rho,p)

-0.007352829220282604 8.637985526942276e-14


In [463]:
rho, p = spearmanr(present_in_1KG_SA_2['1KG_allele_frequency'], present_in_1KG_SA_2['delta_max'])
print(rho,p)

-0.00015112162504674333 0.9928437359082407


In [464]:
rho, p = spearmanr(present_in_1KG_SA_5['1KG_allele_frequency'], present_in_1KG_SA_5['delta_max'])
print(rho,p)

-0.1122804631529113 0.005817787511787715


Let's also consider ancient vs introgressed variants. First Vernot.

In [465]:
Vernot_ancient_variants = data[data['Vernot_allele_origin'] == 'ancient']
Vernot_introgressed_variants = data[data['Vernot_allele_origin'] == 'introgressed']

Vernot_ancient_variants_SA_2 = data_2[data_2['Vernot_allele_origin'] == 'ancient']
Vernot_introgressed_variants_SA_2 = data_2[data_2['Vernot_allele_origin'] == 'introgressed']

Vernot_ancient_variants_SA_5 = data_5[data_5['Vernot_allele_origin'] == 'ancient']
Vernot_introgressed_variants_SA_5 = data_5[data_5['Vernot_allele_origin'] == 'introgressed']

In [466]:
rho, p = spearmanr(Vernot_ancient_variants['1KG_allele_frequency'], Vernot_ancient_variants['delta_max'])
print(rho,p)

-0.003314513124591188 0.006217887126235556


In [467]:
rho, p = spearmanr(Vernot_introgressed_variants['Vernot_introgressed_AF'], Vernot_introgressed_variants['delta_max'])
print(rho,p)

0.003996022439579673 0.2972389787199452


In [468]:
rho, p = spearmanr(Vernot_ancient_variants_SA_2['1KG_allele_frequency'], Vernot_ancient_variants_SA_2['delta_max'])
print(rho,p)

0.023032083134465274 0.27459969993034433


In [469]:
rho, p = spearmanr(Vernot_introgressed_variants_SA_2['Vernot_introgressed_AF'], Vernot_introgressed_variants_SA_2['delta_max'])
print(rho,p)

-0.23778363811750808 0.0001774412637169479


In [470]:
rho, p = spearmanr(Vernot_ancient_variants_SA_5['1KG_allele_frequency'], Vernot_ancient_variants_SA_5['delta_max'])
print(rho,p)

-0.12045161956844723 0.018365211572499436


In [471]:
rho, p = spearmanr(Vernot_introgressed_variants_SA_5['Vernot_introgressed_AF'], Vernot_introgressed_variants_SA_5['delta_max'])
print(rho,p)

-0.010180203372018522 0.9390018302513599


Now Browning.

In [472]:
Browning_ancient_variants = data[data['Browning_allele_origin'] == 'ancient']
Browning_introgressed_variants = data[data['Browning_allele_origin'] == 'introgressed']

Browning_ancient_variants_SA_2 = data_2[data_2['Browning_allele_origin'] == 'ancient']
Browning_introgressed_variants_SA_2 = data_2[data_2['Browning_allele_origin'] == 'introgressed']

Browning_ancient_variants_SA_5 = data_5[data_5['Browning_allele_origin'] == 'ancient']
Browning_introgressed_variants_SA_5 = data_5[data_5['Browning_allele_origin'] == 'introgressed']

In [473]:
rho, p = spearmanr(Browning_ancient_variants['1KG_allele_frequency'], Browning_ancient_variants['delta_max'])
print(rho,p)

-0.0014042305212856882 0.2501708002524762


In [474]:
rho, p = spearmanr(Browning_introgressed_variants['Browning_introgressed_AF'], Browning_introgressed_variants['delta_max'])
print(rho,p)

0.004092787486773656 0.1996781446277722


In [475]:
rho, p = spearmanr(Browning_ancient_variants_SA_2['1KG_allele_frequency'], Browning_ancient_variants_SA_2['delta_max'])
print(rho,p)

0.018438954258859618 0.3878837383270327


In [476]:
rho, p = spearmanr(Browning_introgressed_variants_SA_2['Browning_introgressed_AF'], Browning_introgressed_variants_SA_2['delta_max'])
print(rho,p)

-0.09956359455569352 0.05062683014640129


In [477]:
rho, p = spearmanr(Browning_ancient_variants_SA_5['1KG_allele_frequency'], Browning_ancient_variants_SA_5['delta_max'])
print(rho,p)

-0.11314820835053853 0.02825198996162118


In [478]:
rho, p = spearmanr(Browning_introgressed_variants_SA_5['Browning_introgressed_AF'], Browning_introgressed_variants_SA_5['delta_max'])
print(rho,p)

-0.13688008107823207 0.25500901988007507


Let's examine allele frequency between superpopulations. 

First Vernot.

In [479]:
Vernot_ancient_SA_2_AFR = Vernot_ancient_variants_SA_2['1KG_non_ASW_AFR_AF']
Vernot_ancient_SA_2_AFR.mean()

0.5226508643821909

In [480]:
Vernot_ancient_SA_2_non_AFR = pd.concat([Vernot_ancient_variants_SA_2['1KG_AMR_AF'], Vernot_ancient_variants_SA_2['1KG_EAS_AF'], Vernot_ancient_variants_SA_2['1KG_EUR_AF'], Vernot_ancient_variants_SA_2['1KG_SAS_AF']], axis=0)
Vernot_ancient_SA_2_non_AFR.mean()

0.4761589698046181

In [481]:
mannwhitneyu(Vernot_ancient_SA_2_AFR, Vernot_ancient_SA_2_non_AFR)

MannwhitneyuResult(statistic=10963956.0, pvalue=2.6636825400212013e-09)

Let's use the Vernot AFs here that are for the archaic allele. Important to note that AFR may include ASW samples.

In [482]:
Vernot_introgressed_SA_2_AFR = Vernot_introgressed_variants_SA_2['Vernot_AFR_AF']
Vernot_introgressed_SA_2_AFR.mean()

0.0005038524590163934

In [483]:
Vernot_introgressed_SA_2_non_AFR = pd.concat([Vernot_introgressed_variants_SA_2['Vernot_AMR_AF'], Vernot_introgressed_variants_SA_2['Vernot_EAS_AF'], Vernot_introgressed_variants_SA_2['Vernot_EUR_AF'], Vernot_introgressed_variants_SA_2['Vernot_SAS_AF']], axis=0)
Vernot_introgressed_SA_2_non_AFR.mean()

0.027982571721311474

In [484]:
mannwhitneyu(Vernot_introgressed_SA_2_AFR, Vernot_introgressed_SA_2_non_AFR)

MannwhitneyuResult(statistic=45360.5, pvalue=5.0924222907304063e-54)

Now Browning.

In [485]:
Browning_ancient_SA_2_AFR = Browning_ancient_variants_SA_2['1KG_non_ASW_AFR_AF']
Browning_ancient_SA_2_AFR.mean()

0.5356676302246378

In [486]:
Browning_ancient_SA_2_non_AFR = pd.concat([Browning_ancient_variants_SA_2['1KG_AMR_AF'], Browning_ancient_variants_SA_2['1KG_EAS_AF'], Browning_ancient_variants_SA_2['1KG_EUR_AF'], Browning_ancient_variants_SA_2['1KG_SAS_AF']], axis=0)
Browning_ancient_SA_2_non_AFR.mean()

0.4851947608200456

In [487]:
mannwhitneyu(Browning_ancient_SA_2_AFR, Browning_ancient_SA_2_non_AFR)

MannwhitneyuResult(statistic=10506748.0, pvalue=5.408902214406806e-11)

We need to do a little extra work for Browning to handle the introgressed reference variants. Let's start with AFR.

In [488]:
Browning_introgressed_SA_2_AFR_refs = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 0), '1KG_non_ASW_AFR_AF']
Browning_introgressed_SA_2_AFR_refs = 1-Browning_introgressed_SA_2_AFR_refs
Browning_introgressed_SA_2_AFR_refs.head(5)

235107    0.001949
735814    0.000975
Name: 1KG_non_ASW_AFR_AF, dtype: float64

In [489]:
Browning_introgressed_SA_2_AFR_alts = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 1), '1KG_non_ASW_AFR_AF']
Browning_introgressed_SA_2_AFR_alts.head(5)

10928    0.003899
14504    0.007797
17107    0.000000
17139    0.000000
19011    0.000000
Name: 1KG_non_ASW_AFR_AF, dtype: float64

In [490]:
Browning_introgressed_SA_2_AFR = pd.concat([Browning_introgressed_SA_2_AFR_refs, Browning_introgressed_SA_2_AFR_alts], ignore_index=True)
Browning_introgressed_SA_2_AFR.head(5)

0    0.001949
1    0.000975
2    0.003899
3    0.007797
4    0.000000
Name: 1KG_non_ASW_AFR_AF, dtype: float64

In [491]:
Browning_introgressed_SA_2_AFR.mean()

0.0014736221632773263

Now do AMR, EAS, EUR, and SAS.

In [492]:
Browning_introgressed_SA_2_AMR_refs = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 0), '1KG_AMR_AF']
Browning_introgressed_SA_2_EAS_refs = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 0), '1KG_EAS_AF']
Browning_introgressed_SA_2_EUR_refs = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 0), '1KG_EUR_AF']
Browning_introgressed_SA_2_SAS_refs = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 0), '1KG_SAS_AF']
Browning_introgressed_SA_2_non_AFR_refs = pd.concat([Browning_introgressed_SA_2_AMR_refs, Browning_introgressed_SA_2_EAS_refs, Browning_introgressed_SA_2_EUR_refs, Browning_introgressed_SA_2_SAS_refs], ignore_index=True)
Browning_introgressed_SA_2_non_AFR_refs = 1 - Browning_introgressed_SA_2_non_AFR_refs
Browning_introgressed_SA_2_non_AFR_refs.head(10)

0    0.25
1    0.07
2    0.00
3    0.10
4    0.17
5    0.14
6    0.03
7    0.04
dtype: float64

In [493]:
Browning_introgressed_SA_2_AMR_alts = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 1), '1KG_AMR_AF']
Browning_introgressed_SA_2_EAS_alts = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 1), '1KG_EAS_AF']
Browning_introgressed_SA_2_EUR_alts = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 1), '1KG_EUR_AF']
Browning_introgressed_SA_2_SAS_alts = Browning_introgressed_variants_SA_2.loc[(Browning_introgressed_variants_SA_2['Browning_ref_alt'] == 1), '1KG_SAS_AF']
Browning_introgressed_SA_2_non_AFR_alts = pd.concat([Browning_introgressed_SA_2_AMR_alts, Browning_introgressed_SA_2_EAS_alts, Browning_introgressed_SA_2_EUR_alts, Browning_introgressed_SA_2_SAS_alts], ignore_index=True)
Browning_introgressed_SA_2_non_AFR_alts.head(10)

0    0.00
1    0.06
2    0.01
3    0.02
4    0.02
5    0.00
6    0.00
7    0.00
8    0.01
9    0.00
dtype: float64

In [494]:
Browning_introgressed_SA_2_non_AFR = pd.concat([Browning_introgressed_SA_2_non_AFR_refs, Browning_introgressed_SA_2_non_AFR_alts], ignore_index=True)
Browning_introgressed_SA_2_non_AFR.mean()

0.03881299734748011

In [495]:
mannwhitneyu(Browning_introgressed_SA_2_AFR, Browning_introgressed_SA_2_non_AFR)

MannwhitneyuResult(statistic=nan, pvalue=nan)

## Introgressed Genes <a class = 'anchor' id = 'introgressedgenes'></a>

Let's take a quick peek at the highest frequency introgressed genes.

In [496]:
SA_2_introgressed = data_2[data_2['Vernot_allele_origin']=='introgressed']
SA_2_introgressed_AF = SA_2_introgressed.sort_values(['Vernot_introgressed_AF'], ascending = [False])
SA_2_introgressed_AF.head(20)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
939756,chr20,62224595,G,A,G,derived,snv,0/1,1/1,0/0,0/1,True,True,False,True,Neanderthal,yes,1138.0,5096.0,0.22,0.51,0.21,0.01,0.11,0.31,0.000975,yes,23692.0,143104.0,0.165558,yes,G,A,1.0,0.03503,0.00099,0.10663,0.51488,0.21173,0.03704,0.30982,G,G,A,chr20_62195671_62229244,0.228810,yes,1.0,0.22,GMEB2,0.66048,2.202000,0.049787,3.94620,-0.837,0.01,0.00,0.41,0.02,0.41,15,-44,2,-2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,21.0,yes,introgressed,introgressed
246812,chr11,11507780,G,A,G,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,917.0,5096.0,0.18,0.14,0.31,0.02,0.21,0.28,0.003899,yes,33636.0,143182.0,0.234918,yes,G,A,1.0,0.05732,0.00397,0.21182,0.13988,0.31014,0.09259,0.28323,G,A,NaN,chr11_11494384_11530287,0.189808,yes,1.0,0.18,GALNT18,0.87026,0.879900,0.387360,3.02520,-0.302,0.00,0.00,0.21,0.00,0.21,37,-14,2,-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,introgressed,introgressed
1058119,chr4,38805942,G,C,G,derived,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,911.0,5096.0,0.18,0.39,0.22,0.01,0.15,0.16,0.001949,yes,19931.0,143224.0,0.139160,yes,G,C,1.0,0.04459,0.00198,0.14986,0.39484,0.22068,0.05556,0.16360,G,G,C,chr4_38294200_38857896,0.186192,yes,1.0,0.18,TLR1,1.09920,-0.701570,1.411200,-1.72720,-0.219,0.00,0.00,0.00,0.38,0.38,27,-15,1,-15,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,yes,introgressed,introgressed
1267615,chr6,52138226,C,G,C,derived,snv,0/1,0/0,0/0,0/0,True,False,False,False,Altai,yes,782.0,5096.0,0.15,0.24,0.17,0.00,0.09,0.29,0.001949,yes,17455.0,143064.0,0.122008,yes,C,G,1.0,0.01274,0.00198,0.09366,0.24206,0.16600,0.09259,0.29448,C,C,G,chr6_52138225

In [497]:
SA_2_introgressed = data_2[data_2['Browning_allele_origin']=='introgressed']
SA_2_introgressed_AF = SA_2_introgressed.sort_values(['Browning_introgressed_AF'], ascending = [False])
SA_2_introgressed_AF.head(10)

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
894154,chr2,238970511,G,A,G,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,1274.0,5096.0,0.25,0.23,0.45,0.04,0.43,0.23,0.018519,yes,44715.0,143004.0,0.312684,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.25,SCLY,1.01520,-0.091207,0.759470,1.02300,0.561,0.00,0.00,0.09,0.46,0.46,31,-5,-34,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,yes,ancient,introgressed
894150,chr2,238970308,A,G,A,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,1274.0,5096.0,0.25,0.23,0.45,0.04,0.43,0.23,0.018519,yes,44716.0,143046.0,0.312599,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.25,SCLY,1.01520,-0.091207,0.759470,1.02300,0.402,0.00,0.25,0.03,0.00,0.25,48,2,39,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,yes,ancient,introgressed
894153,chr2,238970511,G,A,G,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,1274.0,5096.0,0.25,0.23,0.45,0.04,0.43,0.23,0.018519,yes,44715.0,143004.0,0.312684,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.25,UBE2F-SCLY,NaN,NaN,NaN,NaN,0.561,0.00,0.00,0.04,0.31,0.31,31,-5,-34,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,yes,ancient,introgressed
939756,chr20,62224595,G,A,G,derived,snv,0/1,1/1,0/0,0/1,True,True,False,True,Neanderthal,yes,1138.0,5096.0,0.22,0.51,0.21,0.01,0.11,0.31,0.000975,yes,23692.0,143104.0,0.165558,yes,G,A,1.0,0.03503,0.00099,0.10663,0.51488,0.21173,0.03704,0.30982,G,G,A,chr20_62195671_62229244,0.228810,yes,1.0,0.22,GMEB2,0.66048,2.202000,0.049787,3.94620,-0.837,0.01,0.00,0.41,0.02,0.41,15,-44,2,-2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1

## ASE <a class = 'anchor' id = 'ASE'></a>

Is there overlap between introgressed variants implicated in allele-specific expression from McCoy et al. 2017 and our data?

In [498]:
mccoy_header = ['chrom','pos','ref_allele','alt_allele']
mccoy = pd.read_csv('../ASE/McCoy_et_al_2017_ASE_variants.txt', sep = '\t', names = mccoy_header)
mccoy.head(10)

,chrom,pos,ref_allele,alt_allele
0,chr9,34569494,G,T
1,chr10,126696061,T,G
2,chr11,8454637,A,C
3,chr1,222987816,A,T
4,chr2,107872056,T,C
5,chr13,86949835,C,G
6,chr17,8026078,C,G
7,chr4,149556752,G,A
8,chr16,86754480,C,T
9,chr8,130718756,C,G


In [499]:
len(mccoy)

1236

In [500]:
mccoy_merge = pd.merge(data[['chrom','pos','ref_allele','alt_allele','delta_max','Vernot_allele_origin','Browning_allele_origin','annotation']], mccoy[['chrom','pos','ref_allele','alt_allele']], on=['chrom','pos','ref_allele','alt_allele'], indicator=True)
mccoy_merge.head(10)

,chrom,pos,ref_allele,alt_allele,delta_max,Vernot_allele_origin,Browning_allele_origin,annotation,_merge
0,chr1,3383286,G,A,0.00,introgressed,introgressed,ARHGEF16,both
1,chr1,3411396,T,C,0.01,introgressed,introgressed,MEGF6,both
2,chr1,3411561,G,A,0.01,introgressed,introgressed,MEGF6,both
3,chr1,3411652,G,A,0.04,introgressed,introgressed,MEGF6,both
4,chr1,3551792,G,A,0.01,introgressed,introgressed,WRAP73,both
5,chr1,10368282,G,C,0.00,introgressed,introgressed,KIF1B,both
6,chr1,11905995,C,A,0.00,introgressed,introgressed,NPPA,both
7,chr1,12247128,C,T,0.00,introgressed,introgressed,TNFRSF1B,both
8,chr1,12260399,C,T,0.01,introgressed,introgressed,TNFRSF1B,both
9,chr1,12261613,T,C,0.00,introgressed,introgressed,TNFRSF1B,both


In [501]:
len(mccoy_merge)

862

In [502]:
mccoy_SAV = mccoy_merge[mccoy_merge['delta_max'] >= 0.2]

In [503]:
len(mccoy_SAV)

16

In [504]:
mccoy_SAV.head(16)

,chrom,pos,ref_allele,alt_allele,delta_max,Vernot_allele_origin,Browning_allele_origin,annotation,_merge
22,chr1,22174518,G,T,0.98,introgressed,introgressed,HSPG2,both
36,chr1,55537474,C,G,0.33,introgressed,low-confidence ancient,USP24,both
46,chr1,161681848,C,T,0.20,introgressed,introgressed,FCRLA,both
90,chr1,212985592,G,A,0.52,introgressed,introgressed,TATDN3,both
170,chr11,86159859,G,A,0.26,introgressed,ancient,ME3,both
269,chr12,133272470,G,T,0.26,introgressed,introgressed,PXMP2,both
270,chr12,133272470,G,T,0.26,introgressed,introgressed,RP13-672B3.2,both
365,chr15,85403496,G,A,0.33,introgressed,introgressed,ALPK3,both
403,chr16,88924425,C,G,0.41,introgressed,introgressed,TRAPPC2L,both
449,chr19,40913595,G,A,0.23,low-confidence ancient,introgressed,PRX,both


# Gene Expression <a class = 'anchor' id = 'geneexpression'></a>

## sQTLs <a class = 'anchor' id = 'sqtls'></a>

Now let's examine overlap between our variants and sQTLs from GTEx.

In [505]:
data.groupby('sQTL').size().to_frame('N')

,N
sQTL,
no,1359027
yes,248323


In [506]:
non_sQTLs = data[data['N_GTEx_tissues'].isnull()]
sQTLs = data[~data['N_GTEx_tissues'].isnull()]

In [507]:
len(non_sQTLs)

1359027

In [508]:
len(sQTLs)

248323

In [509]:
sQTLs['delta_max'].min()

0.0

In [510]:
sQTLs['delta_max'].max()

1.0

In [511]:
sQTLs['delta_max'].mean() - non_sQTLs['delta_max'].mean()

0.0025026787676935133

In [512]:
mannwhitneyu(non_sQTLs['delta_max'], sQTLs['delta_max'])

MannwhitneyuResult(statistic=160822988988.5, pvalue=0.0)

Let's save the delta max for sQTLs just in case.

In [513]:
sQTLs.to_csv('sQTLs.txt', sep="\t", header=True, index=False)

Now let's look at SAV sQTLs.

In [514]:
data_2.groupby('sQTL').size().to_frame('N')

,N
sQTL,
no,4569
yes,1381


How many possible SAVs are sQTLs?

In [515]:
1381/(5950-2186)

0.3668969181721573

In [516]:
data_2.groupby('N_GTEx_tissues').size().reset_index(name='N')

,N_GTEx_tissues,N
0,1.0,297
1,2.0,130
2,3.0,93
3,4.0,76
4,5.0,57
5,6.0,39
6,7.0,36
7,8.0,45
8,9.0,32
9,10.0,25


In [517]:
non_SAV_sQTLs = data_2[data_2['N_GTEx_tissues'].isnull()]
SAV_sQTLs = data_2[~data_2['N_GTEx_tissues'].isnull()]

In [518]:
len(non_SAV_sQTLs)

4569

In [519]:
len(SAV_sQTLs)

1381

What does their distribution look like?

In [520]:
SAV_sQTLs.groupby(['Vernot_allele_origin']).size().to_frame('N')

,N
Vernot_allele_origin,
ancient,1145
introgressed,52
low-confidence ancient,184


In [521]:
SAV_sQTLs.groupby(['Browning_allele_origin']).size().to_frame('N')

,N
Browning_allele_origin,
ancient,1113
introgressed,95
low-confidence ancient,173


### sQTL Allele Origin Differences <a class = 'anchor' id = 'sqtlalleleorigindifferences'></a>

Check the tissue distribution for tissue specific variants. 

In [522]:
tissues = data_2[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood','N_GTEx_tissues']]
tissues.head(10)

,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,15.0
218,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14.0
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0
377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,49.0


Let's get totals for the 49 tissues.

In [523]:
tissue_totals = tissues.sum()

What if we compare ancient vs. introgressed?

In [524]:
Vernot_ancient_tissues = data_2[data_2['Vernot_allele_origin'] == 'ancient']
Vernot_ancient_tissues = Vernot_ancient_tissues[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']]
Vernot_ancient_tissue_totals = Vernot_ancient_tissues.sum()

In [525]:
Vernot_introgressed_tissues = data_2[data_2['Vernot_allele_origin'] == 'introgressed']
Vernot_introgressed_tissues = Vernot_introgressed_tissues[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']]
Vernot_introgressed_tissue_totals = Vernot_introgressed_tissues.sum()

In [526]:
Browning_ancient_tissues = data_2[data_2['Browning_allele_origin'] == 'ancient']
Browning_ancient_tissues = Browning_ancient_tissues[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']]
Browning_ancient_tissue_totals = Browning_ancient_tissues.sum()

In [527]:
Browning_introgressed_tissues = data_2[data_2['Browning_allele_origin'] == 'introgressed']
Browning_introgressed_tissues = Browning_introgressed_tissues[['Adipose_Subcutaneous','Adipose_Visceral_Omentum','Adrenal_Gland','Artery_Aorta','Artery_Coronary','Artery_Tibial','Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra','Breast_Mammary_Tissue','Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes','Colon_Sigmoid','Colon_Transverse','Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa','Esophagus_Muscularis','Heart_Atrial_Appendage','Heart_Left_Ventricle','Kidney_Cortex','Liver','Lung','Minor_Salivary_Gland','Muscle_Skeletal','Nerve_Tibial','Ovary','Pancreas','Pituitary','Prostate','Skin_Not_Sun_Exposed_Suprapubic','Skin_Sun_Exposed_Lower_leg','Small_Intestine_Terminal_Ileum','Spleen','Stomach','Testis','Thyroid','Uterus','Vagina','Whole_Blood']]
Browning_introgressed_tissue_totals = Browning_introgressed_tissues.sum()

In [528]:
allele_origin_tissue_counts = pd.concat({'Vernot_ancient': Vernot_ancient_tissue_totals, 'Vernot_introgressed': Vernot_introgressed_tissue_totals, 'Browning_ancient': Browning_ancient_tissue_totals, 'Browning_introgressed': Browning_introgressed_tissue_totals}, axis = 1)
allele_origin_tissue_counts['tissue'] = allele_origin_tissue_counts.index
allele_origin_tissue_counts = allele_origin_tissue_counts.reset_index()
allele_origin_tissue_counts = allele_origin_tissue_counts[['tissue','Vernot_ancient','Vernot_introgressed','Browning_ancient','Browning_introgressed']]
allele_origin_tissue_counts.head(10)

,tissue,Vernot_ancient,Vernot_introgressed,Browning_ancient,Browning_introgressed
0,Adipose_Subcutaneous,530.0,25.0,511.0,45.0
1,Adipose_Visceral_Omentum,451.0,12.0,436.0,27.0
2,Adrenal_Gland,279.0,6.0,275.0,11.0
3,Artery_Aorta,433.0,16.0,423.0,27.0
4,Artery_Coronary,259.0,5.0,258.0,6.0
5,Artery_Tibial,525.0,19.0,508.0,40.0
6,Brain_Amygdala,120.0,2.0,119.0,3.0
7,Brain_Anterior_cingulate_cortex_BA24,158.0,3.0,158.0,3.0
8,Brain_Caudate_basal_ganglia,226.0,7.0,225.0,8.0
9,Brain_Cerebellar_Hemisphere,283.0,10.0,275.0,19.0


Let's also add the GTEx sample size.

In [529]:
allele_origin_tissue_counts['sample_size'] = [663,541,258,432,240,663,152,176,246,215,241,255,209,197,202,246,205,159,139,459,504,174,373,406,375,555,515,429,432,85,226,578,162,803,619,180,328,283,245,604,701,187,241,359,361,653,142,156,755]
allele_origin_tissue_counts.head(5)

,tissue,Vernot_ancient,Vernot_introgressed,Browning_ancient,Browning_introgressed,sample_size
0,Adipose_Subcutaneous,530.0,25.0,511.0,45.0,663
1,Adipose_Visceral_Omentum,451.0,12.0,436.0,27.0,541
2,Adrenal_Gland,279.0,6.0,275.0,11.0,258
3,Artery_Aorta,433.0,16.0,423.0,27.0,432
4,Artery_Coronary,259.0,5.0,258.0,6.0,240


Check the correlations between each allele origin and sample size.

In [530]:
rho, p = spearmanr(allele_origin_tissue_counts['Browning_ancient'], allele_origin_tissue_counts['sample_size'])
print(rho,p)

0.9133729685808691 5.732392167246112e-20


In [531]:
rho, p = spearmanr(allele_origin_tissue_counts['Browning_introgressed'], allele_origin_tissue_counts['sample_size'])
print(rho,p)

0.8864031196766793 2.4665240198700857e-17


In [532]:
rho, p = spearmanr(allele_origin_tissue_counts['Vernot_ancient'], allele_origin_tissue_counts['sample_size'])
print(rho,p)

0.9118742676565625 8.434829773157069e-20


In [533]:
rho, p = spearmanr(allele_origin_tissue_counts['Vernot_introgressed'], allele_origin_tissue_counts['sample_size'])
print(rho,p)

0.848729659663223 1.3402094505815186e-14


Now for correlations among the Vernot and Browning allele origins.

In [534]:
rho, p = spearmanr(allele_origin_tissue_counts['Browning_ancient'], allele_origin_tissue_counts['Browning_introgressed'])
print(rho,p)

0.9432898208149728 3.797623224449068e-24


In [535]:
rho, p = spearmanr(allele_origin_tissue_counts['Vernot_ancient'], allele_origin_tissue_counts['Vernot_introgressed'])
print(rho,p)

0.9195145487047715 1.090872224680547e-20


Save this dataframe.

In [536]:
allele_origin_tissue_counts.to_csv('../GTEx_sQTLs/sQTLs_allele_origin_tissue_counts.txt', sep = '\t', header = True, index = False)

### Tissue Specifcity <a class = 'anchor' id = 'tissuespecificity'></a>

Now let's look at tissue specificity.

In [537]:
tissues[tissues['N_GTEx_tissues'] < 3].sum().to_frame('N_sQTLs')

,N_sQTLs
Adipose_Subcutaneous,14.0
Adipose_Visceral_Omentum,12.0
Adrenal_Gland,1.0
Artery_Aorta,11.0
Artery_Coronary,1.0
Artery_Tibial,29.0
Brain_Amygdala,1.0
Brain_Anterior_cingulate_cortex_BA24,2.0
Brain_Caudate_basal_ganglia,2.0
Brain_Cerebellar_Hemisphere,5.0


In [538]:
data_2[data_2['N_GTEx_tissues'] > 40].groupby('Vernot_allele_origin').size().to_frame('N_sQTLs')

,N_sQTLs
Vernot_allele_origin,
ancient,107
low-confidence ancient,5


In [539]:
data_2[data_2['N_GTEx_tissues'] > 40].groupby('Browning_allele_origin').size().to_frame('N_sQTLs')

,N_sQTLs
Browning_allele_origin,
ancient,107
low-confidence ancient,5


In [540]:
data_2[data_2['Vernot_allele_origin'] == 'ancient'].groupby('N_GTEx_tissues').size().to_frame('N')

,N
N_GTEx_tissues,
1.0,225
2.0,98
3.0,76
4.0,60
5.0,48
6.0,30
7.0,32
8.0,39
9.0,25


In [541]:
data_2[data_2['Browning_allele_origin'] == 'ancient'].groupby('N_GTEx_tissues').size().to_frame('N')

,N
N_GTEx_tissues,
1.0,220
2.0,98
3.0,75
4.0,56
5.0,47
6.0,30
7.0,31
8.0,37
9.0,25


In [542]:
data_2[data_2['N_GTEx_tissues'] > 40].groupby('distribution').size().to_frame('N_sQTLs')

,N_sQTLs
distribution,
Altai,2
Chagyrskaya,3
Denisovan,8
Late Neanderthal,2
Neanderthal,9
Other,14
Shared,74


In [543]:
data_2[data_2['Vernot_allele_origin'] == 'introgressed'].groupby('N_GTEx_tissues').size().to_frame('N')

,N
N_GTEx_tissues,
1.0,12
2.0,11
3.0,1
4.0,2
5.0,4
6.0,3
7.0,1
9.0,2
15.0,1


In [544]:
data_2[data_2['Browning_allele_origin'] == 'introgressed'].groupby('N_GTEx_tissues').size().to_frame('N')

,N
N_GTEx_tissues,
1.0,20
2.0,13
3.0,3
4.0,7
5.0,6
6.0,5
7.0,3
8.0,2
9.0,2


Do ancient and introgressed sQTLs differ in the average number of tissues? First let's look at Vernot.

In [545]:
Vernot_ancient_SAV_sQTLs = SAV_sQTLs[SAV_sQTLs['Vernot_allele_origin'] == 'ancient']
Vernot_introgressed_SAV_sQTLs = SAV_sQTLs[SAV_sQTLs['Vernot_allele_origin'] == 'introgressed']

In [546]:
len(Vernot_ancient_SAV_sQTLs)

1145

In [547]:
len(Vernot_introgressed_SAV_sQTLs)

52

In [548]:
Vernot_ancient_SAV_sQTLs['N_GTEx_tissues'].mean() - Vernot_introgressed_SAV_sQTLs['N_GTEx_tissues'].mean()

4.922119583473295

In [549]:
mannwhitneyu(Vernot_ancient_SAV_sQTLs['N_GTEx_tissues'], Vernot_introgressed_SAV_sQTLs['N_GTEx_tissues'])

MannwhitneyuResult(statistic=35123.5, pvalue=0.02740137285965415)

Now Browning.

In [550]:
Browning_ancient_SAV_sQTLs = SAV_sQTLs[SAV_sQTLs['Browning_allele_origin'] == 'ancient']
Browning_introgressed_SAV_sQTLs = SAV_sQTLs[SAV_sQTLs['Browning_allele_origin'] == 'introgressed']

In [551]:
len(Browning_ancient_SAV_sQTLs)

1113

In [552]:
len(Browning_introgressed_SAV_sQTLs)

95

In [553]:
Browning_ancient_SAV_sQTLs['N_GTEx_tissues'].mean() - Browning_introgressed_SAV_sQTLs['N_GTEx_tissues'].mean()

5.0007471508961086

In [554]:
mannwhitneyu(Browning_ancient_SAV_sQTLs['N_GTEx_tissues'], Browning_introgressed_SAV_sQTLs['N_GTEx_tissues'])

MannwhitneyuResult(statistic=60337.0, pvalue=0.02149969439234634)

### European sQTLs <a class = 'anchor' id = 'eursqtls'></a>

Let's take a quick peek at European allele frequencies and N sQTL SAVs given that a lot of GTEx samples come from individuals with European ancestry.

In [555]:
EUR_SAVs = data_2.groupby('1KG_EUR_AF').size().to_frame('N_SAVs')
EUR_SAVs = EUR_SAVs.reset_index()
EUR_SAVs = EUR_SAVs.sort_values('1KG_EUR_AF', ascending=False)
EUR_SAVs

,1KG_EUR_AF,N_SAVs
100,1.00,171
99,0.99,25
98,0.98,19
97,0.97,11
96,0.96,15
95,0.95,16
94,0.94,21
93,0.93,11
92,0.92,14
91,0.91,21


In [556]:
EUR_SAV_sQTLs = SAV_sQTLs.groupby('1KG_EUR_AF').size().to_frame('N_SAV_sQTLs')
EUR_SAV_sQTLs = EUR_SAV_sQTLs.reset_index()
EUR_SAV_sQTLs

,1KG_EUR_AF,N_SAV_sQTLs
0,0.00,49
1,0.01,22
2,0.02,29
3,0.03,28
4,0.04,29
5,0.05,26
6,0.06,23
7,0.07,19
8,0.08,22
9,0.09,25


In [557]:
EUR_sQTL_proportion = pd.merge(EUR_SAVs, EUR_SAV_sQTLs, on='1KG_EUR_AF')
bins = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]
EUR_sQTL_proportion['binned_AF'] = pd.cut(EUR_sQTL_proportion['1KG_EUR_AF'], bins)
EUR_sQTL_proportion = EUR_sQTL_proportion.groupby('binned_AF').sum().reset_index()
EUR_sQTL_proportion['proportion_sQTLs'] = (EUR_sQTL_proportion['N_SAV_sQTLs'])/(EUR_sQTL_proportion['N_SAVs'])
EUR_sQTL_proportion['binned_AF'] = EUR_sQTL_proportion['binned_AF'].astype(str)
EUR_sQTL_proportion = EUR_sQTL_proportion[['binned_AF','N_SAVs','N_SAV_sQTLs','proportion_sQTLs']]
EUR_sQTL_proportion

,binned_AF,N_SAVs,N_SAV_sQTLs,proportion_sQTLs
0,"(0.0, 0.05]",447,134,0.299776
1,"(0.05, 0.1]",208,108,0.519231
2,"(0.1, 0.15]",148,84,0.567568
3,"(0.15, 0.2]",137,85,0.620438
4,"(0.2, 0.25]",110,68,0.618182
5,"(0.25, 0.3]",114,75,0.657895
6,"(0.3, 0.35]",89,61,0.685393
7,"(0.35, 0.4]",108,63,0.583333
8,"(0.4, 0.45]",97,62,0.639175
9,"(0.45, 0.5]",99,66,0.666667


In [558]:
EUR_sQTL_proportion.to_csv('../GTEx_sQTLs/EUR_sQTLs.txt', sep="\t", header=True, index=False)

## Gene-Level <a class = 'anchor' id = 'genelevel'></a>

In which tissues are archaic splice variants that occur in modern humans expressed? We have data kindly provided by Mary Lauren Benton on expression specifity metrics. That dataframe contains Ensembl IDs so first we need to build a dictionary to get the GENCODE annotation.

In [559]:
genes = []
ids = []

with open("../annotations/gencode.v24lift37.basic.annotation.gff3") as f: 
    lines = f.readlines()
for line in lines:
	info=line.split(';')
	for x in info:
		if x.startswith("gene_id="):
			b=re.split('=|\.',x) # use re here because there are multiple delimiters (';' and '.')
			ids.append(b[1])
		elif x.startswith("gene_name="):
			c=x.split("=")
			genes.append(c[1])
            
ensembl_id_to_gene_id = dict(zip(ids, genes))

Load in Mary Lauren's data.

In [560]:
expression = pd.read_csv("../gene_expression/gene-specificity_GTEx_TPM_alltissues.tsv", sep='\t', header=0)
expression.head(10)

,Gene,adipose tissue,adrenal gland,amygdala,basal ganglia,breast,cerebellum,cerebral cortex,"cervix, uterine",colon,endometrium,esophagus,fallopian tube,heart muscle,hippocampal formation,hypothalamus,kidney,liver,lung,midbrain,ovary,pancreas,pituitary gland,prostate,salivary gland,skeletal muscle,skin,small intestine,spinal cord,spleen,stomach,testis,thyroid gland,urinary bladder,vagina,entropy,rel_entropy,tau
0,ENSG00000000003,27.4,15.5,7.3,7.7,32.0,2.5,5.6,30.6,32.6,22.8,35.5,27.7,4.5,7.0,9.9,16.2,23.8,12.0,6.5,50.3,7.6,36.5,17.5,30.7,2.0,8.8,16.2,6.4,7.8,8.8,49.5,18.7,37.6,23.8,0.068330,0.037445,0.639135
1,ENSG00000000005,20.7,0.0,0.1,0.0,11.9,0.0,0.1,0.2,0.6,0.1,0.0,0.3,0.1,0.1,0.1,0.5,0.0,0.0,0.4,1.3,0.0,0.0,0.4,0.4,1.3,3.1,0.5,0.0,0.1,0.0,0.1,0.2,0.1,0.2,0.534839,0.404403,0.974235
2,ENSG00000000419,33.6,37.7,12.4,15.6,34.0,24.9,22.4,35.8,31.5,39.2,33.8,36.5,21.6,14.6,17.4,20.5,17.3,31.5,14.1,33.2,18.2,28.4,29.9,28.0,26.2,30.5,28.6,16.7,34.0,24.7,42.7,35.2,32.7,34.7,0.013113,0.006362,0.364559
3,ENSG00000000457,5.4,4.5,1.5,1.7,6.7,5.2,2.3,7.9,5.4,6.5,5.7,6.3,2.0,1.7,2.1,3.4,3.8,5.1,1.7,5.9,2.5,4.5,6.8,6.7,3.6,6.7,5.4,2.5,5.7,3.8,6.3,7.4,9.1,6.6,0.028105,0.010124,0.489510
4,ENSG00000000460,0.7,0.5,0.2,0.2,0.7,1.5,0.2,0.9,0.6,0.7,0.7,0.5,0.2,0.2,0.3,0.3,0.5,0.5,0.3,0.6,0.2,0.5,0.7,0.6,0.1,0.6,0.6,0.5,0.9,0.4,4.9,0.8,1.0,0.7,0.104682,0.051871,0.984539
5,ENSG00000000938,19.9,4.8,2.7,3.2,13.7,2.4,3.8,9.1,5.0,8.6,3.8,18.7,3.7,3.2,3.7,4.4,4.2,86.1,4.9,2.4,1.7,2.7,6.2,5.8,1.3,3.7,8.2,6.5,124.3,4.1,3.2,7.6,7.1,6.7,0.268393,0.194874,0.933421
6,ENSG00000000971,122.4,27.3,5.0,7.5,54.3,3.5,5.5,30.9,26.3,12.5,30.7,27.5,163.3,5.8,6.0,19.1,286.4,73.2,10.1,166.9,4.3,17.5,24.2,107.1,9.9,71.9,41.9,11.3,9.0,22.1,11.5,117.8,97.4,26.6,0.170275,0.163749,0.855013
7,ENSG00000001036,27.4,35.1,7.1,7.8,24.8,5.9,6.3,32.5,23.3,26.6,11.1,27.7,14.0,7.9,10.7,20.7,15.2,27.3,10.5,26.8,12.7,16.8,16.3,21.7,3.0,11.5,23.9,17.6,22.4,18.8,11.1,26.6,24.9,20.7,0.031989,0.019567,0.497885
8,ENSG00000001084,11.0,9.9,7.6,9.8,9.1,5.7,8.6,9.6,13.9,8.4,18.0,10.5,3.1,8.2,8.1,6.1,12.6,7.0,9.6,6.0,3.7,5.5,12.1,10.7,3.7,8.6,15.6,10.9,15.8,11.0,5.2,12.4,44.5,14.3,0.043669,0.033442,0.787334
9,ENSG00000001167,9.9,7.4,6.0,5.9,10.9,11.2,6.3,12.4,7.2,10.9,8.7,11.3,4.7,6.4,6.0,5.7,4.5,12.7,6.8,14.1,4.3,9.9,9.6,8.3,4.5,8.6,8.9,10.6,10.4,6.1,42.3,17.6,11.2,9.9,0.041457,0.015504,0.793037


Now map our dictionary.

In [561]:
expression['annotation'] = expression['Gene'].map(ensembl_id_to_gene_id)
expression.head(10)

,Gene,adipose tissue,adrenal gland,amygdala,basal ganglia,breast,cerebellum,cerebral cortex,"cervix, uterine",colon,endometrium,esophagus,fallopian tube,heart muscle,hippocampal formation,hypothalamus,kidney,liver,lung,midbrain,ovary,pancreas,pituitary gland,prostate,salivary gland,skeletal muscle,skin,small intestine,spinal cord,spleen,stomach,testis,thyroid gland,urinary bladder,vagina,entropy,rel_entropy,tau,annotation
0,ENSG00000000003,27.4,15.5,7.3,7.7,32.0,2.5,5.6,30.6,32.6,22.8,35.5,27.7,4.5,7.0,9.9,16.2,23.8,12.0,6.5,50.3,7.6,36.5,17.5,30.7,2.0,8.8,16.2,6.4,7.8,8.8,49.5,18.7,37.6,23.8,0.068330,0.037445,0.639135,TSPAN6
1,ENSG00000000005,20.7,0.0,0.1,0.0,11.9,0.0,0.1,0.2,0.6,0.1,0.0,0.3,0.1,0.1,0.1,0.5,0.0,0.0,0.4,1.3,0.0,0.0,0.4,0.4,1.3,3.1,0.5,0.0,0.1,0.0,0.1,0.2,0.1,0.2,0.534839,0.404403,0.974235,TNMD
2,ENSG00000000419,33.6,37.7,12.4,15.6,34.0,24.9,22.4,35.8,31.5,39.2,33.8,36.5,21.6,14.6,17.4,20.5,17.3,31.5,14.1,33.2,18.2,28.4,29.9,28.0,26.2,30.5,28.6,16.7,34.0,24.7,42.7,35.2,32.7,34.7,0.013113,0.006362,0.364559,DPM1
3,ENSG00000000457,5.4,4.5,1.5,1.7,6.7,5.2,2.3,7.9,5.4,6.5,5.7,6.3,2.0,1.7,2.1,3.4,3.8,5.1,1.7,5.9,2.5,4.5,6.8,6.7,3.6,6.7,5.4,2.5,5.7,3.8,6.3,7.4,9.1,6.6,0.028105,0.010124,0.489510,SCYL3
4,ENSG00000000460,0.7,0.5,0.2,0.2,0.7,1.5,0.2,0.9,0.6,0.7,0.7,0.5,0.2,0.2,0.3,0.3,0.5,0.5,0.3,0.6,0.2,0.5,0.7,0.6,0.1,0.6,0.6,0.5,0.9,0.4,4.9,0.8,1.0,0.7,0.104682,0.051871,0.984539,C1orf112
5,ENSG00000000938,19.9,4.8,2.7,3.2,13.7,2.4,3.8,9.1,5.0,8.6,3.8,18.7,3.7,3.2,3.7,4.4,4.2,86.1,4.9,2.4,1.7,2.7,6.2,5.8,1.3,3.7,8.2,6.5,124.3,4.1,3.2,7.6,7.1,6.7,0.268393,0.194874,0.933421,FGR
6,ENSG00000000971,122.4,27.3,5.0,7.5,54.3,3.5,5.5,30.9,26.3,12.5,30.7,27.5,163.3,5.8,6.0,19.1,286.4,73.2,10.1,166.9,4.3,17.5,24.2,107.1,9.9,71.9,41.9,11.3,9.0,22.1,11.5,117.8,97.4,26.6,0.170275,0.163749,0.855013,CFH
7,ENSG00000001036,27.4,35.1,7.1,7.8,24.8,5.9,6.3,32.5,23.3,26.6,11.1,27.7,14.0,7.9,10.7,20.7,15.2,27.3,10.5,26.8,12.7,16.8,16.3,21.7,3.0,11.5,23.9,17.6,22.4,18.8,11.1,26.6,24.9,20.7,0.031989,0.019567,0.497885,FUCA2
8,ENSG00000001084,11.0,9.9,7.6,9.8,9.1,5.7,8.6,9.6,13.9,8.4,18.0,10.5,3.1,8.2,8.1,6.1,12.6,7.0,9.6,6.0,3.7,5.5,12.1,10.7,3.7,8.6,15.6,10.9,15.8,11.0,5.2,12.4,44.5,14.3,0.043669,0.033442,0.787334,GCLC
9,ENSG00000001167,9.9,7.4,6.0,5.9,10.9,11.2,6.3,12.4,7.2,10.9,8.7,11.3,4.7,6.4,6.0,5.7,4.5,12.7,6.8,14.1,4.3,9.9,9.6,8.3,4.5,8.6,8.9,10.6,10.4,6.1,42.3,17.6,11.2,9.9,0.041457,0.015504,0.793037,NFYA


In [562]:
len(expression)

18392

Let's reorder the columns.

In [563]:
expression = expression[['Gene','annotation','entropy','rel_entropy','tau','adipose tissue','adrenal gland','amygdala','basal ganglia','breast','cerebellum','cerebral cortex','cervix, uterine','colon','endometrium','esophagus','fallopian tube','heart muscle','hippocampal formation','hypothalamus','kidney','liver','lung','midbrain','ovary','pancreas','pituitary gland','prostate','salivary gland','skeletal muscle','skin','small intestine','spinal cord','spleen','stomach','testis','thyroid gland','urinary bladder','vagina']]
expression.head(10)

,Gene,annotation,entropy,rel_entropy,tau,adipose tissue,adrenal gland,amygdala,basal ganglia,breast,cerebellum,cerebral cortex,"cervix, uterine",colon,endometrium,esophagus,fallopian tube,heart muscle,hippocampal formation,hypothalamus,kidney,liver,lung,midbrain,ovary,pancreas,pituitary gland,prostate,salivary gland,skeletal muscle,skin,small intestine,spinal cord,spleen,stomach,testis,thyroid gland,urinary bladder,vagina
0,ENSG00000000003,TSPAN6,0.068330,0.037445,0.639135,27.4,15.5,7.3,7.7,32.0,2.5,5.6,30.6,32.6,22.8,35.5,27.7,4.5,7.0,9.9,16.2,23.8,12.0,6.5,50.3,7.6,36.5,17.5,30.7,2.0,8.8,16.2,6.4,7.8,8.8,49.5,18.7,37.6,23.8
1,ENSG00000000005,TNMD,0.534839,0.404403,0.974235,20.7,0.0,0.1,0.0,11.9,0.0,0.1,0.2,0.6,0.1,0.0,0.3,0.1,0.1,0.1,0.5,0.0,0.0,0.4,1.3,0.0,0.0,0.4,0.4,1.3,3.1,0.5,0.0,0.1,0.0,0.1,0.2,0.1,0.2
2,ENSG00000000419,DPM1,0.013113,0.006362,0.364559,33.6,37.7,12.4,15.6,34.0,24.9,22.4,35.8,31.5,39.2,33.8,36.5,21.6,14.6,17.4,20.5,17.3,31.5,14.1,33.2,18.2,28.4,29.9,28.0,26.2,30.5,28.6,16.7,34.0,24.7,42.7,35.2,32.7,34.7
3,ENSG00000000457,SCYL3,0.028105,0.010124,0.489510,5.4,4.5,1.5,1.7,6.7,5.2,2.3,7.9,5.4,6.5,5.7,6.3,2.0,1.7,2.1,3.4,3.8,5.1,1.7,5.9,2.5,4.5,6.8,6.7,3.6,6.7,5.4,2.5,5.7,3.8,6.3,7.4,9.1,6.6
4,ENSG00000000460,C1orf112,0.104682,0.051871,0.984539,0.7,0.5,0.2,0.2,0.7,1.5,0.2,0.9,0.6,0.7,0.7,0.5,0.2,0.2,0.3,0.3,0.5,0.5,0.3,0.6,0.2,0.5,0.7,0.6,0.1,0.6,0.6,0.5,0.9,0.4,4.9,0.8,1.0,0.7
5,ENSG00000000938,FGR,0.268393,0.194874,0.933421,19.9,4.8,2.7,3.2,13.7,2.4,3.8,9.1,5.0,8.6,3.8,18.7,3.7,3.2,3.7,4.4,4.2,86.1,4.9,2.4,1.7,2.7,6.2,5.8,1.3,3.7,8.2,6.5,124.3,4.1,3.2,7.6,7.1,6.7
6,ENSG00000000971,CFH,0.170275,0.163749,0.855013,122.4,27.3,5.0,7.5,54.3,3.5,5.5,30.9,26.3,12.5,30.7,27.5,163.3,5.8,6.0,19.1,286.4,73.2,10.1,166.9,4.3,17.5,24.2,107.1,9.9,71.9,41.9,11.3,9.0,22.1,11.5,117.8,97.4,26.6
7,ENSG00000001036,FUCA2,0.031989,0.019567,0.497885,27.4,35.1,7.1,7.8,24.8,5.9,6.3,32.5,23.3,26.6,11.1,27.7,14.0,7.9,10.7,20.7,15.2,27.3,10.5,26.8,12.7,16.8,16.3,21.7,3.0,11.5,23.9,17.6,22.4,18.8,11.1,26.6,24.9,20.7
8,ENSG00000001084,GCLC,0.043669,0.033442,0.787334,11.0,9.9,7.6,9.8,9.1,5.7,8.6,9.6,13.9,8.4,18.0,10.5,3.1,8.2,8.1,6.1,12.6,7.0,9.6,6.0,3.7,5.5,12.1,10.7,3.7,8.6,15.6,10.9,15.8,11.0,5.2,12.4,44.5,14.3
9,ENSG00000001167,NFYA,0.041457,0.015504,0.793037,9.9,7.4,6.0,5.9,10.9,11.2,6.3,12.4,7.2,10.9,8.7,11.3,4.7,6.4,6.0,5.7,4.5,12.7,6.8,14.1,4.3,9.9,9.6,8.3,4.5,8.6,8.9,10.6,10.4,6.1,42.3,17.6,11.2,9.9


And save the dataframe while we're at it.

In [564]:
expression.to_csv('../gene_expression/GTEX_gene_expression_with_gene_ids.txt', sep="\t", header=True, index=False)

Why don't we examine how gene expression is related to the number of SAVs. Remember our N splice variants dataframe from earlier?

In [565]:
n_splice_variants_2.reset_index(inplace=True)
n_splice_variants_2.head(10)

,name,variant_count_2
0,A1BG,1
1,AADACL4,1
2,AADAT,1
3,AAMP,2
4,AAR2,1
5,AARS2,1
6,AATF,1
7,AATK,1
8,ABAT,1
9,ABC7-42404400C24.1,1


In [566]:
entropy_N_SAVs = n_splice_variants_2.rename(columns={'name':'annotation'})
entropy_N_SAVs = pd.merge(expression[['annotation','rel_entropy']], entropy_N_SAVs, on='annotation', how='left').dropna()
entropy_N_SAVs.head(10)

,annotation,rel_entropy,variant_count_2
4,C1orf112,0.051871,1.0
8,GCLC,0.033442,1.0
15,CFTR,0.462603,1.0
16,ANKIB1,0.010808,1.0
21,LAP3,0.020556,1.0
24,AOC1,0.483678,1.0
26,HECW1,0.300240,1.0
27,MAD1L1,0.010812,1.0
28,LASP1,0.013782,1.0
30,TMEM176A,0.180257,1.0


In [567]:
entropy_bins = [-np.inf, 0.1, 0.5, np.inf]
entropy_N_SAVs['rel_entropy_bin'] = pd.cut(entropy_N_SAVs['rel_entropy'], entropy_bins, labels=['low', 'medium', 'high'])
entropy_N_SAVs.head(10)

,annotation,rel_entropy,variant_count_2,rel_entropy_bin
4,C1orf112,0.051871,1.0,low
8,GCLC,0.033442,1.0,low
15,CFTR,0.462603,1.0,medium
16,ANKIB1,0.010808,1.0,low
21,LAP3,0.020556,1.0,low
24,AOC1,0.483678,1.0,medium
26,HECW1,0.300240,1.0,medium
27,MAD1L1,0.010812,1.0,low
28,LASP1,0.013782,1.0,low
30,TMEM176A,0.180257,1.0,medium


In [568]:
len(entropy_N_SAVs)

4061

In [569]:
low_entropy = entropy_N_SAVs.loc[entropy_N_SAVs['rel_entropy_bin'] == 'low', 'variant_count_2']
med_entropy = entropy_N_SAVs.loc[entropy_N_SAVs['rel_entropy_bin'] == 'medium', 'variant_count_2']
high_entropy = entropy_N_SAVs.loc[entropy_N_SAVs['rel_entropy_bin'] == 'high', 'variant_count_2']

In [570]:
len(low_entropy)

2434

In [571]:
len(med_entropy)

1268

In [572]:
len(high_entropy)

359

In [573]:
kruskal(low_entropy, med_entropy, high_entropy)

KruskalResult(statistic=1.8914270893492804, pvalue=0.3884023295875893)

Now let's look at the relationship between gene expression and delta max.

In [574]:
entropy_delta = data_2[['annotation','delta_max']]
entropy_delta

,annotation,delta_max
24,SAMD11,0.22
65,SAMD11,0.29
130,PLEKHN1,0.29
155,AGRN,0.24
210,AGRN,0.22
...,...,...
1606940,EHMT1,0.33
1606952,EHMT1,0.31
1606989,CACNA1B,0.28
1607338,CACNA1B,0.25


Map on the relative entropy values.

In [575]:
entropy_delta = pd.merge(entropy_delta, expression[['annotation','rel_entropy']], on='annotation', how='left')
entropy_delta = entropy_delta.dropna(axis = 0)
entropy_delta.head(10)

,annotation,delta_max,rel_entropy
0,SAMD11,0.22,0.089470
1,SAMD11,0.29,0.089470
2,PLEKHN1,0.29,0.295620
3,AGRN,0.24,0.036305
4,AGRN,0.22,0.036305
5,AGRN,0.43,0.036305
6,AGRN,0.48,0.036305
7,TTLL10,0.27,0.411293
8,SDF4,0.38,0.008625
9,SDF4,0.52,0.008625


In [576]:
len(entropy_delta)

5700

In [577]:
entropy_delta['rel_entropy_bin'] = pd.cut(entropy_delta['rel_entropy'], entropy_bins, labels=['low', 'medium', 'high'])
entropy_delta.head(10)

,annotation,delta_max,rel_entropy,rel_entropy_bin
0,SAMD11,0.22,0.089470,low
1,SAMD11,0.29,0.089470,low
2,PLEKHN1,0.29,0.295620,medium
3,AGRN,0.24,0.036305,low
4,AGRN,0.22,0.036305,low
5,AGRN,0.43,0.036305,low
6,AGRN,0.48,0.036305,low
7,TTLL10,0.27,0.411293,medium
8,SDF4,0.38,0.008625,low
9,SDF4,0.52,0.008625,low


In [578]:
low_entropy = entropy_delta.loc[entropy_delta['rel_entropy_bin'] == 'low', 'delta_max']
med_entropy = entropy_delta.loc[entropy_delta['rel_entropy_bin'] == 'medium', 'delta_max']
high_entropy = entropy_delta.loc[entropy_delta['rel_entropy_bin'] == 'high', 'delta_max']

In [579]:
len(low_entropy)

3384

In [580]:
len(med_entropy)

1826

In [581]:
len(high_entropy)

490

In [582]:
kruskal(low_entropy, med_entropy, high_entropy)

KruskalResult(statistic=6.599476430123113, pvalue=0.03689282412288473)

### Gene Allele Origin Differences <a class = 'anchor' id = 'genealleleorigindifferences'></a>

Now let's examine the correlation between the number of genes expressed among different allele origins. We'll use a TPM >= 1 to define a gene as "expressed".

First Vernot.

In [583]:
Vernot_ancient_expression = expression[expression['annotation'].isin(Vernot_ancient_genes_2['gene'])]
Vernot_ancient_expression = Vernot_ancient_expression.loc[:, ~Vernot_ancient_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Vernot_ancient_expression[Vernot_ancient_expression < 1] = 0
Vernot_ancient_expression[Vernot_ancient_expression >= 1] = 1
Vernot_ancient_expression_sums = Vernot_ancient_expression.sum()

In [584]:
Vernot_archaic_specific_expression = expression[expression['annotation'].isin(archaic_specific_genes_2['gene'])]
Vernot_archaic_specific_expression = Vernot_archaic_specific_expression.loc[:, ~Vernot_archaic_specific_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Vernot_archaic_specific_expression[Vernot_archaic_specific_expression < 1] = 0
Vernot_archaic_specific_expression[Vernot_archaic_specific_expression >= 1] = 1
Vernot_archaic_specific_expression_sums = Vernot_archaic_specific_expression.sum()

In [585]:
Vernot_introgressed_expression = expression[expression['annotation'].isin(Vernot_introgressed_genes_2['gene'])]
Vernot_introgressed_expression = Vernot_introgressed_expression.loc[:, ~Vernot_introgressed_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Vernot_introgressed_expression[Vernot_introgressed_expression < 1] = 0
Vernot_introgressed_expression[Vernot_introgressed_expression >= 1] = 1
Vernot_introgressed_expression_sums = Vernot_introgressed_expression.sum()

In [586]:
Vernot_expression = pd.DataFrame({ 'ancient': Vernot_ancient_expression_sums, 'archaic-specific': Vernot_archaic_specific_expression_sums, 'introgressed': Vernot_introgressed_expression_sums })
Vernot_expression

,ancient,archaic-specific,introgressed
adipose tissue,1298.0,1352.0,166.0
adrenal gland,1248.0,1264.0,167.0
amygdala,1285.0,1298.0,150.0
basal ganglia,1342.0,1377.0,158.0
breast,1309.0,1367.0,169.0
cerebellum,1321.0,1346.0,164.0
cerebral cortex,1375.0,1379.0,169.0
"cervix, uterine",1420.0,1430.0,174.0
colon,1362.0,1398.0,173.0
endometrium,1295.0,1342.0,164.0


In [587]:
rho, p = spearmanr(Vernot_expression['ancient'], Vernot_expression['archaic-specific'])
print(rho,p)

0.963019571146141 8.623592464716148e-20


In [588]:
rho, p = spearmanr(Vernot_expression['ancient'], Vernot_expression['introgressed'])
print(rho,p)

0.8562253956893983 1.0711542961831134e-10


In [589]:
rho, p = spearmanr(Vernot_expression['archaic-specific'], Vernot_expression['introgressed'])
print(rho,p)

0.8808690051661383 6.3617463212295455e-12


Now Browning.

In [590]:
Browning_ancient_expression = expression[expression['annotation'].isin(Browning_ancient_genes_2['gene'])]
Browning_ancient_expression = Browning_ancient_expression.loc[:, ~Browning_ancient_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Browning_ancient_expression[Browning_ancient_expression < 1] = 0
Browning_ancient_expression[Browning_ancient_expression >= 1] = 1
Browning_ancient_expression_sums = Browning_ancient_expression.sum()

In [591]:
Browning_archaic_specific_expression = expression[expression['annotation'].isin(archaic_specific_genes_2['gene'])]
Browning_archaic_specific_expression = Browning_archaic_specific_expression.loc[:, ~Browning_archaic_specific_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Browning_archaic_specific_expression[Browning_archaic_specific_expression < 1] = 0
Browning_archaic_specific_expression[Browning_archaic_specific_expression >= 1] = 1
Browning_archaic_specific_expression_sums = Browning_archaic_specific_expression.sum()

In [592]:
Browning_introgressed_expression = expression[expression['annotation'].isin(Browning_introgressed_genes_2['gene'])]
Browning_introgressed_expression = Browning_introgressed_expression.loc[:, ~Browning_introgressed_expression.columns.isin(['Gene', 'annotation','entropy','rel_entropy','tau'])]
Browning_introgressed_expression[Browning_introgressed_expression < 1] = 0
Browning_introgressed_expression[Browning_introgressed_expression >= 1] = 1
Browning_introgressed_expression_sums = Browning_introgressed_expression.sum()

In [593]:
Browning_expression = pd.DataFrame({ 'ancient': Browning_ancient_expression_sums, 'archaic-specific': Browning_archaic_specific_expression_sums, 'introgressed': Browning_introgressed_expression_sums })
Browning_expression

,ancient,archaic-specific,introgressed
adipose tissue,1274.0,1352.0,252.0
adrenal gland,1226.0,1264.0,250.0
amygdala,1262.0,1298.0,240.0
basal ganglia,1316.0,1377.0,252.0
breast,1284.0,1367.0,253.0
cerebellum,1296.0,1346.0,254.0
cerebral cortex,1350.0,1379.0,266.0
"cervix, uterine",1396.0,1430.0,269.0
colon,1339.0,1398.0,258.0
endometrium,1271.0,1342.0,250.0


In [594]:
rho, p = spearmanr(Browning_expression['ancient'], Browning_expression['archaic-specific'])
print(rho,p)

0.9588936542275524 4.547848246251568e-19


In [595]:
rho, p = spearmanr(Browning_expression['ancient'], Browning_expression['introgressed'])
print(rho,p)

0.9357169147823144 4.920930114343845e-16


In [596]:
rho, p = spearmanr(Browning_expression['archaic-specific'], Browning_expression['introgressed'])
print(rho,p)

0.9306003291060468 1.6145461837822605e-15


# Genes of Evolutionary Significance <a class = 'anchor' id = 'evolutionarysignificantgenes'></a>

Let's look at SAVs among a few genes of evolutionary significance.

In [597]:
data_2[data_2['annotation']=='OAS1']

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
385314,chr12,113355275,G,T,G,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,1.0,5096.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes,12.0,143230.0,0.000084,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,OAS1,1.0792,-0.42591,0.54623,1.7991,-0.137,0.26,0.0,0.0,0.0,0.26,10,1,10,-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient


In [598]:
data_2[data_2['annotation']=='EPAS1']

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
784210,chr2,46584859,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,12.0,5096.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,yes,21.0,143324.0,0.000147,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,EPAS1,1.0198,-0.15408,0.21127,4.4975,-0.450,0.0,0.00,0.37,0.0,0.37,0,32,0,-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,low-confidence ancient,low-confidence ancient
784257,chr2,46610904,A,G,A,derived,snv,0/1,0/0,0/0,0/0,True,False,False,False,Altai,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,EPAS1,1.0198,-0.15408,0.21127,4.4975,-0.313,0.0,0.23,0.00,0.0,0.23,5,28,-18,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific


In [599]:
data_2[data_2['annotation']=='ERAP2']

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
1181365,chr5,96235896,A,G,A,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,yes,2812.0,5096.0,0.55,0.53,0.52,0.6,0.58,0.53,0.590643,yes,77437.0,142990.0,0.541555,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ERAP2,1.009,-0.07147,0.92859,0.46067,0.460,0.00,0.00,0.00,0.51,0.51,-37,-3,20,-3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,32.0,yes,ancient,ancient
1181375,chr5,96238551,T,G,T,derived,snv,1/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,yes,118.0,5096.0,0.02,0.00,0.06,0.0,0.05,0.02,0.000975,yes,5387.0,143282.0,0.037597,yes,T,G,1.0,0.00318,0.00099,0.05043,0.0,0.05368,0.0,0.02352,T,G,NaN,chr5_95966719_96398083,0.025724,yes,1.0,0.02,ERAP2,1.009,-0.07147,0.92859,0.46067,0.455,0.00,0.00,0.53,0.00,0.53,-41,28,-5,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,yes,introgressed,introgressed
1181403,chr5,96248413,A,G,A,derived,snv,0/1,0/0,0/0,0/0,True,False,False,False,Altai,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ERAP2,1.009,-0.07147,0.92859,0.46067,-0.376,0.24,0.01,0.00,0.00,0.24,-26,2,47,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific


From McCoy et al. 2017

In [600]:
data[data['annotation']=='ADAMTSL3']

,chrom,pos,ref_allele,alt_allele,ancestral_allele,anc_dev,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,present_in_1KG,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,1KG_non_ASW_AFR_AF,present_in_gnomAD,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,Vernot_introgressed,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,Vernot_haplotype_tag,Vernot_introgressed_AF,Browning_introgressed,Browning_ref_alt,Browning_introgressed_AF,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Colon_Sigmoid,Colon_Transverse,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,Heart_Atrial_Appendage,Heart_Left_Ventricle,Kidney_Cortex,Liver,Lung,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,Ovary,Pancreas,Pituitary,Prostate,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,N_GTEx_tissues,sQTL,Vernot_allele_origin,Browning_allele_origin
540636,chr15,84322854,T,G,T,derived,snv,0/0,0/0,./.,0/1,False,False,False,True,Vindija,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ADAMTSL3,0.95678,0.4755,0.5073,4.3946,0.334,0.00,0.00,0.0,0.00,0.00,-11,40,-1,-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
540637,chr15,84324441,G,A,G,derived,snv,0/0,0/1,0/0,0/1,False,True,False,True,Late Neanderthal,yes,743.0,5096.0,0.15,0.06,0.28,0.02,0.37,0.12,0.002924,yes,28823.0,143164.0,0.201329,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ADAMTSL3,0.95678,0.4755,0.5073,4.3946,0.561,0.00,0.00,0.0,0.00,0.00,40,5,13,-32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,yes,ancient,ancient
540638,chr15,84324778,C,T,C,derived,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,ADAMTSL3,0.95678,0.4755,0.5073,4.3946,-0.254,0.00,0.00,0.0,0.00,0.00,14,-31,-30,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,archaic-specific,archaic-specific
540639,chr15,84325970,C,T,C,derived,snv,1/1,1/1,0/0,0/1,True,True,False,True,Neanderthal,yes,738.0,5096.0,0.14,0.06,0.28,0.02,0.37,0.11,0.002924,yes,28868.0,77878.0,0.370682,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,1.0,0.14,ADAMTSL3,0.95678,0.4755,0.5073,4.3946,0.561,0.00,0.00,0.0,0.00,0.00,-2,38,3,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Average SAVs in 1KG <a class = 'anchor' id = 'avg1KGSAVs'></a>

Is the distribution of SAVs among the archaics unusual? Here, we import data on modern humans to which we have applied SpliceAI.

In [601]:
avg_SAV_header = ['chrom','pos','ref_allele','HG00137','HG00338','HG00619','HG01198','HG01281','HG01524','HG01802','HG02142','HG02345','HG02629','HG03060','HG03190','HG03708','HG03711','HG03800','HG04014','NA11830','NA18552','NA18868','NA19011','NA19452','NA19741','NA20537','NA21141','alt_allele','annotation','ag_delta','al_delta','dg_delta','dl_delta','ag_pos','al_pos','dg_pos','dl_pos']
avg_SAV = pd.read_csv('../thousand_genomes/thousand_genomes_avg_SAVs/spliceai_1KG_snvs.txt', sep = '\t', names = avg_SAV_header)
avg_SAV.head(10)

/scratch/cbrand/ipykernel_47833/2307328771.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  avg_SAV = pd.read_csv('../thousand_genomes/thousand_genomes_avg_SAVs/spliceai_1KG_snvs.txt', sep = '\t', names = avg_SAV_header)


,chrom,pos,ref_allele,HG00137,HG00338,HG00619,HG01198,HG01281,HG01524,HG01802,HG02142,HG02345,HG02629,HG03060,HG03190,HG03708,HG03711,HG03800,HG04014,NA11830,NA18552,NA18868,NA19011,NA19452,NA19741,NA20537,NA21141,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
0,10,47460,A,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,C,TUBB8,0.00,0.02,0.0,0.0,10,-22,-43,-22
1,10,47754,T,./.,./.,./.,./.,0|1,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,C,TUBB8,0.09,0.00,0.0,0.0,-1,-2,-1,18
2,10,47792,A,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,G,TUBB8,0.00,0.00,0.0,0.0,34,-8,-20,-7
3,10,47804,T,./.,./.,./.,./.,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,C,TUBB8,0.00,0.00,0.0,0.0,22,-20,-32,0
4,10,47876,C,1|0,0|1,./.,0|1,0|1,1|0,./.,0|1,./.,1|0,1|0,1|0,1|0,0|1,1|0,1|0,0|1,0|1,./.,./.,0|1,0|1,./.,0|1,T,TUBB8,0.03,0.00,0.0,0.0,-7,12,-6,-8
5,10,48005,G,1|0,0|1,./.,0|1,1|0,0|1,./.,0|1,./.,1|0,1|0,0|1,1|0,1|0,1|1,1|0,0|1,0|1,0|1,./.,0|1,0|1,./.,0|1,A,TUBB8,0.00,0.00,0.0,0.0,-7,-31,2,-31
6,10,48078,T,0|1,./.,./.,0|1,0|1,./.,./.,1|0,1|0,./.,./.,./.,0|1,0|1,./.,./.,./.,1|0,./.,./.,./.,./.,1|0,1|0,C,TUBB8,0.00,0.00,0.0,0.0,36,37,-27,37
7,10,48221,T,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,C,TUBB8,0.00,0.14,0.0,0.0,-1,5,5,-2
8,10,48232,G,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,./.,./.,./.,./.,./.,./.,./.,A,TUBB8,0.23,0.14,0.0,0.0,-6,50,-5,50
9,10,48249,C,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,1|0,./.,./.,T,TUBB8,0.14,0.31,0.0,0.0,33,-23,-47,33


In [602]:
avg_SAV = avg_SAV[(avg_SAV['chrom'] != 'X')]
len(avg_SAV)

4582422

In [603]:
avg_SAV = avg_SAV[(avg_SAV['chrom'] != 'X')]
avg_SAV = avg_SAV[(avg_SAV['ag_delta'] >= 0.2) | (avg_SAV['al_delta'] >= 0.2) | (avg_SAV['dg_delta'] >= 0.2) | (avg_SAV['dl_delta'] >= 0.2)]
avg_SAV.head(10)

,chrom,pos,ref_allele,HG00137,HG00338,HG00619,HG01198,HG01281,HG01524,HG01802,HG02142,HG02345,HG02629,HG03060,HG03190,HG03708,HG03711,HG03800,HG04014,NA11830,NA18552,NA18868,NA19011,NA19452,NA19741,NA20537,NA21141,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
8,10,48232,G,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,./.,./.,./.,./.,./.,./.,./.,A,TUBB8,0.23,0.14,0.00,0.00,-6,50,-5,50
9,10,48249,C,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,1|0,./.,./.,T,TUBB8,0.14,0.31,0.00,0.00,33,-23,-47,33
262,10,220176,A,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,./.,./.,./.,G,ZMYND11,0.00,0.00,0.27,0.00,0,39,-1,-32
417,10,288178,C,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,G,DIP2C,0.01,0.28,0.00,0.00,-38,0,-18,1
675,10,349332,T,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,C,DIP2C,0.00,0.00,0.12,0.21,-14,-2,-14,-1
1229,10,487391,A,1|1,1|1,1|1,1|0,1|1,1|1,1|1,1|0,1|1,0|1,./.,1|1,1|1,1|1,0|1,0|1,1|1,1|1,0|1,1|1,./.,1|1,1|1,1|1,G,DIP2C,0.00,0.00,0.03,0.46,-16,2,-16,2
1996,10,662757,T,./.,./.,0|1,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,C,DIP2C,0.00,0.00,0.45,0.00,1,-3,1,-3
2564,10,996086,A,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,G,GTPBP4,0.22,0.00,0.11,0.00,1,20,-49,49
2638,10,1008958,G,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,./.,./.,./.,./.,./.,A,GTPBP4,0.07,0.37,0.00,0.00,25,0,25,0
2648,10,1010007,A,./.,./.,1|1,./.,0|1,./.,1|0,./.,./.,./.,./.,./.,./.,./.,./.,./.,0|1,./.,./.,./.,./.,1|1,./.,./.,G,GTPBP4,0.00,0.00,0.57,0.00,-40,17,-5,-28


In [604]:
len(avg_SAV)

14006

In [605]:
avg_SAV = avg_SAV.replace({'./.': 0, '0|1': 1, '1|0': 1, '1|1': 1})
avg_SAV.head(10)

,chrom,pos,ref_allele,HG00137,HG00338,HG00619,HG01198,HG01281,HG01524,HG01802,HG02142,HG02345,HG02629,HG03060,HG03190,HG03708,HG03711,HG03800,HG04014,NA11830,NA18552,NA18868,NA19011,NA19452,NA19741,NA20537,NA21141,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
8,10,48232,G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,A,TUBB8,0.23,0.14,0.00,0.00,-6,50,-5,50
9,10,48249,C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,T,TUBB8,0.14,0.31,0.00,0.00,33,-23,-47,33
262,10,220176,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,G,ZMYND11,0.00,0.00,0.27,0.00,0,39,-1,-32
417,10,288178,C,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,G,DIP2C,0.01,0.28,0.00,0.00,-38,0,-18,1
675,10,349332,T,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C,DIP2C,0.00,0.00,0.12,0.21,-14,-2,-14,-1
1229,10,487391,A,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,G,DIP2C,0.00,0.00,0.03,0.46,-16,2,-16,2
1996,10,662757,T,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,C,DIP2C,0.00,0.00,0.45,0.00,1,-3,1,-3
2564,10,996086,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,G,GTPBP4,0.22,0.00,0.11,0.00,1,20,-49,49
2638,10,1008958,G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,A,GTPBP4,0.07,0.37,0.00,0.00,25,0,25,0
2648,10,1010007,A,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,G,GTPBP4,0.00,0.00,0.57,0.00,-40,17,-5,-28


In [606]:
avg_SAV[['HG00137','HG00338','HG00619','HG01198','HG01281','HG01524','HG01802','HG02142','HG02345','HG02629','HG03060','HG03190','HG03708','HG03711','HG03800','HG04014','NA11830','NA18552','NA18868','NA19011','NA19452','NA19741','NA20537','NA21141']].sum(axis = 0, skipna = True)

HG00137    3266
HG00338    3344
HG00619    3320
HG01198    3514
HG01281    3493
HG01524    3363
HG01802    3363
HG02142    3349
HG02345    3385
HG02629    4120
HG03060    4160
HG03190    4142
HG03708    3368
HG03711    3379
HG03800    3435
HG04014    3421
NA11830    3200
NA18552    3382
NA18868    4102
NA19011    3321
NA19452    4078
NA19741    3284
NA20537    3168
NA21141    3487
dtype: int64

# Introgressed Reference Alleles <a class = 'anchor' id = 'introgressedrefs'></a>

Let's check if switching the reference and alternate alleles results in the same SpliceAI results for our introgressed variants. 

Load in the original dataframe where the introgressed variants are the reference alleles.

In [607]:
introgressed_ref_tag_header = ['chrom','pos','ref_allele','alt_allele','variant_type','altai_gt','chagyrskaya_gt','denisovan_gt','vindija_gt','altai_gt_boolean','chagyrskaya_gt_boolean','denisovan_gt_boolean','vindija_gt_boolean','distribution','annotation','mis_oe','mis_z','lof_oe','lof_z','phyloP','ag_delta','al_delta','dg_delta','dl_delta','delta_max','ag_pos','al_pos','dg_pos','dl_pos','ancestral_allele','temp','anc_dev','1KG_allele_count','1KG_allele_number','1KG_allele_frequency','1KG_EAS_AF','1KG_EUR_AF','1KG_AFR_AF','1KG_AMR_AF','1KG_SAS_AF','present_in_1KG','1KG_non_ASW_AFR_AF','gnomAD_allele_count','gnomAD_allele_number','gnomAD_allele_frequency','present_in_gnomAD','start','Vernot_ancestral_allele','Vernot_derived_allele','Vernot_ancestral_derived_code','Vernot_AFA_AF','Vernot_AFR_AF','Vernot_AMR_AF','Vernot_EAS_AF','Vernot_EUR_AF','Vernot_PNG_AF','Vernot_SAS_AF','Vernot_Denisovan_base','Vernot_haplotype_tag','Vernot_Neanderthal_base_1','Vernot_Neanderthal_base_2']
introgressed_ref_tag = pd.read_csv('../introgressed_refs/introgressed_ref_tag.txt', sep = '\t', names = introgressed_ref_tag_header)
introgressed_ref_tag.head(10)

,chrom,pos,ref_allele,alt_allele,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2
0,chr1,3046711,T,G,snv,0/1,0/0,0/0,1/1,True,False,False,True,Other,PRDM16,0.87043,1.3471,0.081747,5.9522,0.379,0.0,0.0,0.0,0.00,0.00,-4,35,-24,19,T,T,derived,112.0,5096.0,0.02,0.00,0.06,0.0,0.05,0.01,yes,0.000000,6632.0,143094.0,0.046347,yes,3046710.0,T,G,1.0,0.01274,0.00000,0.04611,0.00000,0.06362,0.00000,0.01125,T,chr1_3015134_3046826,T,G
1,chr1,3046826,G,T,snv,0/1,0/0,0/0,1/1,True,False,False,True,Other,PRDM16,0.87043,1.3471,0.081747,5.9522,0.462,0.0,0.0,0.0,0.00,0.00,-49,-1,11,-1,G,G,derived,112.0,5096.0,0.02,0.00,0.06,0.0,0.05,0.01,yes,0.000000,6626.0,143318.0,0.046233,yes,3046825.0,G,T,1.0,0.01274,0.00000,0.04611,0.00000,0.06362,0.00000,0.01125,G,chr1_3015134_3046826,G,T
2,chr1,3188156,G,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.491,0.0,0.0,0.0,0.00,0.00,-27,9,4,-1,G,G,derived,301.0,5096.0,0.06,0.18,0.00,0.0,0.04,0.08,yes,0.000000,1706.0,143300.0,0.011905,yes,3188155.0,G,A,1.0,0.00318,0.00000,0.04323,0.18254,0.00298,0.07407,0.07975,G,chr1_3066509_3209504,G,A
3,chr1,3199291,C,T,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.297,0.0,0.0,0.0,0.00,0.00,28,30,28,-23,C,C,derived,447.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.10,yes,0.000000,2943.0,143256.0,0.020544,yes,3199290.0,C,T,1.0,0.00955,0.00000,0.12680,0.24603,0.00596,0.11111,0.09509,C,chr1_3066509_3209504,C,T
4,chr1,3199727,C,T,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.436,0.0,0.0,0.0,0.00,0.00,8,44,-2,44,C,C,derived,447.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.10,yes,0.000000,2949.0,143216.0,0.020591,yes,3199726.0,C,T,1.0,0.00955,0.00000,0.12680,0.24603,0.00596,0.11111,0.09509,C,chr1_3066509_3209504,C,T
5,chr1,3209504,G,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.438,0.0,0.0,0.0,0.00,0.00,20,1,-12,-1,G,G,derived,444.0,5096.0,0.09,0.24,0.01,0.0,0.13,0.09,yes,0.000000,2939.0,143312.0,0.020508,yes,3209503.0,G,A,1.0,0.00955,0.00000,0.12680,0.24405,0.00596,0.11111,0.09305,G,chr1_3066509_3209504,G,A
6,chr1,3210305,T,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.379,0.0,0.0,0.0,0.00,0.00,2,7,-4,-21,T,T,derived,443.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.09,yes,0.000000,2939.0,143296.0,0.020510,yes,3210304.0,T,A,1.0,0.00955,0.00000,0.12680,0.24405,0.00596,0.11111,0.09100,T,chr1_3166567_3212428,T,A
7,chr1,3212428,A,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.445,0.0,0.0,0.0,0.01,0.01,3,11,23,11,A,A,derived,452.0,5096.0,0.09,0.26,0.01,0.0,0.13,0.09,yes,0.000000,3291.0,143212.0,0.022980,yes,3212427.0,A,C,1.0,0.00955,0.00000,0.13112,0.25595,0.00696,0.11111,0.08384,A,chr1_3166567_3212428,A,C
8,chr1,3240652,C,T,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.479,0.0,0.0,0.0,0.00,0.00,10,50,29,9,C,C,derived,26.0,5096.0,0.01,0.00,0.00,0.0,0.00,0.02,yes,0.004873,294.0,143304.0,0.002052,yes,3240651.0,C,T,1.0,0.00318,0.00397,0.00144,0.00000,0.00000,0.18519,0.02045,T,chr1_3195401_3250112,C,NaN
9,chr1,3240725,T,C,snv,0/1,0/0,0/0,0/1,True,Fal

In [608]:
len(introgressed_ref_tag)

7977

In [609]:
SAV_introgressed_ref_tag = introgressed_ref_tag[introgressed_ref_tag['delta_max']>=0.2]
len(SAV_introgressed_ref_tag)

26

In [610]:
SAV_introgressed_ref_tag

,chrom,pos,ref_allele,alt_allele,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2
295,chr1,52820680,C,T,snv,0/1,0/0,0/0,0/1,True,False,False,True,Other,CC2D1B,1.12620,-1.00890,0.694430,2.038800,-0.140,0.23,0.00,0.00,0.01,0.23,-22,-40,-22,34,C,C,derived,90.0,5096.0,0.02,0.03,0.00,0.00,0.00,0.05,yes,0.000000,443.0,143266.0,0.003092,yes,52820679.0,C,T,1.0,0.00637,0.00000,0.00000,0.02877,0.00497,0.00000,0.05521,C,chr1_52325475_53062486,C,T
476,chr1,152944501,A,T,snv,0/1,0/0,0/0,0/1,True,False,False,True,Other,SPRR4,1.49550,-1.10600,0.714380,0.442850,0.383,0.23,0.00,0.00,0.00,0.23,13,2,12,-1,A,A,derived,2.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,yes,0.000000,4.0,143152.0,0.000028,yes,152944500.0,A,T,1.0,0.00000,0.00000,0.00000,0.00198,0.00000,0.00000,0.00000,A,chr1_152942848_153033467,A,T
557,chr1,183750131,C,A,snv,0/1,0/0,0/0,0/0,True,False,False,False,Altai,RGL1,0.70413,2.22380,0.177530,5.116100,0.655,0.00,0.00,0.26,0.01,0.26,-7,-11,-7,-11,C,C,derived,50.0,5096.0,0.01,0.05,0.00,0.00,0.00,0.00,yes,0.000000,194.0,143252.0,0.001354,yes,183750130.0,C,A,1.0,0.00000,0.00000,0.00000,0.04861,0.00000,0.01852,0.00000,C,chr1_183411087_183827254,C,A
800,chr1,216945756,C,A,snv,0/1,0/1,0/0,1/1,True,True,False,True,Neanderthal,ESRRG,0.56557,2.55570,0.283790,3.051600,0.655,0.20,0.00,0.00,0.00,0.20,-27,47,-27,48,C,C,derived,169.0,5096.0,0.03,0.14,0.00,0.00,0.01,0.02,yes,0.000000,746.0,143174.0,0.005210,yes,216945755.0,C,A,1.0,0.00000,0.00000,0.01153,0.13988,0.00000,0.00000,0.01840,C,chr1_216944862_216964892,C,A
998,chr10,17373518,T,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,ST8SIA6,1.08980,-0.43129,1.008600,-0.029733,0.533,0.05,0.22,0.00,0.00,0.22,33,-19,30,-19,T,T,derived,56.0,5096.0,0.01,0.00,0.04,0.00,0.02,0.00,yes,0.000000,2946.0,143262.0,0.020564,yes,17373517.0,T,C,1.0,0.01274,0.00000,0.01729,0.00000,0.03479,0.01852,0.00307,T,chr10_17338795_17417730,T,C
1617,chr11,84191111,T,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,DLG2,0.74745,2.07620,0.212280,5.712300,-0.959,0.00,0.00,0.00,0.40,0.40,1,29,1,-1,T,T,derived,12.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.01,yes,0.000000,38.0,143298.0,0.000265,yes,84191110.0,T,C,1.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01227,T,chr11_84051845_84211811,T,C
1669,chr11,99345359,C,G,snv,0/1,0/1,0/0,1/1,True,True,False,True,Neanderthal,CNTN5,1.03930,-0.33557,0.327200,4.624200,-0.229,0.00,0.00,0.67,0.00,0.67,-4,48,-5,37,C,C,derived,177.0,5096.0,0.03,0.11,0.00,0.00,0.00,0.07,yes,0.000000,551.0,143130.0,0.003850,yes,99345358.0,C,G,1.0,0.00318,0.00000,0.00000,0.10615,0.00000,0.11111,0.06748,C,chr11_99309226_99359652,C,G
3230,chr17,14109360,T,G,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,COX10,1.03160,-0.18011,0.290420,2.728300,0.338,0.20,0.04,0.00,0.00,0.20,1,38,50,-23,T,T,derived,155.0,5096.0,0.03,0.08,0.00,0.00,0.08,0.01,yes,0.000000,1030.0,143224.0,0.007192,yes,14109359.0,T,G,1.0,0.00637,0.00000,0.08501,0.08333,0.00000,0.16667,0.00818,G,chr17_13902596_14111504,T,NaN
3473,chr18,60003587,G,A,snv,0/1,1/1,1/1,1/1,True,True,True,True,Shared,TNFRSF11A,0.80508,1.28940,0.330780,3.049700,0.650,0.00,0.31,0.00,0.28,0.31,3,-40,3,33,G,G,derived,444.0,5096.0,0.09,0.00,0.25,0.01,0.12,0.10,yes,0.003899,21727.0,143202.0,0.151723,yes,60003586.0,G,A,1.0,0.01

Now load in the SpliceAI annotated dataframe where these variants were the alternate allele.

In [611]:
swapped_header = ['chrom','pos','ref_allele','altai_gt','chagyrskaya_gt','denisovan_gt','vindija_gt','alt_allele','annotation','ag_delta','al_delta','dg_delta','dl_delta','ag_pos','al_pos','dg_pos','dl_pos']
swapped = pd.read_csv('../introgressed_refs/spliceai_annotated_introgressed_refs.txt', sep = '\t', names = swapped_header)
swapped.head(10)

,chrom,pos,ref_allele,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
0,chr10,1098863,G,1/1,1/1,1/1,1/1,A,WDR37,0.0,0.0,0.0,0.0,-16,-15,-16,45
1,chr10,1113464,T,1/1,1/1,1/1,1/1,A,WDR37,0.0,0.0,0.0,0.0,2,48,18,-6
2,chr10,1175546,G,1/1,1/1,1/1,1/1,A,WDR37,0.0,0.0,0.0,0.0,13,-30,-5,7
3,chr10,1230214,C,1/1,1/1,1/1,1/1,T,ADARB2,0.0,0.0,0.0,0.0,23,-48,45,-22
4,chr10,1460803,A,1/1,1/1,1/1,1/1,C,ADARB2,0.0,0.0,0.0,0.0,45,-8,45,2
5,chr10,1464483,A,1/1,1/1,1/1,1/1,G,ADARB2,0.0,0.0,0.0,0.0,44,-16,-47,37
6,chr10,1682361,A,1/1,1/1,1/1,1/1,T,ADARB2,0.0,0.0,0.0,0.0,31,1,-47,2
7,chr10,1684310,C,1/1,1/1,1/1,1/1,T,ADARB2,0.0,0.0,0.0,0.0,9,1,26,1
8,chr10,1686465,C,1/1,1/1,1/1,1/1,T,ADARB2,0.0,0.0,0.0,0.0,-18,6,3,6
9,chr10,1710328,G,1/1,1/1,1/1,1/1,A,ADARB2,0.0,0.0,0.0,0.0,-22,-41,35,-41


We need to create a delta_max column for comparison.

In [612]:
swapped['delta_max'] = swapped[['ag_delta','al_delta','dg_delta','dl_delta']].max(axis = 1)

In [613]:
len(swapped)

8709

In [614]:
SAV_swapped = swapped[swapped['delta_max']>=0.2]
len(SAV_swapped)

24

Now merge.

In [615]:
introgressed_ref_tag_merge = pd.merge(introgressed_ref_tag, swapped, on = ['chrom','pos','annotation'])
introgressed_ref_tag_merge.head(10)

,chrom,pos,ref_allele_x,alt_allele_x,variant_type,altai_gt_x,chagyrskaya_gt_x,denisovan_gt_x,vindija_gt_x,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta_x,al_delta_x,dg_delta_x,dl_delta_x,delta_max_x,ag_pos_x,al_pos_x,dg_pos_x,dl_pos_x,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,ref_allele_y,altai_gt_y,chagyrskaya_gt_y,denisovan_gt_y,vindija_gt_y,alt_allele_y,ag_delta_y,al_delta_y,dg_delta_y,dl_delta_y,ag_pos_y,al_pos_y,dg_pos_y,dl_pos_y,delta_max_y
0,chr1,3046711,T,G,snv,0/1,0/0,0/0,1/1,True,False,False,True,Other,PRDM16,0.87043,1.3471,0.081747,5.9522,0.379,0.0,0.0,0.0,0.00,0.00,-4,35,-24,19,T,T,derived,112.0,5096.0,0.02,0.00,0.06,0.0,0.05,0.01,yes,0.000000,6632.0,143094.0,0.046347,yes,3046710.0,T,G,1.0,0.01274,0.00000,0.04611,0.00000,0.06362,0.00000,0.01125,T,chr1_3015134_3046826,T,G,G,1/1,1/1,1/1,1/1,T,0.0,0.0,0.00,0.0,35,-4,19,-24,0.00
1,chr1,3046826,G,T,snv,0/1,0/0,0/0,1/1,True,False,False,True,Other,PRDM16,0.87043,1.3471,0.081747,5.9522,0.462,0.0,0.0,0.0,0.00,0.00,-49,-1,11,-1,G,G,derived,112.0,5096.0,0.02,0.00,0.06,0.0,0.05,0.01,yes,0.000000,6626.0,143318.0,0.046233,yes,3046825.0,G,T,1.0,0.01274,0.00000,0.04611,0.00000,0.06362,0.00000,0.01125,G,chr1_3015134_3046826,G,T,T,1/1,1/1,1/1,1/1,G,0.0,0.0,0.00,0.0,-1,-49,-1,11,0.00
2,chr1,3188156,G,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.491,0.0,0.0,0.0,0.00,0.00,-27,9,4,-1,G,G,derived,301.0,5096.0,0.06,0.18,0.00,0.0,0.04,0.08,yes,0.000000,1706.0,143300.0,0.011905,yes,3188155.0,G,A,1.0,0.00318,0.00000,0.04323,0.18254,0.00298,0.07407,0.07975,G,chr1_3066509_3209504,G,A,A,1/1,1/1,1/1,1/1,G,0.0,0.0,0.00,0.0,9,-27,-1,4,0.00
3,chr1,3199291,C,T,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.297,0.0,0.0,0.0,0.00,0.00,28,30,28,-23,C,C,derived,447.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.10,yes,0.000000,2943.0,143256.0,0.020544,yes,3199290.0,C,T,1.0,0.00955,0.00000,0.12680,0.24603,0.00596,0.11111,0.09509,C,chr1_3066509_3209504,C,T,T,1/1,1/1,1/1,1/1,C,0.0,0.0,0.00,0.0,30,28,-23,28,0.00
4,chr1,3199727,C,T,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,0.436,0.0,0.0,0.0,0.00,0.00,8,44,-2,44,C,C,derived,447.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.10,yes,0.000000,2949.0,143216.0,0.020591,yes,3199726.0,C,T,1.0,0.00955,0.00000,0.12680,0.24603,0.00596,0.11111,0.09509,C,chr1_3066509_3209504,C,T,T,1/1,1/1,1/1,1/1,C,0.0,0.0,0.00,0.0,44,8,44,-2,0.00
5,chr1,3209504,G,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.438,0.0,0.0,0.0,0.00,0.00,20,1,-12,-1,G,G,derived,444.0,5096.0,0.09,0.24,0.01,0.0,0.13,0.09,yes,0.000000,2939.0,143312.0,0.020508,yes,3209503.0,G,A,1.0,0.00955,0.00000,0.12680,0.24405,0.00596,0.11111,0.09305,G,chr1_3066509_3209504,G,A,A,1/1,1/1,1/1,1/1,G,0.0,0.0,0.00,0.0,1,20,-1,-12,0.00
6,chr1,3210305,T,A,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.379,0.0,0.0,0.0,0.00,0.00,2,7,-4,-21,T,T,derived,443.0,5096.0,0.09,0.25,0.01,0.0,0.13,0.09,yes,0.000000,2939.0,143296.0,0.020510,yes,3210304.0,T,A,1.0,0.00955,0.00000,0.12680,0.24405,0.00596,0.11111,0.09100,T,chr1_3166567_3212428,T,A,A,1/1,1/1,1/1,1/1,T,0.0,0.0,0.00,0.0,7,2,-21,-4,0.00
7,chr1,3212428,A,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,PRDM16,0.87043,1.3471,0.081747,5.9522,-0.445,0.0,0.0,0.0,0.01,0.01,3,11,23,11,A,A,deriv

In [616]:
len(introgressed_ref_tag_merge[(introgressed_ref_tag_merge['delta_max_x']>=0.2) & (introgressed_ref_tag_merge['delta_max_y']>=0.2)])

24

In [617]:
introgressed_ref_tag_merge[(introgressed_ref_tag_merge['delta_max_x']>=0.2) & (introgressed_ref_tag_merge['delta_max_y']>=0.2)]

,chrom,pos,ref_allele_x,alt_allele_x,variant_type,altai_gt_x,chagyrskaya_gt_x,denisovan_gt_x,vindija_gt_x,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta_x,al_delta_x,dg_delta_x,dl_delta_x,delta_max_x,ag_pos_x,al_pos_x,dg_pos_x,dl_pos_x,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2,ref_allele_y,altai_gt_y,chagyrskaya_gt_y,denisovan_gt_y,vindija_gt_y,alt_allele_y,ag_delta_y,al_delta_y,dg_delta_y,dl_delta_y,ag_pos_y,al_pos_y,dg_pos_y,dl_pos_y,delta_max_y
305,chr1,52820680,C,T,snv,0/1,0/0,0/0,0/1,True,False,False,True,Other,CC2D1B,1.12620,-1.00890,0.694430,2.038800,-0.140,0.23,0.00,0.00,0.01,0.23,-22,-40,-22,34,C,C,derived,90.0,5096.0,0.02,0.03,0.00,0.00,0.00,0.05,yes,0.000000,443.0,143266.0,0.003092,yes,52820679.0,C,T,1.0,0.00637,0.00000,0.00000,0.02877,0.00497,0.00000,0.05521,C,chr1_52325475_53062486,C,T,T,1/1,1/1,1/1,1/1,C,0.00,0.23,0.01,0.00,-40,-22,34,-22,0.23
492,chr1,152944501,A,T,snv,0/1,0/0,0/0,0/1,True,False,False,True,Other,SPRR4,1.49550,-1.10600,0.714380,0.442850,0.383,0.23,0.00,0.00,0.00,0.23,13,2,12,-1,A,A,derived,2.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.00,yes,0.000000,4.0,143152.0,0.000028,yes,152944500.0,A,T,1.0,0.00000,0.00000,0.00000,0.00198,0.00000,0.00000,0.00000,A,chr1_152942848_153033467,A,T,T,1/1,1/1,1/1,1/1,A,0.00,0.23,0.00,0.00,2,13,-1,12,0.23
575,chr1,183750131,C,A,snv,0/1,0/0,0/0,0/0,True,False,False,False,Altai,RGL1,0.70413,2.22380,0.177530,5.116100,0.655,0.00,0.00,0.26,0.01,0.26,-7,-11,-7,-11,C,C,derived,50.0,5096.0,0.01,0.05,0.00,0.00,0.00,0.00,yes,0.000000,194.0,143252.0,0.001354,yes,183750130.0,C,A,1.0,0.00000,0.00000,0.00000,0.04861,0.00000,0.01852,0.00000,C,chr1_183411087_183827254,C,A,A,1/1,1/1,1/1,1/1,C,0.00,0.00,0.01,0.26,-11,-7,-11,-7,0.26
1018,chr10,17373518,T,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,ST8SIA6,1.08980,-0.43129,1.008600,-0.029733,0.533,0.05,0.22,0.00,0.00,0.22,33,-19,30,-19,T,T,derived,56.0,5096.0,0.01,0.00,0.04,0.00,0.02,0.00,yes,0.000000,2946.0,143262.0,0.020564,yes,17373517.0,T,C,1.0,0.01274,0.00000,0.01729,0.00000,0.03479,0.01852,0.00307,T,chr10_17338795_17417730,T,C,C,1/1,1/1,1/1,1/1,T,0.22,0.05,0.00,0.00,-19,33,-19,30,0.22
1655,chr11,84191111,T,C,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,DLG2,0.74745,2.07620,0.212280,5.712300,-0.959,0.00,0.00,0.00,0.40,0.40,1,29,1,-1,T,T,derived,12.0,5096.0,0.00,0.00,0.00,0.00,0.00,0.01,yes,0.000000,38.0,143298.0,0.000265,yes,84191110.0,T,C,1.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01227,T,chr11_84051845_84211811,T,C,C,1/1,1/1,1/1,1/1,T,0.00,0.00,0.40,0.00,29,1,-1,1,0.40
1707,chr11,99345359,C,G,snv,0/1,0/1,0/0,1/1,True,True,False,True,Neanderthal,CNTN5,1.03930,-0.33557,0.327200,4.624200,-0.229,0.00,0.00,0.67,0.00,0.67,-4,48,-5,37,C,C,derived,177.0,5096.0,0.03,0.11,0.00,0.00,0.00,0.07,yes,0.000000,551.0,143130.0,0.003850,yes,99345358.0,C,G,1.0,0.00318,0.00000,0.00000,0.10615,0.00000,0.11111,0.06748,C,chr11_99309226_99359652,C,G,G,1/1,1/1,1/1,1/1,C,0.00,0.00,0.00,0.67,48,-4,37,-5,0.67
3326,chr17,14109360,T,G,snv,0/0,0/0,1/1,0/0,False,False,True,False,Denisovan,COX10,1.03160,-0.18011,0.290420,2.728300,0.338,0.20,0.04,0.00,0.00,0.20,1,38,50,-23,T,T,derived,155.0,5096.0,0.03,0.08,0.00,0.00,0.08,0.01,yes,0.000000,1030.0,143224.0,0.007192,yes,14109359.0,T,G,1.0,0.00637,0.00000,0.08501,0.08333,0.00000,0.16667,0.00818,G,chr17_13902596_14111504,T,NaN,G,1/1,1/1,1/1,1/1,T,0.04,0.21,0.00,0.00,38,1,-23,50,0.21
3577,chr18,60003587,G,A,snv,0/1,1/

Let's identify the two positions where the deltas were different.

In [618]:
diffs = pd.merge(SAV_introgressed_ref_tag[['chrom','pos']], SAV_swapped[['chrom','pos']], how = 'left', indicator=True)
diffs

,chrom,pos,_merge
0,chr1,52820680,both
1,chr1,152944501,both
2,chr1,183750131,both
3,chr1,216945756,left_only
4,chr10,17373518,both
5,chr11,84191111,both
6,chr11,99345359,both
7,chr17,14109360,both
8,chr18,60003587,both
9,chr19,33410289,both


Let's look at the actual deltas. First up is chr1: 216,945,756.

In [619]:
swapped[(swapped['chrom'] == 'chr1') & (swapped['pos'] == 216945756)]

,chrom,pos,ref_allele,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos,delta_max
3553,chr1,216945756,A,1/1,1/1,1/1,1/1,C,ESRRG,0.0,0.19,0.0,0.0,47,-27,48,-27,0.19


In [620]:
SAV_introgressed_ref_tag[(SAV_introgressed_ref_tag['chrom'] == 'chr1') & (SAV_introgressed_ref_tag['pos'] == 216945756)]

,chrom,pos,ref_allele,alt_allele,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2
800,chr1,216945756,C,A,snv,0/1,0/1,0/0,1/1,True,True,False,True,Neanderthal,ESRRG,0.56557,2.5557,0.28379,3.0516,0.655,0.2,0.0,0.0,0.0,0.2,-27,47,-27,48,C,C,derived,169.0,5096.0,0.03,0.14,0.0,0.0,0.01,0.02,yes,0.0,746.0,143174.0,0.00521,yes,216945755.0,C,A,1.0,0.0,0.0,0.01153,0.13988,0.0,0.0,0.0184,C,chr1_216944862_216964892,C,A


Now the other allele: chr7: 157,177,273.

In [621]:
swapped[(swapped['chrom'] == 'chr7') & (swapped['pos'] == 157177273)]

,chrom,pos,ref_allele,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos,delta_max
8039,chr7,157177273,T,1/1,1/1,1/1,1/1,C,DNAJB6,0.0,0.0,0.03,0.16,12,-2,12,-2,0.16


In [622]:
SAV_introgressed_ref_tag[(SAV_introgressed_ref_tag['chrom'] == 'chr7') & (SAV_introgressed_ref_tag['pos'] == 157177273)]

,chrom,pos,ref_allele,alt_allele,variant_type,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,altai_gt_boolean,chagyrskaya_gt_boolean,denisovan_gt_boolean,vindija_gt_boolean,distribution,annotation,mis_oe,mis_z,lof_oe,lof_z,phyloP,ag_delta,al_delta,dg_delta,dl_delta,delta_max,ag_pos,al_pos,dg_pos,dl_pos,ancestral_allele,temp,anc_dev,1KG_allele_count,1KG_allele_number,1KG_allele_frequency,1KG_EAS_AF,1KG_EUR_AF,1KG_AFR_AF,1KG_AMR_AF,1KG_SAS_AF,present_in_1KG,1KG_non_ASW_AFR_AF,gnomAD_allele_count,gnomAD_allele_number,gnomAD_allele_frequency,present_in_gnomAD,start,Vernot_ancestral_allele,Vernot_derived_allele,Vernot_ancestral_derived_code,Vernot_AFA_AF,Vernot_AFR_AF,Vernot_AMR_AF,Vernot_EAS_AF,Vernot_EUR_AF,Vernot_PNG_AF,Vernot_SAS_AF,Vernot_Denisovan_base,Vernot_haplotype_tag,Vernot_Neanderthal_base_1,Vernot_Neanderthal_base_2
7341,chr7,157177273,C,T,snv,0/1,1/1,0/0,1/1,True,True,False,True,Neanderthal,DNAJB6,0.76273,1.1745,0.15377,3.4636,0.455,0.0,0.0,0.31,0.0,0.31,-2,46,-2,2,C,C,derived,57.0,5096.0,0.01,0.05,0.0,0.0,0.0,0.0,yes,0.0,268.0,143270.0,0.001871,yes,157177272.0,C,T,1.0,0.0,0.0,0.0,0.05357,0.0,0.0,0.00204,C,chr7_157116267_157177286,C,T


# Introgressed Fixed Alleles <a class = 'anchor' id = 'introgressedfixed'></a>

Let's take a brief look at the introgressed fixed alleles. 

In [623]:
fixed_header = ['chrom','pos','ref_allele','altai_gt','chagyrskaya_gt','denisovan_gt','vindija_gt','alt_allele','annotation','ag_delta','al_delta','dg_delta','dl_delta','ag_pos','al_pos','dg_pos','dl_pos']
fixed = pd.read_csv('../fixed_refs/spliceai_annotated_fixed_refs.txt', sep = '\t', names = fixed_header)
fixed.head(10)

,chrom,pos,ref_allele,altai_gt,chagyrskaya_gt,denisovan_gt,vindija_gt,alt_allele,annotation,ag_delta,al_delta,dg_delta,dl_delta,ag_pos,al_pos,dg_pos,dl_pos
0,chr1,2998934,C,1/1,1/1,1/1,1/1,T,PRDM16,0.0,0.00,0.0,0.0,17,29,-12,29
1,chr1,2999202,C,1/1,1/1,1/1,1/1,T,PRDM16,0.0,0.00,0.0,0.0,17,33,-20,-3
2,chr1,2999415,C,1/1,1/1,1/1,1/1,T,PRDM16,0.0,0.00,0.0,0.0,14,2,29,8
3,chr1,3000465,G,1/1,1/1,1/1,1/1,A,PRDM16,0.0,0.00,0.0,0.0,32,-25,-20,-34
4,chr1,3046711,G,1/1,1/1,1/1,1/1,T,PRDM16,0.0,0.00,0.0,0.0,35,-4,19,-24
5,chr1,3046826,T,1/1,1/1,1/1,1/1,G,PRDM16,0.0,0.00,0.0,0.0,-1,-49,-1,11
6,chr1,3188156,A,1/1,1/1,1/1,1/1,G,PRDM16,0.0,0.00,0.0,0.0,9,-27,-1,4
7,chr1,3195402,G,1/1,1/1,1/1,1/1,C,PRDM16,0.0,0.04,0.0,0.0,41,-29,-29,31
8,chr1,3199291,T,1/1,1/1,1/1,1/1,C,PRDM16,0.0,0.00,0.0,0.0,30,28,-23,28
9,chr1,3199727,T,1/1,1/1,1/1,1/1,C,PRDM16,0.0,0.00,0.0,0.0,44,8,44,-2


In [624]:
fixed['delta_max'] = fixed[['ag_delta','al_delta','dg_delta','dl_delta']].max(axis = 1)

In [625]:
len(fixed[fixed['delta_max'] >= 0.2])

41